### Data Ingestion from video (Youtube)

In [1]:
import pandas as pd
import youtube_transcript_api
from datetime import timedelta
import srt

#https://www.youtube.com/watch?v=Mot91oWrDX0
- one sample video for data ingestion

In [2]:
videos = {
    "id": ["Mot91oWrDX0"],
    "title": ["BUILD23 - Gary Brecka"]
}

In [3]:
# first expectation test
assert len(videos["title"]) == len(videos["id"])

In [4]:
videos_df = pd.DataFrame.from_dict(videos)
videos_df.index = videos_df["id"]
videos_df = videos_df.drop("id", axis="columns")
videos_df

,title
id,
Mot91oWrDX0,BUILD23 - Gary Brecka


We use the youtube_transcript_api package to pull down the transcripts in a single line of Python.

In [5]:
from youtube_transcript_api import YouTubeTranscriptApi


%time transcripts = [YouTubeTranscriptApi.get_transcript(video_id) for video_id in videos_df.index]

CPU times: user 70.7 ms, sys: 18.4 ms, total: 89.1 ms
Wall time: 874 ms


Conveniently enough, every second of a YouTube video is individually linkable and the transcripts come with timestamps.

But a second of speech is not a useful source.

And by default, the subtitles come "chunked" in time at too fine a grain as well: more like five seconds than the thirty to sixty seconds that it takes to make a reasonable point.

So now, we combine the five-second subtitle timestamps into longer chunks based on character count -- 750 seems to generate nicely sized chunks on our corpus.

In [6]:
transcripts[0][:5]

[{'text': 'this speaker', 'start': 0.12, 'duration': 4.32},
 {'text': 'outperformed Tony Robbins last year he',
  'start': 1.7,
  'duration': 4.78},
 {'text': 'was more popular and people love Tony',
  'start': 4.44,
  'duration': 4.44},
 {'text': "Robbins it's no put in not sailing",
  'start': 6.48,
  'duration': 4.26},
 {'text': "against Tony he's amazing I love the guy",
  'start': 8.88,
  'duration': 4.5}]

In [7]:
TRIGGER_LENGTH = 750  # 30-60 seconds

def merge(subtitles, idx):
    new_content = combine_content(subtitles)

    # preserve start as timedelta
    new_start = seconds_float_to_timedelta(subtitles[0]["start"])
    # merge durations as timedelta
    new_duration = seconds_float_to_timedelta(sum(sub["duration"] for sub in subtitles))

    # combine
    new_end = new_start + new_duration

    return srt.Subtitle(index=idx, start=new_start, end=new_end, content=new_content)


def combine_content(subtitles):
    contents = [subtitle["text"].strip() for subtitle in subtitles]
    return " ".join(contents) + "\n\n"


def get_charcount(subtitle):
    return len(subtitle["text"])


def seconds_float_to_timedelta(x_seconds):
    return timedelta(seconds=x_seconds)


def merge_subtitles(subtitles):
    merged_subtitles = []
    current_chunk, current_length, chunk_idx = [], 0, 1

    for subtitle in subtitles:
        current_chunk.append(subtitle)
        added_length = get_charcount(subtitle)
        new_length = current_length + added_length

        if new_length >= TRIGGER_LENGTH:
            merged_subtitle = merge(current_chunk, chunk_idx)
            merged_subtitles.append(merged_subtitle)
            current_chunk, current_length = [], 0
            chunk_idx += 1
        else:
            current_length = new_length

    if current_chunk:
        merged_subtitle = merge(current_chunk, chunk_idx)
        merged_subtitles.append(merged_subtitle)

    return merged_subtitles


%time subtitle_collections = [merge_subtitles(transcript) for transcript in transcripts]

# get strings as well for quick checks (and easier to write to files)
%time subtitle_strings = [srt.compose(merged_subtitles) for merged_subtitles in subtitle_collections]

CPU times: user 2.23 ms, sys: 224 µs, total: 2.45 ms
Wall time: 2.4 ms
CPU times: user 1.44 ms, sys: 24 µs, total: 1.46 ms
Wall time: 1.73 ms


In [8]:
subtitle_collections[0][:2]

[Subtitle(index=1, start=datetime.timedelta(microseconds=120000), end=datetime.timedelta(seconds=102, microseconds=119000), content="this speaker outperformed Tony Robbins last year he was more popular and people love Tony Robbins it's no put in not sailing against Tony he's amazing I love the guy but people said he was better so let me ask you a question imagine you're living in your dream home you got the cabin for the family the beach house your bills are paid you've built something big but you don't have your health I remember Curtis crawling through his house for months literally sliding on a piece of cardboard his wife would drag him through the house because he couldn't walk how important is our health well our final speaker today is about to rock your world about elite health not just yeah I'm healthy I mean Elite levels of health help me welcome the stage one of my favorite well no no no no no no no\n\n", proprietary=''),
 Subtitle(index=2, start=datetime.timedelta(seconds=50,

In [23]:
gary_text = subtitle_strings[0]

We then add YouTube URLs for those longer subtitles as sources and combine them into a single DataFrame.

In [24]:
with open('gary_timeSt_yt_api.txt','w+') as fh:
          fh.write(gary_text)

In [11]:
base_url_format = "https://www.youtube.com/watch?v={id}"
query_params_format = "&t={start}s"


def create_split_video_df(subtitles, base_url):
    rows = []
    for subtitle in subtitles:
        raw_text = subtitle.content
        text = raw_text.strip()
        start = timestamp_from_timedelta(subtitle.start)
        url = base_url + query_params_format.format(start=start)

        rows.append({"text": text, "source": url})

    video_df = pd.DataFrame.from_records(rows)
    return video_df


def timestamp_from_timedelta(td):
    return int(td.total_seconds())


split_video_dfs = [
    create_split_video_df(subtitles, base_url_format.format(id=video_id))
    for subtitles, video_id in zip(subtitle_collections, videos_df.index)
]

%time split_video_df = pd.concat(split_video_dfs, ignore_index=True)

CPU times: user 310 µs, sys: 685 µs, total: 995 µs
Wall time: 1.24 ms


In [12]:
%time split_video_df

CPU times: user 2 µs, sys: 1 µs, total: 3 µs
Wall time: 8.82 µs


,text,source
0,this speaker outperformed Tony Robbins last ye...,https://www.youtube.com/watch?v=Mot91oWrDX0&t=0s
1,roll the video here we go you're one of my bes...,https://www.youtube.com/watch?v=Mot91oWrDX0&t=50s
2,side of the spreadsheets that I was working on...,https://www.youtube.com/watch?v=Mot91oWrDX0&t=...
3,of you whoa whoa yeah oh my God it's so good t...,https://www.youtube.com/watch?v=Mot91oWrDX0&t=...
4,warrior and how to be a superhuman no Who Want...,https://www.youtube.com/watch?v=Mot91oWrDX0&t=...
...,...,...
84,it's the immune system properly doing its job ...,https://www.youtube.com/watch?v=Mot91oWrDX0&t=...
85,what's causing the pulmonary embolism right so...,https://www.youtube.com/watch?v=Mot91oWrDX0&t=...
86,single person needs to know more about your ow...,https://www.youtube.com/watch?v=Mot91oWrDX0&t=...
87,um and Pulmonary embolisms and embolisms of al...,https://www.youtube.com/watch?v=Mot91oWrDX0&t=...


In [21]:
split_video_df.iloc[0]

text      this speaker outperformed Tony Robbins last ye...
source     https://www.youtube.com/watch?v=Mot91oWrDX0&t=0s
Name: 0, dtype: object

In [15]:
#transcript = ("This speaker outperformed Tony Robbins last year. He was more popular and people love Tony Robbins It's no put is not saying against Tony's amazing. I love the guy, but people said he was better So let me ask you question Imagine you're living in your dream home you get the cabin for the family the beach house your bills are paid You've built something big, but you don't have your health I remember Curtis crawling through his house for months literally sliding on a piece of cardboard his wife would drag up through Thou's because he couldn't walk How important is our health? Well our final speaker today is about to rock your world about elite health not just yeah, I'm healthy I mean elite levels of health help me walk up the stage under my favorite well. No. No. No. No. No. No Roll the video here we go You're one of my best friends. You've been such a good friend of me blah blah blah There's nothing I could ever do for your summer to do that. So I'm gonna introduce you to this guy named Gary Brecker Gary back on our friend. Yeah, Gary. I love Gary by the way. I love Gary. Yeah My name is Gary Brecker. I'm a human biologist researcher in a biohacker for 20 years I spent my lifetime working in the insurance industry predicting mortality to the month Which meant if I got five years of medical records and five years of demographic data The team that I was associated with could actually tell him insurance company how long you had to live to the month After years and years at this type of research and analysis I realized one thing There were human beings on the other side of the spreadsheets that I was working on and that I didn't want to spend another day of my lifetime Predicting how long people were going to live and I wanted to help them live happier healthier more fulfilling lives You see we were putting the circuit through God gave us everything that we need in order to manifest our destiny and thrive Clean clear cognitive energy the waking energy of a tiger this guy is incredible He's amazing. Yeah, and this guy can come in and literally change your life And if we want optimal health we have to educate ourselves and go out and find ways to go inside of our bodies Find the missing raw material that's keeping us from being the absolute best replace that raw material and then thrive And today I am so blessed to share my journey All of my research Oh my god is so good to be back at EXP I Absolutely love you guys Last year I'm let's had a heart attack running around this audience So I'm really excited to be here hats off to the EXP crew for throwing another amazing Amazing conference we got a lot to cover today guys I broke my my talk today down into 15 minute segments I know some of you have heard me speak before some of you haven't for those of you that haven't the first 15 minutes My talk will be very similar to what it was last year the next 15 minutes I'm gonna talk about how every single person in this audience can wake up like a Spartan warrior Can have to superhuman inside of them and I'm gonna bring it out of every person in this audience I'm gonna teach you how to wake up like a Spartan warrior and how to be a super human Well who wants to be a super human Amazing and then we're gonna spend 15 minutes talking about some of the most incredible research in the world right now I just got back from the most humbling retreat of my lifetime I was invited by Richard Branson to Nekker Island with 40 of the top thought leaders from around the world He took people from every industry that he thought were the greatest thought leaders He put us all on an island for four days no phones no recording devices We just shared and collaborated on research that is impacting humanity and some of the most profound discoveries in science That are changing the entire direction of humanity and health and wellness and I'm gonna share some of those with you today Because you have no idea how powerful our mind really is and I'm gonna teach you how to Access an area of your mind that you may not be tapping into every day and it's robbing you of 75% of your potential so we're gonna unlock that in the third 15 minutes and in the last 15 minutes of my talk And I do this every time I take the stage. I make two big bold promises Every time I take a stage and the first promise is if you do what I ask you to do today I will add seven years to the lifespan and the health span of every person in this room Great who wants to do that and The second bold promise is that I will take any ailment That you or a loved one suffers from Any ailment whether it's an autoimmune disease an emotional disorder a mental disorder It's hypothyroid it's hypertension any ailment that you are a loved one suffers from and right here in front of this audience from this stage I will tell you exactly what nutrient is missing from that body that is causing that condition to exist And then I'm gonna take 15 minutes of Q&A from you guys so we've got a lot to cover you guys ready to get going All right awesome So for those of you that don't know me my name's Gary Brekkah I'm a human biologist a researcher. I spent 22 years of my life in mortality research for large life insurance companies and You see large life insurance companies have a database. It's got three hundred and seventy million lives in it At that database that they hold could see the light of day It would permanently change the face of humanity It would up in modern medicine in a way that would be catastrophic You see there's three hundred seventy million lives in this database and what this database has that no other clinical study has no research Study has no university no governmental agency no other place on earth Knows the day the date the time the location and the cause of death for three hundred and seventy million lives So they take that data and they pull it back into people's lives and the team that I was on If we were able to get five years of medical records on you and five years of demographic data We could tell the insurance company how long you had to live to the month and Yes, it's that accurate and if you want to know how accurate insurance companies are just look at what happened in the financial services crisis Right 2008 2009 we had 364 banks fail not one life insurance company failed There's no other financial enterprise on the planet that would take that much risk on one variable 25 million 50 million or a hundred million dollars worth of risk and the only thing that matters is how many more months does this person have left on earth? So I worked in this research department. I read medical records six eight hours a day six days a week for 20 years And I was brainwashed by that industry Because they led me to believe that this was just data that I couldn't make an impact That it didn't matter. I didn't have any influence I didn't put these people in this position. I couldn't get them out of this position It was not my responsibility my responsibility was to take the data and tell my boss how many more months this person had left on earth So they could charge them for a life insurance policy And I realized one day That there were human beings on the other side of those spreadsheets and I made a decision That day to turn in my resignation to leave that industry and spend the balance of my lifetime helping people live healthier happier Longer lives and so that's what I'm here to do today and I'm in the process of building an army because we need an army If the pandemic didn't teach you anything about the intention of our government about the intention of modern medicine about the intention of our officials Then you must have been asleep at the switch so it's not up to them. It's up to us Right it's up to us. So I want you guys to be on my army So I'm not gonna hold any information back from you because we learned two things When I was predicting mortality here's some of the mortality predictions That I did but we learned two things when I was predicting mortality and I say this every single time I take the stage This is the most impactful statement you will ever hear in terms of how many more years you have left on earth You're waking energy your cognitive function the depth of your delta wave of sleep You're waking your water retention your hormone balance the entire trajectory of your life And that is that the presence of oxygen is the absence of disease So I'm gonna talk to you about how we get more of it how we manage it better and how we keep it at a level that helps human beings Thrive and makes this entire biome inhospitable to disease Right so the presence of oxygen is the absence of disease the second thing that we learned Was it nearly every complication that we accept as a consequence of aging? Wake in water retention brain fog ADD ADHD OCD manic depression bi polar all kinds of mental disorders the history of Genetically inherited disease is absolutely none of those things It is a missing raw material from the human body a deficiency that is leading to those diseases You know recently I was invited to speak at a very large age management medicine group conference It was a room this size full of physicians and when I took the stage I could feel like the the judgey Mick judger sinness like if they weren't getting CE credits not a single one of them would have been in the room I mean it was like it was an oppressive level of hatred coming at me and I said Let me ask you guys something. How many physicians in this audience right now? My show of hands how many of you are treating patients for genetically inherited diseases? Genetically inherited diseases like hypertension Like hypothyroid like hypercholestral lemia hypertragalist aridemia even drug and alcohol addiction anxiety depression all of these diseases that run in families and Before I got through five diseases every single hand in the room was up And I said while you have your hands up Understand if you're treating patients for these genetically inherited diseases understand we've mapped the entire human genome We know every gene in the human body can any of you tell me what gene Causes any of those diseases Every hand in the room went down And I said the reason why you can't tell me what gene Causes any of those diseases is because that gene doesn't exist Which means that those diseases do not exist? And then I stood there for 30 seconds and even more hate came to the stage Like the the Hado man momometer was all the way in the red but no one could take issue with that and if you're a physician I invite you to take issue with that too. We do not pass disease from generation to generation That is a lie that we have been sold because we have an industry that is built around Managing disease, but if we understood that what happens is we pass on something called methylation We pass on the inability for our body to take a raw material A vitamin, a mineral, an amino acid, a nutrient that enters our body and convert it to the usable form You see just like we pull gasoline out of the ground But we can't put it into our gas tank because the car doesn't understand that fuel source until I'm sorry crude oil Until it's refined to gasoline. There is not a single compound known to mankind not one There's no vitamin no mineral no amino acid no protein no nutrient of any kind that we put into our bodies That is used in the form that we put it in Without a single exception everything we put into our bodies gets converted by our body into the usable form If you cannot make this conversion You have a deficiency It is this deficiency that leads to those diseases And that deficiency can be fixed So that's what we're going to do today you guys with me All right All right so those are my two bold promises I'll add seven years to your life And I will take any ailment that you were a loved one suffers from and I'll tell you what nutrient is missing from your body In fact, I might even call on some of you guys that that took the gene test last year and supplemented and have had a life changing result If you had a terrible result, please stay quiet But if you had a life changing result I want to hear from you So This the first test that I'm talking about is is a gene test and I'm going to go through this quickly because I covered it last year But so many of us that have these conditions that we chalk up to a consequence of aging are not a consequence of aging So many of you right now more than more than 90% of this audience is walking around at 60% of your true state of normal Even if you think you feel good you have no idea how good normal feels God meant for us to thrive for us to have it all and when we put the right raw materials back into our bodies We start to thrive in ways we never thought possible And so thank you. Whoo. I love you guys. Where was the lady used to give me all the woos last year? Somebody used to give me woos every time I say she'll go And then when I refuse to do some she's like I'll give you a woo So all of these things brain fog, high blood pressure, gut issues all of these things are consequences of these five genetic breaks There are five genes in every human beings body in this room That determine what raw materials you can convert into the usable form and what you can't And when these genes are broken This is where those conditions come from And these can be fixed These can be supplemented for and they can be fixed So we do not need to walk around with the kind of ailments like we do in society If you look at the trends in modern medicine, you know diabetes is a billion dollar annual industry Do you know that? 110 billion dollars a year Do you think that there's a meeting going on somewhere in a big farm a boardroom right now to put that out of business? Oh no You know if you said to me nobody would ever say this but if you did if you said to me Gary write me a diet That would guarantee I get diabetes You know what I would do? I go right to the American Diabetic Association website. I download their dietary guidelines and I'd send them to you So I'll bad that is do it American Diabetic Association. You know what they recommend for type two diabetic first thing in the morning glass of orange juice Bolivot meal with crushed brown sugar and natural honey As a snack yogurt with fruit on the bottom 44 grams of sugar, right? You know what that is? That's an insulin dependency diet Do you know that Pfizer just paid 6.6 billion dollars for arena pharmaceuticals? Do you know what arena pharmaceuticals does? Anybody know Fixes myocarditis pericarditis and diffuse vasculitis as a consequence of vaccine injury That's a fact So we need to understand that sometimes the powers to be don't necessarily have our best interest at heart But God has our best interest at heart and he has given us Everything that we need to thrive and we're going to talk about how we tap into that today Right So this is what this gene test looks like it is a q-tip we stick it in your mouth and we rub your cheek Or you order it and it comes to your house and you rub your cheek and you rub the cheeks of your family members Only get that test for the people you love The other ones will leave you there your life early How is it joke? Um So you know what right before I came up here somebody literally back backstage told me stay away from politics and stay away from the vaccine I've already covered the vaccine so I'll stay away from politics Is there like don't talk about Joe Biden like why would I talk about Joe Biden? Yeah um So I but I can't help thinking you know like if a if a spaceship like landed in the United States You know, they all seem to land in Arizona So that's where all the spaceships land and actually one landed in Vegas recently But if a spaceship like landing in the United States and they got out and said take me to your leader I just think about that once in a while Ah What would they think? We'll just leave it at that so There I covered politics and the vaccine and 40% of the audience still likes me Come on David's here leader So these five genes so what happens when these genes are broken? I put Grant Card down there because that's actually Grant Cardone's um gene breaks One of the reasons why I'm wearing this 10x health shirt is because I fixed those genetic breaks and Grant Cardone It got rid of joint pain and sleeplessness and aches and pains in his body and brain fog and as he realized the Consequences of that in his business. He called me one day and said Gary your stuff works You can be in my office at two o'clock tomorrow afternoon. I'll make you a rich man And I was in his office at one fifty nine the one meeting of my life that I was actually early for Was with that man to make me a rich man. He kept this promise So these five genes everyone he has those when you have these genes that are broken. Let me point some of these out Do you see this gene right here? 44% of you have this gene not working. I won't tell you what the nickname is for that gene Brad, I'm gonna keep my promise. Where are you? Okay, I Went to the speaker meeting this morning. Yes, that is called the MTHFR gene If you ever want to look up research on that gene, please capitalize all of those letters or you will find yourself on some really colorful websites So this gene 44% of you actually have a break in this gene This is one of the most important genes in the human body because it takes folic acid And converts it into methyl folate and while that doesn't sound like a big deal folic acid is the most prevalent nutrient in the human diet folic acid by the way Is an entirely man-made chemical we make it in a laboratory folic acid does not occur anywhere naturally on the surface of the earth You cannot find folic acid in nature We make it in a lab and then we have sold it to the public as a health As a health vitamin something that pregnant women need to have a healthy pregnancy That's completely false. They need methyl folate to have a healthy pregnancy 85% of all postpartum depression Is directly related to the prenatal vitamin There is not a single peer-reviewed published clinical study in the world linking pregnancy hormones to postpartum depression There is tons of research linking that gene to postpartum depression So what happens is when the most prevalent nutrient in the human body cannot be converted into something the body can use You have a deficiency and now what kind of things come from that deficiency? Well the first thing is gut issues You have gut issues and you think you have an allergy But it drives you crazy because you cannot You eat the same thing over and over and sometimes you have a reaction and sometimes you don't Because it has nothing to do with what you're eating has to do with the pace of your gut The speed of your gut is determined by that gene if that gene is broken You have suffered your entire life with gas Floating diarrhea, constipation, irritability, cramping And you've never been able to pinpoint why Because your body is missing a simple raw material called methyl folate If you have this gene right here There is about a 90% chance that you suffer from anxiety And some of you may suffer from crippling anxiety Even borderline depression. You know what anxiety is? Anxiety is a fear of the future. It is an increase in your brain in a category of neurotransmitters called Catacolamines The same neurotransmitters that come out in a fight or flight scenario How many of you suffer from anxiety or know somebody who suffers from anxiety? Holy cow 70% of the room You ask somebody that suffers from anxiety. These three questions I will prove to you That it is coming from that gene And it can be fixed with methylated vitamins And if anyone in this room has either had that problem and it is now fixed or had a loved one who had that problem last year And it is now fixed. Is there anyone in here that would like to tell that story? Because I have ran into a couple people at the pool There is one right there with their hand up Can you come up this microphone? If you start to say something bad I am just going to burn one of your eyes out with this laser point I have already cried on stage Nothing will learn you like losing an eye to a laser pointer Yes so it was last year when you said this talk and my daughter Just crippling anxiety And you know just very inward in everything she did I was all in, ordered it for the whole family So within like three weeks of being on this supplements She was in the horses, it is kind of her way of dealing And all of that And her horse instructor didn't even recognize her She was yelling across the barn and just a totally different person Amazing She joined the cheerleading team at her school Woo And like just wildly different She couldn't even get out of her seat To get on stage when her class mates were just going on stage to graduate Before And she took five of the eight Awards Wow just this past year That's amazing And she was on stage and waving and everybody and just She's on no drug, she's on no pharmaceuticals That little girl took that test and all she's taking is vitamin That's how we're going to change the world That's how we're going to change the world You have this gene right here You have a history of hypertension in your family How many of you either suffer from hypertension or have a loved one suffering from high blood pressure You see high blood pressure is the second leading cause of death in America And most of people that actually suffer from high blood pressure when they go to their doctor They have a normal EKG, they have a normal EEG, they have normal heart and lung sounds, they have a normal Thycone stress study, they have a normal cardiac catheterization All their cardiac workup is normal Why do they have high blood pressure? Their doctor will tell you Because your father had it and your grandfather had it And your great grandmother and their mother's side had it You genetically inherited that hypertension Next time you know what to say Oh, what gene did I inherit that caused that? Watch their face, go blank What happens is when you have this gene mutation You cannot lower an amino acid in your blood called homocysteine If this is a naturally occurring amino acid But genetically we break this amino acid down into another harmless amino acid with quiet's our mind And if we can't quiet our mind now we're told that we have ADHD Or ADD, that's another famous one, or OCD And so if I can't actually lower that amino acid then what happens is it rises in my bloodstream As it circulates around in my bloodstream my arteries clamp down If you make the pipes smaller in a fixed system pressure goes up And now we hold the heart responsible for a crime it hasn't committed When we start pounding on a perfectly healthy organ with beta blockers and calcium Channablockers and diuretics trying to pound a healthy organ into submission For a crime it is not committing and we do this for decades You will never convince me 44% of Americans are so depressed they need to be on an anti-depressant 51% of Americans over 55 years old have a failing heart to the point where it needs medical intervention Or that nearly 60% of Americans suffer from a mental disorder I'll never believe that What I will believe is we are so nutrient deficient that without certain nutrients we get the Expression of those conditions And so if you would lower homocysteine by the way if you ever watched my video with Dana White This is exactly what happened to Dana White He was on three blood pressure medications when I met him 10 weeks later He was on no blood pressure medication he had perfectly normal blood pressure He was off of his anti-depressants and off of his thyroid medication And so we put these raw materials back into the human body And that's how we just restore that vascular Lactity to normal and then the pressure returns to normal Now for you high functioning entrepreneurs if you have that gene broken right there You have what's called consummate over a cheaper profile You are your own worst enemy You hold yourself to a very very high standard Right you put a lot of pressure on yourself You you you essentially Decide the success or the lack thereof of your day by how many things you get done on an agenda that you set for yourself You find yourself very easily disappointed in other people because you feel like they just can't do the basics When you know I know those laughs are like yep, that was me right there. I got spouses pointing at spouses So and when you lay down to go to sleep at night You have a hard time falling asleep not because you're not tired, but because your mind keeps you awake Right, so what happens is as your environment quiet your mind wakes up So what if we could pull all these little anchors up off the ground What if when you lay down you could actually quiet the catacolomines and fall asleep What if you could actually process thought in the right sequence? Which is what we call attention deficit disorder there actually is no such thing as ADD or ADHD They are Woo, I love you got I mean I love you guys Usually people are like boo and that's a true and they need riddle in and they need Adderall and the 85 bands um Interestingly the same people that make folic acid monsanto Also makes vi bands in riddle in so folic acid is the leading cause of ADD ADHD and manic depression And it's treated with riddle in vi bands um and Adderall just to Quintin and it's another quintin. It's a point out for you not being a conspiracy theorist So So what happens is attention deficit disorder is not an attention deficit at all. It's an attention overload disorder It is too many windows open at the same time And so what happens is because we create thought at a faster rate than we dismantle thought the mind gets clouded You think it about a job you're working on your friend walks up So you think it about a job you start talking to your friend while you're talking to your friend You see a logo on their jacket reminds you of vacation you want to take And you think it about a job talking to your friend look into the logo thinking about a vacation you want to take right all at the same time Your friends like hey max, you know my grandmother died on Sunday and you go that's a great idea That's when you know you got ADHD So what if we could just put amino acids like elmathionine back into the body Elmathionine as simple as elmathionine or trimethoglyceine amino acids that are already in your body And you learn to down regulate catacolamines you learn to take those those category of neurotransmitters there Were in your brain and you get to suppress them you get to break down thought so that you can sequence thought properly And now you all of a sudden don't have ADHD because you are not having an overactive mind Modern medicine says if the mind is racing Let's put an infetamine into the body race the central nervous system to match the pace of the mind That creates dependency it creates something called pacaphylaxis that can permanently alter the neuroplasticity of the brain Elmathionine is in your bloodstream right now So if you have that gene mutation you take elmathionine you will notice a demonstrative change in your behavior You will notice a night and day difference between Children that have that MTHFR gene mutation and you take them off of folic acid and put them on a methylated vitamin And you have an entirely new child in the house in three to seven days Happens f-asts Right instead of the school calling you and saying little Johnny's not paying attention You disrupt the lead doesn't finish his work. We need to bring in the riddle in right so Every single person that hasn't gotten that test I want every human being to do that test once in your life. It's about 600 bucks. It's a gene test You put the cheek swab in your mouth. You send it into a lab. They send it back You will never guess what you need to supplement with for the rest of your life You will stop supplementing for the sake of supplementing and you'll start supplementing for deficiency And when you supplement for deficiency you start to thrive And so that's my pitch now Moving to part two because now we're gonna have some real fun Um, I'm gonna talk about something called the superhuman protocol because The superhuman protocol is basically taking everything from mother nature that is good from outside and bringing it in You have two choices you can pay a hundred and fifty thousand dollars Which a lot of our patients do for the superhuman equipment? Or you can do it for free. Which one do you want me to explain to you guys? I knew you guys were cheap. Yeah um So what what is so amazing About the research that is going on now In longevity in AITI aging in biohacking and in optimization And when I was on on on Neckar Island with Richard and these thought leaders It really hit me That everything is now come full circle And we're just getting right back to the basics The further we get away from the basics the thicker we become Human beings were originally built to spend 85% of our time outside The average human spends less than two percent of their lifetime outdoors now So now what's happened when we've come indoors? We've stopped contacting the surface of the earth Do you know that the earth is a giant magnetic field? Think about the last time that you have bare feet touching bare soil dirt, grass or sand That's the last time that you discharged into the earth And this is very real by the way if you follow me on instagram You'll see that I took a voltage meter I plugged it into the dirt in the sand into the into the ground And I held the other pole of this voltage meter And I showed you how there was zero voltage going through my body of standing in tennis shoes Then I took my shoes off and I stepped on the surface of the earth and the pin pegged All the way to the right hand side So what happens when we touch the surface of the earth? It's called earthing. It's called grounding You see we don't want it to be this easy, but it is Right when we touch the surface of the earth we discharge into the earth It actually changes the polarity of the body It can shift the alkalinity in the blood It can change the polarity of 32 trillion cells in the human body In six seconds And we don't touch the surface of the earth anymore What we did was we came up with a PEMF mat Pole selector magnetic field mat that you can put in your bed and get the same thing as grounding But you can take your shoes off and contact the surface of the earth For five minutes three times a week and maintain that same polarity The science has now caught up to the magnetism of the earth And the impact that actually contacting mother earth has on human beings The second thing that we don't get enough of is light The number one nutrient deficiency in the world is vitamin D3 The single most important nutrient in the human body 50% of this audience is clinically deficient in vitamin D3 85% of the African American population in Latinos because of the pigment of their skin Are deficient in vitamin D3 This is why COVID disproportionately affected minorities Had nothing to do with their minority status It has to do with the pigment of the skin The darker your skin the lower the vitamin D3 The lower the vitamin D3 The higher the all-cause mortality And the weaker the immune system In fact one of the worst things that we did during the pandemic Was further weakened our immune system by social distancing Residential quarantining Masking You know what human beings were meant to do They're meant to do what we're doing right now Interact with other human beings Right that veiled a strong immune system What am I saying? Favorite saying is that aging is the aggressive pursuit of comfort The more aggressively we pursue comfort the faster we age The reason why most of us don't exercise or take cold showers or do cold plunges Or like to diet and deal with a little pang of hungers Because we don't like to be uncomfortable And more comfort we seek the faster we age We've got to stop telling grandma not to go outside It's too hot Not to go outside It's too cold Just to lay down Just to relax To eat at the first pang of hunger That's all nonsense Right The body has a response called hormesis where you stress it and it strengthens And we're going to talk about how we can tap into that So the second thing is light We are light beings The truth is most of us are not getting enough sun It's not that we're getting too little sun So when I wake up in the morning the very first thing I do And I don't care how cold or hot or rainy Or windy it is outside I go outside and a pair of shorts my neighbors think I'm nuts I go out and put tight and a pair of shorts Don't I every single day And you can ask my wife And I go out and I take my shirt off And I get 15 to 20 minutes of sunlight And I do 8 minutes of deep breath work Every single day like clockwork It will change the trajectory of your life Right And then the last thing that we talk about is oxygen So you can use a hypermax oxygen system Or you can learn to do breath work And there's several styles of breath work All kinds of breath work that's out there Last year we talked about breath work There is a style of breath work called Haka breathing How many of you have ever heard of that Woo Oh, is that because you guys follow me or have you Actually know what that is The Haka dance had you ever seen like the all blacks in Australia They do those crazy dances before rugby games You know why they do that It increases the oxygen content It improves physical performance It raises their emotional state It elevates their mood And it is pain modulatory Right They actually don't feel as much pain during contact So how would you guys like to learn a 60 second breath work technique So you could wake up like a Spartan warrior every single morning You guys want to do it right now? All right Because um So after this breath work technique They are definitely going to think we're a cult Right So you know the spaceships come in We're all going to heaven Right Um I knew there was something weird about that crew Now they're definitely going to think it So what does has to do with is you know Breath work does several things for us What it's going to do It's going to change the oxygen tension in our tissue It's going to change the ratio of gas in our tissue The second thing is to do it's going to raise our emotional state It's going to elevate our mood Because believe it or not Every emotion that a human being can experience Every single emotion That is an elevated emotion Passion, elation, joy, arousal, libido All the heck yeah one the lottery emotions If you actually looked at that molecule It has oxygen as a component of its structure If you pull oxygen out of that molecular structure You can't have that emotion This is why no human being has ever woken up laughing But you can wake up angry Why can you wake up angry but not wake up laughing You don't have the oxidative state to experience laughter But anger, vengeance, despair, jealousy Depression require zero oxygen Right If you want to do a fun little experiment tonight Wait for your spouse to sit in a deep sleep And give him a good deep pinch Watch him wake up really angry Well Gary was right I was just checking it out You know That's not on me You didn't wake up laughing say He was right Really smart Human biologist So If we want to reach elevated emotional tears We need to have the oxidative state To reach elevated emotional tears So that's what we're going to do The fourth thing that it does is it releases sertuins How many of you have ever heard of a sertuin No one as sertuin is It's a survival gene It happens when we slip under the ice It happens when we get too close to a fire It happens when a lion gets too close to us And threatens us They are called sertuins These scour the body of free radical oxidation They increase the rate of protein synthesis And you can access them for free Every single day You guys want to do it All right It's going to get a little weird in here All right So you're just going to sit You put first we'll just put our hands Comfortably on our laps Ben What I want you to do Men Is I want you to make a fist And when I tell you We're going to tap up in our chest like this You are not trying to knock your boob over your shoulder All right It's like somehow it becomes like a man kinder It's like Hit themself in the face You're going to tag you And to take your fist And you're going to tap up on your breasts like this And we're going to do it four times Wait till I started We're all going to do it together We're going to take a deep breath in like this And when we breathe out We're going to say the word Ha Four times and we're just going to tap our chest like this Ha Women You're going to cross your hands Raise them up and you're going to tap Your fists on your shoulder Okay, and we're going to take a deep breath in And on the way out You're going to say Ha Ha Four times I know this sounds crazy Way too We do three of these breasts And way too You see how you feel You're going to release your tunes We're going to raise our emotional state We're going to elevate our mood We're going to flood the brain with oxygen You guys ready to go? All right I want an obnoxious breath in We're going to do three rounds of this Ready? A deep breath in Ha Ha Ha Ha Ha Ha Ha Ha Ha Ha Ha Ha Ha Woo Yeah Now aside from offending your neighbors And everybody else in the hotel If you actually did that on your balcony Every morning You would wake up Like a Spartan warrior Right My poor wife has to listen to it every day And then I'm so chit chatty when I'm done She likes to sleep in I come in I want to tell her all about my day And Get stuff off my chest I used to have a whole Instagram special It was called Good Morning Babe And right after I would do breath work And Haka breathing I would go in and wake her up And go Ooh Good Chakka And dance on the bed She won time on Instagram And front of half a million people She threatened to stab me in the neck You did I still have the video So So in any case So we're going to get oxygen into our bodies I'm going to talk about supplements in a second Because you know I don't want anybody to just start taking supplements For the sake of supplementing I want you to do the gene test We have a booth outside By the way the 10X health booth I want you to go to that booth And talk to those guys about How to get that gene test for you and your family Once And then never guess on what you need to supplement with And now I want to talk about the brain Because Joe dispensa And MIT Woo you guys like Joe dispensa All right Oh man I really love you guys We're working on filming a documentary Called The Frequency of Truth And part of the Part of the documentary is about The power of the human brain Which we are only now Beginning to understand God is so much more powerful than modern medicine That it's unbelievable The capability that this has To influence the environment around it To heal the environment around it To heal itself is absolutely uncanny In Scandinavia they just did a clinical trial A clinical study On terminally ill cancer patients On by the way any of the studies that I ever talk about from the stage If you'd like to see those They're on my link tree on Instagram Or I'm happy to send you these articles I am a science nerd So if you ever would like to know what study I'm referring to I'm happy to send that study to you But what they did was they took two groups of people Both of these people Or diagnosed with the same terminal illness Only they took one into one room and one into another And with one placebo group They told them the truth about their illness That it was terminal The percentage chance of five years survival And that modern medicine had exhausted Everything that they had And there was essentially nothing they could do Other than to extend them palliative care Hospice palliative care They took an identical group of people Into another room And they said You have this type of cancer But luckily for you you qualify For clinical trial And during the in this trial We've had a 100% Churrate 100% of the patients that have qualified for this trial Have gone into remission And you're so lucky You have the tissue type that matches this trial And so there's virtually a 100% chance That you're going to go into remission What they measured Was the shift in frequency The shift in that person's emotional state And what we realize now Is that there's an area in the brain called the amygdala When you feel emotion Any mood Any emotional state that you can feel You are experiencing that emotion In an area of the brain called the amygdala Okay, what's special about this area Of you're happy if you're related If you're upset If you're angry You're feeling that In that area of the brain called the amygdala What's interesting about the amygdala Is that it is the only gateway To the hippocampus To the memory There is only one hallway That you can go down to reach your memory And that is through the amygdala Which means your emotional state Determines what memories you retrieve Well your memory links to the prefrontal cortex Your conscience And your conscience is your future And so what this means is that your current emotional state determines your future If you do not learn to master your emotional state You will never take control of your future So So my whole reason for existing in life is because I want to elevate the emotional state of humanity I want to elevate the mood of humanity By putting the right raw materials into your body So you can manufacture the neurotransmitters To stay in an elevated state So you retrieve healthy memories You think healthy thoughts And you project a very healthy future Right And so imagine the emotional state of someone Who was just told they had a terminal illness Versus the emotional state of somebody Who was actually told There's virtually a 100% chance they're going to live Just compare those two emotional states And imagine what memories they started to access And then imagine what their conscience began to think of And imagine what happened to their future 47% of terminally old patients went into spontaneous remission That's almost half of the patients in that study Right this is the power of frequency This is the power of the human body This is the power of the mind We can actually use this to our advantage to influence the world In a positive way around us Last year you might have heard me talk about the Spain scale of emotion The scale of positive and negative emotion Right where they took participants They put them in a room It's called a Faraday's cage It's the void of frequency They started to measure frequency Leaving a human being's body And what they realized Was that the most powerful frequency To leave a human's being's body Somebody yell it out Authenticity If you want to influence The world around you Everything from your children To the people that you interact with To a client you're trying to gain a listing from From a team you're trying to motivate From a spouse you're trying to convince From a child you're trying to discipline If you can master the frequency If you can emanate the frequency of authenticity You can actually change their emotional state This is a fact By-o-a You know we say that women Are more intuitive Have you heard that? Okay that's actually not true Women are more sensitive to frequency We call it intuition Is a sensitivity to frequency I can't even tell you how many times my wife has said Nope not that guy Something's a little magic can't put a finger on it But I don't like him I'm like the babe get a shoes and a s- Got a nice suit Drobopharari How bad could he be In six months later I'm taking the knife out of my back Right how many of you guys can identify with that Right so you are sensitive to this frequency of authenticity It's the most powerful frequency to leave a human being's body There's a law in physics called constructive interference Look it up if you'd like It says if two frequencies of equal wavelength Me the size of the frequency doubles This proves the universal law of attraction Right it's a it's a universal law of attraction Is actually a law in physics So but the point is most of us don't understand How to master this in our lives So I'm actually going to teach you a technique To master our emotion To pull from the right memory To project the proper consciousness To change our future And it is powerful enough That when you use it in business And in life and in your career And in your relationships You will see the entire trajectory of these change It is so powerful that it will influence The decision making capacity of another individual Because we know now That emotion is what triggers our memory And memory triggers our conscience And our conscience is our future We also know What percentage by the way I'll just ask you What percentage of decision making is emotional By the way this is just to publish in the journal of neuropsychology What percentage of decision making is emotional 95 100 not not quite 100 maybe Maybe with my wife it's 100 But um 75 to 78 percent of decision making is emotional So if what should be the most important thing that you do Before you get on a phone call To or a team meeting or a zoom to motivate your team You should first of all I should master your own emotion I'm going to tell you how you do that You should try to impact the emotional state Of the people that you're that you're talking to In fact if I could teach you how to influence the emotional state Of a client Then when you get went to get a listing If you actually trigger the right emotion in that client That makes them recall the proper memory They will decide to give you that listing So how many of you would like to do that All right, so let's talk about how we tap into this Right and and this is um you know I'm far along right now, but this is what's about to be Published in our in our documentary So first of all these areas of the brain need certain amino acids They need certain vitamins. They need certain nutrients Because this entire cascade happens by neurotransmitters So the second way that you can tap into it Is to get yourself in a state of authenticity Authenticity only occurs when two things are present Number one when your words are truthful And number two when you believe what you're saying If either one of those is not present You cannot actually emanate the frequency of authenticity And by the way this is part of the clinical study The spain scale of emotion I'm happy to send that to you So this is what some women pick up on Because somebody's words are not truthful Or they don't believe what they're saying And we don't pick up on that but very often women will pick up on that And say there's just something that's not adding up here Because that person is not emanating the frequency of authenticity So if you want to shift into an authentic state I want you to do this before your next team meeting I want you to do this before your next client meeting I want you to do this before your next big event So that you can actually have influence over that person in a positive way Okay, ready? So first it starts with intention setting And second it moves to something called visualization But only about 5% of the population really understands the power of visualization When I work with athletes and top fighters in the UFC and running backs in the NFL Part of what I do is I teach them how to visualize outcomes Never ever ever visualize the journey If you visualize the journey you will set yourself up for failure If you visualize the destination you will set yourself up for success This is why I love being a man of faith Because it's not only delayed gratification but I already know where I'm going So I know what the destination is So all that stuff in the middle doesn't bother me so much But so here's how we do it How do we set our intention? So for example before I got on this stage today I sat quietly in a chair back there Close my eyes I took 3D breasts like this And I thought about what was my intention of getting on this stage today My intention was to add value to your life Right so that's the only reason why I'm here right now For the next 60 to 90 minutes of my life I said I'm going to give everything that I can So I can provide value to this audience And then I created a picture of every person in this room hearing my words Believing those words agreeing to put those words into action Of thanking me for speaking and I actually lived that moment Right and then when I got up here I didn't have any fear And my words were truthful and I believed what I was saying So I was able to deliver an authentic message And we can use the same thing in business We can use the same thing when we're getting on a call to motivate our team So the first thing we do is we take 3 put your hands just like this Everybody just close your eyes And we're going to take 3 deep breasts through our nose and we're going to let it out through our mouth ready Second one Third one Now in this state This is where we set our intention What is my intention of this team meeting To intend to motivate my team I tend to inspire my team Well message to I want them to receive Only use one word to describe your intention And once you know what your intention is for this meeting Right I want to gain this listing I want this person to give me the trusting confidence to sign over their home To me and my team I want to motivate my team I want to take the people that are struggling I want to bring them back into the herd So you set the intention The second thing you do is you visualize the outcome And this is where visualization goes wrong I want you to create a picture of the end destination The end of that meeting Everybody thanking you for being inspiring today Thanking you for the message that you gave them I want you to see that client shaking your hand congratulating you on gaining that listing Because they had the trust and the confidence in you And then I want you to open your eyes And now that you've set your intention And you visualize that you know what you've done You've set your emotional state You're now on an authentic state Guess what happens in your hippocampus Your hippocampus records everything in a picture Memory is stored in pictures When you hear a song that reminds you of a loved one It's not the song that's in your memory It's the image that's in your memory That song triggered an emotion And that emotion linked to your memory And then you form the picture of that last time That you are around that that loved one Right? That's why you can be driving down the road You can hear a country music song And all of a sudden tears come to your eyes Because it reminds you of your mother or your father Or a loved one that's left your life Right? That is an emotion triggering a picture So if you actually change your emotional state Through breathwork Set your intention which sets your emotion Form a picture Now what happens to your conscience in your future Your brain starts to live that And you will emanate the frequency of authenticity Authenticity will hit that person's body Authenticity will match the frequency in their body It will raise their elevated emotional state And they will have no idea why they trust you so much Why they are so confident in you But they will give you that listing Your team will be inspired You will find that your objection hits with more meaning And so that's how we start to set our self up For these kinds of events Try it it takes about 60 seconds to shift into that state Wait till you see the Hormetic research that is coming out on this On the capacity for human beings to influence the world around them You know in the animal world This is how animals are eliminated and killed And how they seek safety and stay together If you want, if you look at how dolphins use sonar If a fish separates from the school They can send sonar into a school of fish And find the one fish out of 15,000 fish That is not emanating the right frequency And they can go right into that school of fish and hit that fish When a lion is hunting a herd They look for the weakest animal And they separate that weakest animal And human beings we can have the same kind of influence On one another by emanating the frequency of authenticity Did you guys find that helpful? All right, woo! All right, hi tech, you know, don't talk about science Don't talk about politics, don't talk about the vaccine I think we covered it all Like any other sensitive subjects we need to go over Um So, everybody asked me what my morning routine is I'm going to run through my morning routine I'm going to teach you how to wake up like a sport and warrior Woo! And I'm going to tell you three things you need to take out of your diet Three things you need to put into your day So you can start to become a superhuman These are all for free And then I'm going to open it up for questions And testimonials, there's a microphone right there And I think there's a microphone over here Yeah, there's a microphone right here Um, if you have questions And because if you have a question And I take that answer Probably people in the audience have the same answer My morning routine is very, very simple I wake up in the morning I find sunlight I go out in a pair of shorts I sit in sunlight I do eight minutes of breath work Every single day My wife will tell you I never, ever, ever, ever miss I will miss a commercial flight Not to miss breath work I didn't miss it this morning I don't miss it when I travel You can actually set your circadian rhythm by this So when you change time zones If you breathe at the same time Every morning your body knows this is the time to wake up I clear catacolomines from my brain I make my body inhospitable to disease To pathology I raise my mood I raise my emotional state And I get it all from God and get it all for free All right Woo! So first thing in the morning I get natural sunlight And I do an eight minute breath work routine And it looks just like the breath work That you guys just did Sometimes they do the hawk breathing Unless somebody is sleeping And I don't like to wake them up But I do three rounds of 30 breaths Deep breaths I do 30 of those breaths I hold my breath as long as I can I take a deep breath in I do 30 more of those breaths I hold my breath as long as I can I take a deep breath in I do 30 breaths Hold Take a deep breath in And I'm done It takes about eight minutes to do that breath work It takes a lot longer now I can actually hold my breath for four minutes In between rounds of breath work When I started I could only hold my breath for 30 seconds Now I can do 30 breaths and I can sit for four minutes So that's what takes the longest amount of time So I do breath work Then I go out and I get it I get first light I do my breath work I get ten ounces of water by the way First thing I want you to get out of your diet I'm kind of jumping ahead here is Tap water Never ever ever ever again drink water out of the tap Right Whoo And the reason for this is look if you don't If if having a water filtration system is not in the budget Get a get a bitter water filter that at a minimum gets chlorine and fluoride out of your water If you have seen the clinical trials the clinical studies in the last 15 months coming out on fluoride They are absolutely devastating They're devastating harbored university just launched a study through the Freedom of Information Act We sued the CDC and the FDA and we actually got the research That they kept suppressed on on water if you don't believe you can go to the National Institute of Health Look up the study yourself There is zero levels of safe fluoride In human beings It's a neurotoxin They find that the higher the fluoride concentration in the water the lower the IQ In every state where they studied this So stop drinking tap water if you need to just go buy a six or ten dollar Britter water filter if you want to know the filtration system that I use There's a link on my Instagram I don't have any affiliation with this company But I put everything that I use in my own life there so I want you to get tap water What's that? Oh, welles are welles are fine Yeah, so well and septic is fine. It's the tap water that's bad. It's the concentration of fluoride I try to ground myself every day If I can't ground myself I lay on a PMF mat If you if you don't want to spend the money on a pulse-electromagnetic field mat Take your shoes off and touch the earth for 10 minutes three times a week I know the sound simple, you know, I talk about breath work and sunlight and cold showers and things It seems so simple that people don't want to do it I promise it can change the trajectory of your life I do a cold plunge if you don't have a cold plunge Turn the water as cold as you can make it step into it for a minute and just deal with it Stop pursuing comfort by yeah, for stop aggressively aging by pursuing comfort Okay, so that is my morning routine I would really like to open up the The room for to questions any ailment that you are a loved one suffers from and maybe if you could come up to the microphone If you're comfortable enough to ask the question in front of this group I'd really love to answer it for you Because this is this is where the real value comes into play. Oh, and then I take my supplements Oh, what's the two other things that I want you to get out of your diet So first is I want you to stop drinking tap water the second is I want you to get rid of seed oils Seed oils I got censored all over Instagram my entire account deleted by the fact checkers for it I never said seed oils are bad. I said industrial processed seed oils are bad So let's just cover this for 10 seconds you take a canola oil plant you press it it comes out like gum So how do you degum it they use hexane a neurotoxin to degum it then they take degummed Neurotoxic oil and they heat it to 405 degrees that makes it rancid now It smells so bad they can't bottle it so they deodorize it with with sodium chloride which is a known Carcinogen now it's deodorized, but it's cloudy so they make it clear by bleaching it with chlorine So they take a neurotoxin they add a carcinogen and then they bleach it with chlorine And then they put it on the shelf and they serve it to you You need four oils in your life You need a coconut oil for cooking a ghee butter a grass-fed butter or tallow You need an olive oil and an avocado oil at room temperature. That's it That's all you need Right so four to cook and two to use for salads and and dressings and everything else if it's a palm oil a current A sunflower oil a sunflower oil guys these are rancid toxic inflammatory Neuroinhibitory chemicals that we're putting into our body so I want you to get those out of your body as well yes, sir My dad has ankle-osin spondylitis where the spine is fusing itself together What would you say for something like that? Yeah, so ankle-osin spondylitis is also known as canoax spine Right, so it's actually where if you actually look at the spine on the next rate it'll look like canoax It'll look like those are dripping. This is actually not an issue with the bone It's an issue right above the disco space with something called the periosteum and if you actually Google Neonic stem cell and Enkle-osin spondylitis thesis. I know that's a big word But if you but I want you to put the word amneonic stem cell in front of it You're going to see some of the most astounding clinical trials where they're actually able to repair the periosteal layer of the bone and Stop this waxy dripping stop the spine from actually starting to disintegrate in on itself We always used to think that once these pathologies had started that they were irreversible We know now that that is absolutely not true Right, we don't just have to manage disease. We can stop it and reverse it Um, so if you like to see me after the breakout. I'll give you some links to some of the research and some of the places Um in California that are actually treating that Yeah naturally Whoo. Yes ma'am So two questions last year we were here and my husband stood up and um he has Parkinson's disease And I don't know if you're able to answer that question last year. I don't recall. Yes This second part of it is I Ha you talked about genetic diseases things like that But I had a son who passed away of adrenal leukodistrophy Which is an ex-linked genetic disease that is a true genetic disease And then I have a son who's now 27 who test positive for that but hasn't shown any symptoms And I guess I'm just wondering what It what can be done like if it's ex-linked versus genetic like you talked about and then what is possible for Yeah, so let's let me answer the first one real quick and then only into the second one So the first one with Parkinson's so Parkinson's is is a neuroinflammatory disorder, right? I mean at the end of the day, this is inflammation in the nerves So if we can reduce the inflammatory cascade, right and we can heal the bile in in the nerve Um, then we can stop or even reverse the the the symptoms of Parkinson's We also know now that Parkinson's disease is linked to the gut bacteria There are actually gases that are emitted from gut bacteria when we are absent certain flora We are absent certain um, we're absent certain gut flora. We're absent certain necessary gases in our gut And and these actually go up to the brain and they cause neural inflammation You would not believe how much of our health and wellness is actually related to the health of our gut When we destroy our gut with antibiotics and we destroy our gut with artificial sweeteners Which by the way, you know, aspartame is about to be named by the World Health Organization Is a known carcinogen. It's about to be outlawed in all European countries It will stay in the United States for 10 years Right because we're the only governmental agency in the world that uses something called single-dose toxicity to decide whether or not we can poison our population Right, we don't use cumulative dose toxicity. We use single-dose toxicity So Parkinson's I'd love to give you some of the research on on on Parkinson's There's something called vinyarest veritrol. There's another product called stem region Which is very has shows extreme promise in treating Parkinson's disorder But there's also a product called TSO1000 which repopulates the gut specifically with the floor of the combats The neuron flammatory condition in Parkinson's So we did those two tests last year. I mean not to we each did your test And you said it's one time. Does that mean that the results will never change? The results will never change. The genes are born with or the genes you die with. That's why that test is so important Because now you're supplementing for deficiency. Remember that DNA exists in the exact same form in every single cell in the human body If it's broken in one cell it's broken in every cell So there's nothing more impactful for human beings than changing our DNA So supplying our DNA with the raw material it needs to do its job Right, I mean DNA only has two roles replication and transcription Created perfect copy of itself and give commands to the cell like a CEO So when the DNA is not giving the right command the cell dysfunctions when the DNA doesn't replicate person Perfectly, this is where you get dysfunction all cancer All cancer is DNA replication run amok If you didn't have DNA replication was not impaired you could not have cancer Right, and this is the same issue with the vaccine right? We took a message from the nucleus of the cell called the messenger RNA And we made a permanent copy of it And we injected this into human beings and now that permanent copy never starts giving it stops giving the command to the cell When the DNA gives a command it He viscerates in two hours when a synthetic messenger RNA enters the human body It never turns off the signal the cell never stops producing the nucleic acid protein Right, so we're only now beginning to know the consequences of that But I'd love to talk to you more about this The second part, I don't know Yeah, I'd love to talk to you more about the Parkinson's offline So we can just get to a few more questions And maybe about my son's disease too when you No question Yeah Yes ma'am That was serious mine is kind of vague You're like how do I lose belly fat in the future? Oh man, no it's actually a disease I'm curious about rosacea I was diagnosed with that and they told me there's no cure It's an autoimmune disease I call pull Yes So autoimmune disease as a general category is actually not autoimmune disease It's autoimmune weakness All autoimmune disease is not it's a progressive dysfunction of the immune system because it's too weak Right, it's a weakening of the immune system And so autoimmune conditions generally are where the immune system is dysfunctional Right, it doesn't know the difference between friendly tissue and foreign tissue But what makes it incapable of recognizing Friendlier foreign tissue whether it's rheumatoid or chagrin Or any number of autoimmune conditions is because the immune system is so weak So we have a theory in modern medicine that we should weaken the immune system to keep it from attacking itself We should actually strengthen the immune system So it can properly tell the difference between healthy tissue and foreign tissue We know now for example Crohn's disease which we call an autoimmune disease It's not the immune system dysfunctioning It's the immune system properly doing its job Attacking bacteria in the lumen a wall the intestine But we try to shut the immune system off instead of getting the bacteria out of the wall Okay, so we're sunshine more sunshine more dirt on the feet No, so so rosacea yeah most sunshine more dirt on the feet Breathwork sunshine dirt on the feet and cold showers but Rosacea is a secondary route of waste elimination for the liver So you want to heal the liver with glutathione Thank you so much. Yeah. Oh, yeah, yes sir. All right Gary Are you guys liking this? I Know this guy got about to you know a few more minutes And I really want to add some value. Yes, sir pulmonary embolite So pulmonary embolism is now tons of what's causing the pulmonary embolism right? So being deep vein thrombosis are a big cause of pulmonary embolism I actually sage and I just actually flew to Dubai a few months ago when we got back one of the passengers That we flew with actually dropped dead in the in the baggage claim area from a pulmonary embolism from it from what's called a deep vein thrombosis I did 30 minutes of CPR on them while I thought well U.S. Customs just stood around and just washed me no one even knew what an AAD was for the wall But so when you travel you'll get where compression gear especially if you're on a flight over five hours But one of the best ways to avoid deep vein thrombosis and pulmonary embolisms is to make sure that your blood stays at the right viscosity This is why I think every single person needs to know more about your own Biome than you do about your business Like when I go up to you and you can tell me your income statement your balance sheet and your PNL But you can't tell me your blood sugar your hormone levels or your nutrient deficiencies That tells me where You're oriented right this is our temple You guys should get information on your temple And you should feed it with things that serve your temple don't steal from it Right because we often wonder why we're in this condition and then you ask them well What what is your sugar bin like what have your hormones been like what has you what what nutrients are your body missing? They have no idea they find out when disease or pathology or embolism strike so the best way to avoid dvts and and And pulmonary embolisms and embolisms of all kinds is to keep your hematocrit level your blood viscosity at you know the right At the right viscosity and you can check that on on a blood test. Okay, right? Yeah, right now it's been blood thinners because there's no evidence of any deep throm deep vein thrombosis or anything else so A lifetime of blood thinners as a defense mechanism no interest in staying on those So there's something called veneer vestra veritrol and there are uh Or other things you can do like regular blood donations to keep your blood thin and let's talk about it at the Let's talk about it at the break because I've got to close things off right here But guys, I appreciate you so much. Thank you so much for listening to me today I hope you feel like you got some value out of today. I love you guys Thank you. Thank you. Thank you")

In [16]:
#transcript[:500]

In [ ]:
#whisper_vL3 = "This speaker outperformed Tony Robbins last year. He was more popular, and people love Tony Robbins. It's no put, I'm not saying anything against Tony. He's amazing, I love the guy. But people said he was better. So let me ask you a question. Imagine you're living in your dream home. You got the cabin for the family, the beach house. Your bills are paid, you've built something big, but you don't have your health. I remember Curtis crawling through his house for months, literally sliding on a piece of cardboard. His wife would drag him through the house because he couldn't walk. How important is our health? Well, our final speaker today is about to rock your world about elite health. Not just, yeah, I'm healthy. I mean elite levels of health. Help me welcome to the stage one of my favorite. Well, no, no, just, yeah, I'm healthy. I mean, elite levels of health. Help me welcome to the stage one of my favorite. Well, no, no, no, no, no, no. Roll the video. Here we go. You're one of my best friends. You've been such a good friend to me, blah, blah, blah. There's nothing I could ever do for you, so I'm going to do this. I'm going to introduce you to this guy named Gary Brekka. I love Gary, by the way. I love Gary. Yeah. My name is Gary Brekka. I'm a human biologist, researcher, and a biohacker. For 20 years, I spent my lifetime working in the insurance industry predicting mortality to the month, which meant if I got five years of medical records and five years of demographic data, the team that I was associated with could actually tell an insurance company how long you had to live to the month. After years and years of this type of research and analysis, I realized one thing. There were human beings on the other side of the spreadsheets that I was working on, and that I didn't want to spend another day of my lifetime predicting how long people were going to live. And I wanted to help them live happier, healthier, more fulfilling lives. You see, we were put on this earth to thrive. God gave us everything that we need in order to manifest our destiny and thrive. Clean, clear, cognitive energy. The waking energy of a tiger. This guy's incredible. He's amazing. And this guy can come in and literally change your life. And if we want optimal health, we have to educate ourselves and go out and find ways to go inside of our bodies, find the missing raw material that's keeping us from being the absolute best, replace that raw material and then thrive. And today I am so blessed to share my journey, all of my research, with all of you. Woo! Woo! Woo! Yeah! Oh my God, it's so good to be back at EXP! I absolutely love you guys. Last year, I almost had a heart attack running around this audience. So I'm really excited to be here. Hats off to the EXP crew for throwing another amazing, amazing conference. We got a lot to cover today, guys. I broke my talk today down into 15-minute segments. I know some of you have heard me speak before. Some of you haven't. For those of you that haven't, the first 15 minutes of my talk will be very similar to what it was last year. The next 15 minutes, I'm going to talk about how every single person in this audience can wake up like a Spartan warrior, can have a superhuman inside of them, and I'm going to bring it out of every person in this audience. I'm going to teach you how to wake up like a Spartan warrior and how to be a superhuman. So who wants to be a superhuman? Amazing. And then we're going to spend 15 minutes talking about some of the most incredible research in the world right now. I just got back from the most humbling retreat of my lifetime. I was invited by Richard Branson to Necker Island with 40 of the top thought leaders from around the world. He took people from every industry that he thought were the greatest thought leaders. He put us all on an island for four days. No phones, no recording devices. We just shared and collaborated on research that is impacting humanity and some of the most profound discoveries in science that are changing the entire direction of humanity and health and wellness. And I'm going to share some of those with you today because you have no idea how powerful our mind really is. And I'm going to teach you how to access an area of your mind that you may not be tapping into every day and it's robbing you of 75% of your potential. So we're going to unlock that in the third 15 minutes. And in the last 15 minutes of my talk, and I do this every time I take the stage, I make two big bold promises every time I take a stage. And the first promise is if you do what I ask you to do today, I will add seven years to the lifespan and the healthspan of every person in this room. Great. Who wants to do that? And the second bold promise is that I will take any ailment that you or a loved one suffers from, any ailment, whether it's an autoimmune disease, an emotional disorder, a mental disorder, it's hypothyroid, it's hypertension, any ailment that you or a loved one suffers from. And right here in front of this audience from this stage, I will tell you exactly what nutrient is missing from that body that is causing that condition to exist. And then I'm going to take 15 minutes of Q&A from you guys. So we've got a lot to cover. You guys ready to get going? All right. Awesome. So for those of you that don't know me, my name is Gary Brekka. I'm a human biologist, a researcher. I spent 22 years of my life in mortality research for large life insurance companies. And you see, large life insurance companies have a database. It's got 370 million lives in it. If that database that they hold could see the light of day, it would permanently change the face of humanity. It would upend modern medicine in a way that would be catastrophic. You see, there's 370 million lives in this database. And what this database has that no other clinical study has, no research study has, no university, no governmental agency, no other place on Earth knows the day, the date, the time, the location, and the cause of death for 370 million lives. So they take that data and they pull it back into people's lives and the team that I was on if we were able to get five years of medical records on you and five years of demographic data we could tell the insurance company how long you had to live to the month and yes it's that accurate and if you want to know how accurate insurance companies are just look at what happened in the financial services crisis right 2008 2009 we had 364 banks fail not one life insurance company failed there's no other financial enterprise on the planet that would take that much risk on one variable 25 million 50 million or a hundred million dollars worth of risk and the only thing that matters is how many more months does this person have left on earth so I worked in this research department I read medical records six eight hours a day six days a week for 20 years and I was brainwashed by that industry because they led me to believe that this was just data that I couldn't make an impact that it didn't matter I didn't have any influence I didn't put these people in this position I couldn't get them out of this position it was not my responsibility my responsibility was to take the data and tell my boss how many more months this person had left on earth so they could charge them for a life insurance policy and I realized one day that there were human beings on the other side of those spreadsheets. And I made a decision that day to turn in my resignation, to leave that industry, and spend the balance of my lifetime helping people live healthier, happier, longer lives. And so that's what I'm here to do today. And I'm in the process of building an army. Because we need an army. If the pandemic didn't teach you anything about the intention of our government, about the intention of modern medicine, about the intention of our officials, then you must have been asleep at the switch. So it's not up to them. It's up to us. Right. It's up to us. So I want you guys to be on my army. So I'm not going to hold any information back from you because we learned two things when I was predicting mortality. Here's some of the mortality predictions that I did, but we learned two things when I was predicting mortality. And I say this every single time I take the stage. This is the most impactful statement you will ever hear in terms of how many more years you have left on earth your waking energy your cognitive function the depth of your delta wave of sleep your weight gain your water retention your hormone balance the entire trajectory of your life and that is that the presence of oxygen is the absence of disease so i'm going to talk to you about how we get more of it how we manage it better and how we keep it at a level that helps human beings thrive and makes this entire by-home inhospitable to disease. So the presence of oxygen is the absence of disease. The second thing that we learned was that nearly every complication that we accept as a consequence of aging, that we accept as a consequence of aging, weight gain, water retention, brain fog, ADD, ADHD, OCD, manic depression, bipolar, all kinds of mental disorders, the history of a genetically inherited disease, is absolutely none of those things. It is a missing raw material from the human body, a deficiency that is leading to those diseases. You know, recently, I was invited to speak at a very large age management medicine group conference. It was a room this size full of physicians. And when I took the stage, I could feel like the, uh, the judgy McJudgerson-ness, you know, like if they weren't getting CE credits, not a single one of them would have been in the room. I mean, it it was like it was an oppressive level of hatred coming at me and I said let me ask you guys something um how many physicians in this audience right now by show of hands how many of you are treating patients for genetically inherited diseases genetically inherited diseases like hypertension like hypothyroid like hypercholesterolemia, hypertriglyceridemia, even drug and alcohol addiction, anxiety, depression, all of these diseases that run in families. And before I got through five diseases, every single hand in the room was up. And I said, while you have your hands up, understand if you're treating patients for these genetically inherited diseases, understand we've mapped the entire human genome. If you're treating patients for these genetically inherited diseases, understand we've mapped the entire human genome. We know every gene in the human body. Can any of you tell me what gene causes any of those diseases? Every hand in the room went down. And I said, the reason why you can't tell me what gene causes any of those diseases is because that gene doesn't exist, which means that those diseases is because that gene doesn't exist which means that those diseases do not exist and then I stood there for 30 seconds and even more hate came to the stage like the the Hato mammometer was all the way in the red but no one could take issue with that and if you're a physician I invite you to take issue with that too we do not pass disease from generation to. That is a lie that we have been sold because we have an industry that is built around managing disease. But if we understood that what happens is we pass on something called methylation. We pass on the inability for our body to take a raw material, a vitamin, a mineral, an amino acid, a nutrient that enters our body and convert it to the usable form you see just like we pull gasoline out of the ground but we can't put it into our gas tank because the car doesn't understand that fuel source until I'm sorry crude oil until it's refined to gasoline there is not a single compound known to mankind not one. There's no vitamin, no mineral, no amino acid, no protein, no nutrient of any kind that we put into our bodies that is used in the form that we put it in. Without a single exception, everything we put into our bodies gets converted by our body into the usable form. If you cannot make this conversion, you have a deficiency. It is this deficiency that leads to those diseases, and that deficiency can be fixed. So that's what we're going to do today. You guys with me? All right. All right. So those are my two bold promises. I'll add seven years to your life, and I will take any ailment that you or a loved one suffers from and I'll tell you what nutrient is missing from your body in fact I might even call on some of you guys that took the gene test last year and supplemented and have had a life-changing result if you had a terrible result please stay quiet but hum but if you had a life-changing result I want to hear from you so this the first test that I'm talking about is a gene test, and I'm going to go through this quickly because I covered it last year, but so many of us that have these conditions that we chalk up to a consequence of aging are not a consequence of aging. So many of you right now, more than 90% of this audience is walking around at 60% of your true state of normal. is walking around at 60% of your true state of normal. Even if you think you feel good, you have no idea how good normal feels. God meant for us to thrive, for us to have it all. And when we put the right raw materials back into our bodies, we start to thrive in ways we never thought possible. And so, thank you. Woo! I love you guys. Where was the lady that used to give me all the woos last year? Woo! Somebody used to give me all the woos last year? Somebody used to give me woos. Every time I'd say she would go, and then when I refused to do something, she's like, I'll give you a woo. So all of these things, brain fog, high blood pressure, gut issues, all of these things are consequences of these five genetic breaks. There are five genes in every human being's body in this room that determine what raw materials you can convert into the usable form and what you can't. And when these genes are broken, this is where those conditions come from. And these can be fixed. is where those conditions come from and these can be fixed these can be supplemented for and they can be fixed so we do not need to walk around with the kind of ailments like we do in society if you look at the trends in modern medicine you know diabetes is a 110 billion dollar annual industry do you know that 110 billion dollars a year do you think that there's a meeting going on somewhere in a big pharma boardroom right now to put that out of business oh no you know that? $110 billion a year. Do you think that there's a meeting going on somewhere in a big pharma boardroom right now to put that out of business? Oh, no. You know, if you said to me, nobody would ever say this, but if you did, if you said to me, Gary, write me a diet that would guarantee I get diabetes, you know what I would do? I'd go right to the American Diabetic Association website, I'd download their dietary guidelines, and I'd send them to you. That's how bad that is. Do it. American Diabetic Association. You know what they recommend for a type 2 diabetic first thing in the morning? Glass of orange juice, bowl of oatmeal with crushed brown sugar and natural honey as a snack. Yogurt with fruit on the bottom. 44 grams of sugar, right? You know what that is? That's an insulin dependency diet. Do you know that Pfizer just paid 6.6 billion dollars for arena pharmaceuticals? Do you know what arena pharmaceuticals does? Anybody know? Fixes myocarditis, pericarditis, and diffuse vasculitis as a consequence of vaccine injury. That's a fact. So we need to understand that sometimes the powers to be don't necessarily have our best interests at heart, but God has our best interests at heart, and he has given us everything that we need to thrive, and we're going to talk about how we tap into that today, right? So this is what this gene test looks like. It is a Q-tip. We stick it in your mouth, and we rub your cheek, or you order it,. We stick it in your mouth and we rub your cheek or you order it and it comes to your house and you rub your cheek and you rub the cheeks of your family members. Only get that test for the people you love. The other ones will leave your life early. That was a joke. That was a joke. So you know what? Right before I came up here, somebody literally backstage told me, stay away from politics and stay away from the vaccine. I've already covered the vaccine, so I'll stay away from politics. They were like, don't talk about Joe Biden. I'm like, why would I talk about Joe Biden? But I can't help thinking, you know, like if a spaceship like landed in the United States, you know, they all seem to land in Arizona. So that's where all the spaceships land. Actually, one landed in Vegas recently. But if a spaceship like landed in the United States, you know, they all seem to land in Arizona, so that's where all the spaceships land. Actually, one landed in Vegas recently, but if a spaceship, like, landed in the United States and they got out and said, take me to your leader, I just think about that once in a while. What would they think? We'll just leave it at that. So there, I covered politics and the vaccine, and 40% of the audience still likes me. Woo! Come on, debut cheerleader. So these five genes, so what happens when these genes are broken? I put Grant Cardone down there because that's actually Grant Cardone's gene breaks. One of the reasons why I'm wearing this 10X Health shirt is because I fixed those genetic breaks in Grant Cardone and got rid of joint pain and sleeplessness and aches and pains in his body and brain fog. And as he realized the consequences of that in his business, he called me one day and said, Gary, your stuff works. If you can be in my office at two o'clock tomorrow afternoon, I'll make you a rich man. And I was in his office at 1.59. The one meeting of my life that I was actually early for was with that man to make me a rich man. He kept his promise. So these five genes, every one of you has those. When you have these genes that are broken, let me point some of these out. Do you see this gene right here? 44% of you have this gene not working. I won't tell you what the nickname is for that gene. Brett, I'm going to keep my promise. Where are you? Okay. I went to the speaker meeting this morning. Yes, that is called the MTHFR gene. If you ever want to look up research on that gene, please capitalize all of those letters or you will find yourself on some really colorful websites. So this gene, 44% of you actually have a break in this gene. This is one of the most important genes in the human body because it takes folic acid and converts it into methylfolate. And while that doesn't sound like a big deal, folic acid is the most prevalent nutrient in the human diet. Folic acid, by the way, is an entirely man-made chemical we make it in a laboratory folic acid does not occur anywhere naturally on the surface of the earth you cannot find folic acid in nature we make it in a lab and then we have sold it to the public as a health as a health vitamin something that pregnant women need to have a healthy pregnancy that's completely false they need methylfolate to have a healthy pregnancy 85 That's completely false. They need methylfolate to have a healthy pregnancy. 85% of all postpartum depression is directly related to the prenatal vitamin. There is not a single peer-reviewed published clinical study in the world linking pregnancy hormones to postpartum depression. There is tons of research linking that gene to postpartum depression. So what happens is when the most prevalent nutrient in the human body cannot be converted into something the body can use, you have a deficiency. And now what kind of things come from that deficiency? Well, the first thing is gut issues. You have gut issues and you think you have an allergy, but it drives you crazy because you eat the same thing over and over and sometimes you have a reaction and sometimes you have a reaction and sometimes you don't. Because it has nothing to do with what you're eating, it has to do with the pace of your gut. The speed of your gut is determined by that gene. If that gene is broken, you have suffered your entire life with gas, bloating, diarrhea, constipation, irritability, cramping, and you've never been able to pinpoint why. Because your body is missing a simple raw material called methylfolate. If you have this gene right here, there is about a 90% chance that you suffer from anxiety. And some of you may suffer from crippling anxiety, even borderline depression. You know what anxiety is? Anxiety is a fear of the future. It is an increase in your brain in a category of neurotransmitters called catecholamines, the same neurotransmitters that come out in a fight or flight scenario. How many of you suffer from anxiety or know somebody who suffers from anxiety? Holy cow. 70% of the room. You ask somebody that suffers from anxiety these three questions, I will prove to you that it is coming from that gene and it can be fixed with methylated vitamins. And if anyone in this room has either had that problem and it's now fixed or had a loved one who had that problem last year and it's now fixed, is there anyone in here that would like to tell that story? Because I ran into a couple people at the pool. There's one right there with their hand up. Can you come up this microphone? If you start to say something bad, I'm just going to burn one of your eyes out with this laser pointer. I've already cried on stage. Nothing will learn you like losing an eye to a laser pointer. Yeah, so it was last year when you said this talk and my daughter, just crippling anxiety and, you know, just very inward in everything she did. And I was all in, ordered it for the whole family. And so within like three weeks of being on the supplements, she was into horses. It's kind of her way of dealing and all of that. And her horse instructor didn't even recognize her. She was yelling across the barn and just a totally different person. Amazing. She joined the cheerleading team at her school. Woo! And like just wildly different. She couldn't even get out of her seat to get on stage when her classmates were, you know, just going on stage to graduate before. And she took five of the eight awards. Wow. Just this past year. That's amazing. And she was on stage and waving at everybody and just she's on no drugs. She's on no pharmaceuticals. That little girl took that test and all she's taking is vitamins. That's how we're going to change the world. That's how we're going to change the world. You have this gene right here. You have a history of hypertension in your family. How many of you either suffer from hypertension or have a loved one suffering from high blood pressure? You see, high blood pressure is the second leading cause of death in America. And most of people that actually suffer from high blood pressure, when they go to their doctor, they have a normal EKG, they have a normal EEG, they have normal heart and lung sounds, they have a normal die contrast study, they have a normal cardiac catheterization. All their cardiac workup is normal. Why do they have high blood pressure? Their doctor will tell you because your father had it and your grandfather had it and your great-grandmother on your mother's side had it. You genetically inherited that hypertension. Next time you know what to say. Oh, what gene did I inherit that caused that? Watch their face go blank. What happens is when you have this gene mutation, you cannot lower an amino acid in your blood called homocysteine if this is a naturally occurring amino acid but genetically we break this amino acid down into another harmless amino acid which quiets our mind and if we can't quiet our mind now we're told that we have ADHD or a DD that's another famous one or OCD right and so if I can't actually lower that amino acid, then what happens is it rises in my bloodstream. As it circulates around in my bloodstream, my arteries clamp down. If you make the pipes smaller in a fixed system, pressure goes up. And now we hold the heart responsible for a crime it hasn't committed. We start pounding on a perfectly healthy organ with beta blockers and calcium channel blockers and diuretics, trying to pound a healthy organ into submission for a crime it is not committing. And we do this for decades. You will never convince me that 44% of Americans are so depressed they need to be on an antidepressant. 51% of Americans over 55 years old have a failing heart to the point where it needs medical intervention or that nearly 60% of Americans suffer from a mental disorder. I'll never believe that. What I will believe is we are so nutrient deficient that without certain nutrients, we get the expression of those conditions. And so if you would lower homocysteine, by the way, if you ever watched my video with Dana White, this is exactly what happened to Dana White. He was on three blood pressure medications. When I met him 10 weeks later, he was on no blood pressure medication. He had perfectly normal blood pressure. It was off of his antidepressants and off of his thyroid medication. And so we put these raw materials back into the human body. And that's how we just restore that vascular laxity to normal. And then the pressure returns to normal. Now for you high functioning entrepreneurs, if you have that gene broken right there, you have what's called consummate overachiever profile. You are your own worst enemy. You hold yourself to a very, very high standard, right? You put a lot of pressure on yourself. You essentially decide the success or the lack thereof of your day by how many things you get done on an agenda that you set for yourself. You find yourself very easily disappointed in other people because you feel like they just can't do the basics. I know those laughs are like, yep, that's me right there. I got spouses pointing at spouses. So, and when you lay down to go to sleep at night, you have a hard time falling asleep, not because you're not tired, but because your mind keeps you awake, right? So what happens is, as your environment quiets, your mind wakes up. So what if we could pull all these little anchors up off the ground? What if when you laid down, you could actually quiet the catecholamines and fall asleep? What if you could actually process thought in the right sequence, which is what we call attention deficit disorder. There actually is no such thing as ADD or ADHD. They are, trust me. I love you guys. I mean, I love you guys. Usually people are like, boo, and that's not true. And they need Ritalin and they need Adderall and they need Vyvanse. Interestingly, the same people that make folic acid, Monsanto, also makes Vyvanse and Ritalin. So folic acid is the leading cause of ADD, ADHD, and manic depression. And it's treated with Ritalin, Vyvanse, and Adderall. Just a coincidence, another coincidence I'll point out for you, not being a conspiracy theorist. So what happens is attention deficit disorder is not an attention deficit at all. It's an attention overload disorder. It is too many windows open at the same time. And so what happens is because we create thought at a faster rate than we dismantle thought, the mind gets clouded. You're thinking about a job you're working on. Your friend walks up. You're thinking about a job. You start talking to your friend. While you're talking to your friend, you see a logo on their jacket. Reminds you of a vacation you want to take. And you're thinking about a job, talking to your friend, looking at the logo, thinking about a vacation you want to take, right? All at the same time. And your friend's like, hey, Max, you know, my grandmother died on Sunday. And you go, that's a great idea, right? That's when you know you got ADHD, right? So what if we could just put amino acids like L-methionine back into the body? L-methionine, as simple as L-methionine or trimethylglycine, amino acids that are already in your body, and you learn to downregulate catecholamines. You learn to take those category of neurotransmitters that were in your brain and you get to suppress them. You get to break down thought so that you can sequence thought properly. And now you all of a sudden don't have ADHD because you are not having an overactive mind. Modern medicine says if the mind is racing, you are not having an overactive mind. Modern medicine says if the mind is racing, let's put an amphetamine into the body, race the central nervous system to match the pace of the mind. That creates dependency. It creates something called tachyphylaxis. It can permanently alter the neuroplasticity of the brain. Almothionine is in your bloodstream right now. So if you have that gene mutation, you take almothionine, you will notice a demonstrative change in your behavior. You will notice a night and day difference between children that have that MTHFR gene mutation and you take them off of folic acid and put them on a methylated vitamin and you have an entirely new child in the house in three to seven days. Happens that fast. Right? Instead of the school calling you and saying, little Johnny's not paying attention, he's disruptive, he doesn't finish his work We need to bring in the Ritalin, right? So Every single person that hasn't gotten that test. I want every human being to do that test once in your life It's about 600 bucks. It's a gene test. You put the cheek swab in your mouth. You send it into a lab They send it back. You will never guess what you need to supplement with for the rest of your life You will stop supplementing for the sake of supplementing and you'll start supplementing for deficiency. And when you supplement for deficiency, you start to thrive. And so that's my pitch. Now moving to part two, because now we're going to have some real fun. I want to talk about something called the superhuman protocol, because the superhuman protocol because the superhuman protocol is basically taking everything from mother nature that is good from outside and bringing it in you have two choices you can pay 150 000 which a lot of our patients do for the superhuman equipment or you can do it for free which one do you want me to explain to you guys I knew you guys were cheap. Yeah. So what is so amazing about the research that is going on now in longevity, in anti-aging, in biohacking, and in optimization, and when I was on Necker Island with Richard and these thought leaders, it really hit me that everything has now come full circle and we're just getting right back to the basics. The further we get away from the basics, the sicker we become. Human beings were originally built to spend 85% of our time outside. The average human spends less than two percent of their lifetime outdoors now so now what's happened when we've come indoors we've stopped contacting the surface of the earth do you know that the earth is a giant magnetic field think about the last time that you had bare feet touching bare soil dirt grass or sand that's the last time that you discharged into the earth and this is very real by the way if you follow me on Instagram, you'll see that I took a voltage meter. I plugged it into the dirt in the sand, into the ground. And I held the other pole of this voltage meter. And I showed you how there was zero voltage going through my body. I was standing in tennis shoes. Then I took my shoes off and I stepped on the surface of the earth and the pin pegged all the way to the right-hand side. So what happens when we touch the surface of the earth? It's called earthing. It's called grounding. You see, we don't want it to be this easy, but it is, right? When we touch the surface of the earth, we discharge into the earth. It actually changes the polarity of the body. It can shift the alkalinity in the blood. It can change the polarity of 32 trillion cells in the human body in six seconds. And we don't touch the surface of the Earth anymore. What we did was we came up with a PEMF mat, pulse electromagnetic field mat, that you can put in your bed and get the same thing as grounding. But you can take your shoes off and contact the surface of the Earth for five minutes, three times a week, and maintain that same polarity. The science has now caught up to the magnetism of the earth and the impact that actually contacting mother earth has on human beings. The second thing that we don't get enough of is light. The number one nutrient deficiency in the world is vitamin D3, the single most important nutrient in the human body. 50% of this audience is clinically deficient in vitamin D3. 85% of the African-American population and Latinos, because of the pigment of their skin, are deficient in vitamin D3. This is why COVID disproportionately affected minorities. Had nothing to do with their minority status. It has to do with the pigment of the skin. The darker your skin, the lower the vitamin D3. The lower the vitamin D3, the higher the all-cause mortality and the weaker the immune system. In fact, one of the worst things that we did during the pandemic was further weaken our immune system by social distancing, residential quarantining, masking. You know what human beings were meant to do? They're meant to do what we're doing right now. Interact with other human beings. They're meant to do what we're doing right now, right? Interact with other human beings Right that builds a strong immune system What am I saying favorite sayings is that aging is the aggressive pursuit of comfort? The more aggressively we pursue comfort the faster we age the reason why most of us don't exercise or take cold showers or do cold plunges or like to diet and deal with a little Pang of hungers because we don't like to be uncomfortable The more comfort we seek, the faster we age. We've got to stop telling grandma not to go outside. It's too hot not to go outside. It's too cold just to lay down, just to relax, to eat at the first pang of hunger. That's all nonsense, right? The body has a response called hormesis where you stress it and it strengthens. And we're going to talk about how we can tap into that. So the second thing is light. We are light beings. The truth is most of us are not getting enough sun. It's not that we're getting too little sun. So when I wake up in the morning, the very first thing I do, and I don't care how cold or hot or rainy or windy it is outside, I go outside in a pair of shorts. My neighbors think I'm nuts. I go outside in a pair of shorts every single day. You can ask my wife. And I go out and I take my shirt off and I get 15 to 20 minutes of sunlight and I do eight minutes of deep breath work every single day like clockwork. It will change the trajectory of your life, right? And then the last thing that we talk about is oxygen. So you can use a hypermax oxygen system or you can learn to do breath work. And there's several styles of breath work. There's all kinds of breath work that's out there. Last year, we talked about breath work. There is a style of breath work called Haka breathing. How many of you ever heard of that? Woo! Oh, is that because you guys follow me or have you, do you actually know what that is? The Haka dance, have you ever seen like the All Blacks in Australia? They do those crazy dances before rugby games. You know why they do that? It increases the oxygen content. It improves physical performance. It raises their emotional state. It elevates their mood. And it is pain modulatory, right? They actually don't feel as much pain during contact. So how would you guys like to learn a 60-second breathwork technique so you could wake up like a Spartan warrior every single morning. You guys want to do it right now? All right. Because, so after this breathwork technique, they are definitely going to think we're a cult, right? So, you know, the spaceship's coming, we're all going to heaven, right? I knew there was something weird about that crew. Now they're definitely going to think it. So what this has to do with is, you know, breath work does several things for us. What it's going to do, it's going to change the oxygen tension in our tissue. It's going to change the ratio of gas in our tissue. The second thing it's going to do, it's going to raise our emotional state. It's going to elevate our mood. Because believe it or not, every emotion that a human being can experience, every single emotion that is an elevated emotion, passion, elation, joy, arousal, libido, all the heck yeah, I won the lottery emotions. If you actually looked at that molecule, it has oxygen as a component of its structure. If you pull oxygen out of that molecular structure, you can't have that emotion. This is why no human being has ever woken up laughing. But you can wake up angry. Why can you wake up angry but not wake up laughing? You don't have the oxidative state to experience laughter. But anger, vengeance, despair, jealousy, depression require zero oxygen. Right? If you want to do a fun little experiment tonight, wait till your spouse is in a deep sleep and give them a good deep pinch. Watch them wake up really angry. Hey, Gary was right. I was just checking it out. That's not on me. You didn't wake up laughing, see? He was right. Really smart human biologist. So if we want to reach elevated emotional tears, we need to have the oxidative state to reach elevated emotional tears we need to have the oxidative state to reach elevated emotional tears so that's what we're going to do the fourth thing that it does is it releases sirtuins how many of you have ever heard of a sirtuin you know what a sirtuin is it's a survival gene it happens when we slip under the ice it happens when we get too close to a fire it happens when a lion gets too close to us and threatens us. They are called sirtuins. These scour the body of free radical oxidation. They increase the rate of protein synthesis, and you can access them for free every single day. You guys want to do it? All right. It's going to get a little weird in here. All right, so we're just going to sit. First, we'll just put our hands comfortably on our laps. Men, what I want you to do, men, is I want you to make a fist. And when I tell you, we're going to tap up on our chest like this. You are not trying to knock your boob over your shoulder, right? It's like somehow it becomes like a mankind. It's like, hit themselves in the face. You're going to take your fist and you're going to tap up on your breast like this. And we're going to do it four times. Wait, wait, wait till I start it. We're all going to take your fist and you're going to tap up on your breast like this. And we're going to do it four times. Wait, wait, wait till I start it. We're all going to do it together. We're going to take a deep breath in like this. And when we breathe out, we're going to say the word ha four times. And we're just going to tap our chest like this. Ha, ha, ha, ha. Women, you're going to cross your hands, raise them up, and you're going to tap your fists on your shoulder. Okay. And we're going to take a deep breath in. And on the way out, you're going to tap your fists on your shoulder Okay, and we're going to take a deep breath in and on the way out you're going to say ha ha ha ha four times I know this sounds crazy Wait till we do three of these breaths and wait till you see how you feel you're going to release your two ends We're going to raise our emotional state. We're going to elevate our mood. We're going to flood the brain with oxygen You guys ready to go? All right, I want an obnoxious breath in. We're going to do three rounds of this. Ready? A deep breath in. Ha, ha, ha, ha. Ha, ha, ha, ha. Ha, ha, ha, ha. Woo! Yeah. Now, aside from offending your neighbors and everybody else in the hotel, if you actually did that on your balcony every morning, you would wake up like a Spartan warrior, right? My poor wife has to listen to it every day. And then I'm so chit-chatty when I'm done. She likes to sleep in. I come in and I want to tell her all about my day and get stuff off my chest. I used to have a whole Instagram special. It was called Good Morning, Babe. And right after I would do breath work and haka breathing, I would go in and wake her up and go, and dance on the bed. She one time on Instagram in front of half a million people, she threatened to stab me in the neck. You did. I still have the video. So in any case, so we're going to get oxygen into our bodies. I'm going to talk about supplements in a second because I don't want anybody to just start taking supplements for the sake of supplementing. I want you to do that gene test. We have a booth outside, by the way, the 10X Health booth. I want you to go to that booth and talk to those guys about how to get that gene test for you and your family once, and then never guess on what you need to supplement with. And now I want to talk about the brain. Because Joe Dispenza and MIT, woo, you guys like Joe Dispenza? All right. Oh, man, I really love you guys. We're working on filming a documentary called The Frequency of Truth. And part of the documentary is about the power of the human brain, which we are only now beginning to understand. God is so much more powerful than modern medicine that it's unbelievable. that it's unbelievable. The capability that this has to influence the environment around it, to heal the environment around it, to heal itself is absolutely uncanny. In Scandinavia, they just did a clinical trial, a clinical study on terminally ill cancer patients. By the way, any of the studies that I ever talk about from the stage, if you'd like to see those, they're on my link tree on Instagram, or I'm happy to send you these articles. I am a science nerd, so if you ever would like to know what study I'm referring to, I'm happy to send that study to you. But what they did was they took two groups of people. Both of these people were diagnosed with the same terminal illness, only they took one into one room and one into another and with one placebo group they told them the truth about their illness that it was terminal the percentage chance of five years survival and that modern medicine had exhausted everything that they had and there was essentially nothing they could do other than to extend them palliative care hospice palliative care they took an identical group of people into another room and they said you have this type of cancer but luckily for you you qualify for a clinical trial. And in this trial, we've had a 100% cure rate. 100% of the patients that have qualified for this trial have gone into remission. And you're so lucky you have the tissue type that matches this trial. And so there's virtually 100% chance that you're going to go into remission. What they measured was the shift in frequency, the shift in that person's emotional state. And what we realize now is that there's an area in the brain called the amygdala. When you feel emotion, any mood, any emotional state that you can feel, you are experiencing that emotion in an area of the brain called the amygdala. Okay, and what's special about this area, if you're happy, if you're elated, if you're upset, if you're angry, you're feeling that in that area of the brain called the amygdala. What's interesting about the amygdala is that it is the only gateway to the hippocampus, gateway to the hippocampus to the memory there is only one hallway that you can go down to reach your memory and that is through the amygdala that which means your emotional state determines what memories you retrieve well your memory links to the prefrontal cortex your conscience and your conscience is your future and so what this means is that your current emotional state determines your future. If you do not learn to master your emotional state, you will never take control of your future. So my whole reason for existing in life is because I want to elevate the emotional state of humanity. I want to elevate the mood of humanity by putting the right raw materials into your body so you can manufacture the neurotransmitters to stay in an elevated state so you retrieve healthy memories, you think healthy thoughts, and you project a very healthy future. Right? And so imagine the emotional state of someone who was just told they had a terminal illness versus the emotional state of somebody who was actually told there's virtually 100% chance they're going to live. Just compare those two emotional states and imagine what memories they started to access. And then imagine what their conscience began to think of. And imagine what happened to their future. 47% of terminally ill patients went into spontaneous remission. That's almost half of the patients in that study. This is the power of frequency. This is the power of the human body. This is the power of the mind. We can actually use this to our advantage to influence the world in a positive way around us. Last year, you might have heard me talk about the Spain scale of emotion, the scale of positive and negative emotion, right? Where they took participants, they put them in a room. It's called a Faraday's cage. It's devoid of frequency. They started to measure frequency, leaving a human being's body. And what they realized was that the most powerful frequency to leave a human being's body, somebody yell it out. Authenticity. If you want to influence the world around you, everything from your children to the people that you interact with, to a client you're trying to gain a listing from, from a team you're trying to motivate, from a spouse you're trying to convince, from a child you're trying to discipline, if you can master the frequency, if you can emanate the frequency of authenticity, you can actually change their emotional state. This is a fact, by the way. You know, we say that women are more intuitive. Have you heard that? that women are more intuitive. Have you heard that? Okay, that's actually not true. Women are more sensitive to frequency. We call it intuition. It's a sensitivity to frequency. I can't even tell you how many times my wife has said, nope, not that guy. Something's a little magic. Can't put a finger on it, but I don't like him. I'm like, but babe, look at his shoes. He's got a nice suit. Drove a Ferrari. How bad could he be? And six months later, I'm like, but babe, look at his shoes. He's got a nice suit. Drove a Ferrari. How bad could he be? And six months later, I'm taking the knife out of my back. How many of you guys can identify with that? So you are sensitive to this frequency of authenticity. It's the most powerful frequency to leave a human being's body. There's a law in physics called constructive interference. Look it up if you'd like. It says if two frequencies of equal wavelength meet, the size of the frequency doubles. This proves the universal law of attraction. Right? The universal law of attraction is actually a law in physics. But the point is most of us don't understand how to master this in our lives. So I'm actually going to teach you a technique to master our emotion, to pull from the right memory, to project the proper consciousness, to change our future. And it is powerful enough that when you use it in business and in life and in your career and in your relationships, you will see the entire trajectory of these change. It is so powerful that it will influence the decision-making capacity of another individual. Because we know now that emotion is what triggers our memory, and memory triggers our conscience, and our conscience is our future. We also know what percentage, by the way, I'll just ask you, what percentage of decision-making is emotional? By the way, this was just published in, what percentage of decision-making is emotional? By the way, this was just published in the Journal of Neuropsychology. What percentage of decision-making is emotional? 95? 100? Not quite 100. Maybe with my wife, it's 100. But 75 to 78% of decision-making is emotional. So what should be the most important thing that you do before you get on a phone call or a team meeting or a Zoom to motivate your team? First of all, you should master your own emotion. I'm going to tell you how you do that. You should try to impact the emotional state of the people that you're talking to. In fact, if I could teach you how to influence the emotional state of a client then when you get went to get a listing if you actually trigger the right emotion in that client that makes them recall the proper memory they will decide to give you that listing so how many of you would like to do that all right so let's talk about how we tap into this right and and this is, you know, I'm far along right now, but this is what's about to be published in our documentary. So first of all, these areas of the brain need certain amino acids, they need certain vitamins, they need certain nutrients, because this entire cascade happens by neurotransmitters. So the second way that you can tap into it is to get yourself in a state of authenticity. Authenticity only occurs when two things are present. Number one, when your words are truthful. And number two, when you believe what you're saying. If either one of those is not present, you cannot actually emanate the frequency of authenticity. And by the way, this is part of the clinical study, the Spain Scale of Emotion. I'm happy to send that to you. So this is what some women pick up on. Because somebody's words are not truthful. Or they don't believe what they're saying. And we don't pick up on that. But very often, women will pick up on that and say, there's just something that's not adding up here. Because that person is not emanating the frequency of authenticity. So if you want to shift into an authentic state, I want you to do this before your next team meeting. I want you to do this before your next client meeting. I want you to do this before your next client meeting. I want you to do this before your next big event so that you can actually have influence over that person in a positive way. Okay, you ready? So first, it starts with intention setting. And second, it moves to something called visualization. But only about 5% of the population really understands the power of visualization. When I work with athletes and top fighters in the UFC and running backs in the NFL, part of what I do is I teach them how to visualize outcomes. Never, ever, ever visualize the journey. If you visualize the journey, you will set yourself up for failure. If you visualize the destination, you will set yourself up for success. This is why I love being a man of faith, because it's not only delayed gratification, but I already know where I'm going, right? So I know what the destination is. So all the stuff in the middle doesn't bother me so much. But so here's how we do it. How do we set our intention? So for example, before I got on this stage today, I sat quietly in a chair back there, closed my eyes. I took three deep breaths like this. And I thought about what was my intention of getting on this stage today. My intention was to add value to your life, right? So that's the only reason why I'm here right now. For the next 60 to 90 minutes of my life, I said I'm going to give everything that I can so I can provide value to this audience. And then I created a picture of every person in this room hearing my words, believing those words, agreeing to put those words into action, of thanking me for speaking. And I actually lived that moment, right? And then when I got up here, I didn't have any fear. And my words were truthful, and I believed what I was saying, so I was able to deliver an authentic message. And we can use the same thing in business. We can use the same thing when we're getting on a call to motivate our team. So the first thing we do is we take three, put your hands just like this. Everybody just close your eyes. And we're going to take three deep breaths through our nose and we're going to let it out through our mouth. Ready? Ready? Second one. second one, third one. Now in this state, this is where we set our intention. What is my intention of this team meeting? Do I intend to motivate my team? Do I intend to motivate my team? Do I intend to inspire my team? What message do I want them to receive? Only use one word to describe your intention. And once you know what your intention is for this meeting, right, I want to gain this listing. I want this person to give me the trust and confidence to sign over their home to me and my team. I want to motivate my team. I want to take the people that are straggling. I want to bring them back into the herd. So you set the intention. The second thing you do is you visualize the outcome. And this is where visualization goes wrong. I want you to create a picture of the end destination, the end of that meeting. Everybody thanking you for being inspiring today, thanking you for the message that you gave them. I want you to see that client shaking your hand, congratulating you on gaining that listing because they had the trust and the confidence in you. And then I want you to open your eyes. And now that you've set your intention and you visualize that, you know what you've done? You've set your emotional state. You're now in an authentic state. Guess what happens in your hippocampus? Your hippocampus records everything in a picture. Memory is stored in pictures. When you hear a song that reminds you of a loved one, it's not the song that's in your memory, it's the image that's in your memory. That song triggered an emotion and that emotion linked to your memory and then you formed a picture of that last time that you are around that loved one. Right? That's why you can be driving down the road and you hear a country music song and all of a sudden tears come to your eyes because it reminds you of your mother or your father or a loved one that's left your life. That is an emotion triggering a picture. So if you actually change your emotional state through breathwork, set your intention, which sets your emotion, form a picture, now what happens to your conscience and your future? Your brain starts to live that. And you will emanate the frequency of authenticity. Authenticity will hit that person's body. Authenticity will match the frequency in their future. Your brain starts to live that. And you will emanate the frequency of authenticity. Authenticity will hit that person's body. Authenticity will match the frequency in their body. It will raise their elevated emotional state. And they will have no idea why they trust you so much, why they are so confident in you. But they will give you that listing. Your team will be inspired. You will find that your objection hits with more meaning. And so that's how we start to set ourself up for these kinds of events. Try it. It takes about 60 seconds to shift into that state. Wait till you see the hormetic research that is coming out on this, on the capacity for human beings to influence the world around them. You know, in the animal world, this is how animals are eliminated and killed and how they seek safety and stay together. If you look at how dolphins use sonar, if a fish separates from the school, they can send sonar into a school of fish and find the one fish out of 15,000 fish that is not emanating the right frequency, and they can go right into that school of fish and hit that fish. When a lionating the right frequency and they can go right into that school of fish and hit that fish. When a lion is hunting a herd, they look for the weakest animal and they separate that weakest animal. And human beings, we can have the same kind of influence on one another by emanating the frequency of authenticity. Did you guys find that helpful? All right. All right. I said. You know, they're like, don't talk about science. Don't talk about politics. Don't talk about the vaccine. I think we covered it all. Any other sensitive subjects we need to go over? So everybody asked me what my morning routine is. I'm going to run through my morning routine. I'm going to teach you how to wake up like a sporting warrior. And I'm going to tell you three things you need to take out of your diet, three things you need to put into your day so you can start to become a superhuman these are all for free and then I'm gonna open it up for questions and testimonials there's a microphone right there and I think I think there's a microphone over here yeah there's a microphone right here if you have questions and because if you have a question and I and I take that answer probably people in the audience have the same answer. My morning routine is very, very simple. I wake up in the morning. I find sunlight. I go out in a pair of shorts. I sit in sunlight. I do eight minutes of breath work every single day. My wife will tell you I never, ever, ever, ever miss. I will miss a commercial flight not to miss breath work. I didn't miss it this morning. I don't miss it when I travel. You can actually set your circadian rhythm by this. So when you change time zones, if you breathe at the same time every morning, your body knows this is the time to wake up. I clear catecholamines from my brain. I make my body inhospitable to disease, to pathology. I raise my mood. I raise my emotional state. And I get it all from God and I get it all for free. All right? Woo! So first thing in the morning, I get natural sunlight and I do an eight-minute breathwork routine. And it looks just like the breathwork that you guys just did. Sometimes I do the Haka breathing unless somebody is sleeping. Then I don't like to wake them up. But I do three rounds of 30 breaths, deep breaths. I do 30 of those breaths. I hold my breath as long as I can. I take a deep breath in. I do 30 more of those breaths. I hold my breath as long as I can. I take a deep breath in. I do 30 breaths, hold, take a deep breath in, and I'm done. It takes about eight minutes to do that breath work. It takes a lot longer now. I can actually hold my breath for four minutes in between rounds of breath work. When I started, I could only hold my breath for 30 seconds. Now I can do 30 breaths and I can sit for four minutes. So that's what takes the longest amount of time so i do breath work then i go out and i get it i'll get first light i do my breath work i get 10 ounces of water by the way the first thing i want you to get out of you out of your diet um i'm kind of jumping ahead here is um tap water um never ever ever ever again drink water out of the tap. Right? Woo! And the reason for this, look, if having a water filtration system is not in the budget, get a Brita water filter that at a minimum gets chlorine and fluoride out of your water. If you have seen the clinical trials, the clinical studies in the last 15 months coming out on fluoride, they are absolutely devastating. They're devastating. Harvard University just launched a study through the Freedom of Information Act we sued the CDC and the FDA and we actually got the research that they kept suppressed on on water if you don't believe me you can go to the National Institute of Health look up the study yourself there is zero levels of safe fluoride in human beings it's a neurotoxin they find that the higher the fluoride in human beings. It's a neurotoxin. They find that the higher the fluoride concentration in the water, the lower the IQ in every state where they studied this. So stop drinking tap water. If you need to just go buy a six or $10 Brita water filter. If you want to know the filtration system that I use, there's a link on my Instagram. I don't have any affiliation with this company, but I put everything that I use in my own life there. So I want you to get tap water. but I put everything that I use in my own life there, so I want you to get tap water. What's that? Oh, wells are fine. Yeah, so well and septic is fine. It's the tap water that's bad. It's the concentration of fluoride. I try to ground myself every day. If I can't ground myself, I lay on a PMF mat. If you don't want to spend the money on a pulse electromagnetic field mat, take your shoes off and touch the earth for 10 minutes three times a week I know this sounds simple you know I talk about breathwork and sunlight and cold showers and things it seems so simple that people don't want to do it I promise it can change the trajectory of your life I do a cold plunge if you don't have a cold plunge turn the water as cold as you can make it step into it for a minute and just deal with it. Stop pursuing comfort by, yeah, stop aggressively aging by pursuing comfort. Okay. So that is my morning routine. I would really like to open up the room to questions. Any ailment that you or a loved one suffers from, and maybe if you could come up to the microphone, if you're comfortable enough to ask the question in front of this group, I'd really love to answer it for you because this is where the real value comes into play. Oh, and then I take my supplements. Oh, what's the two other things that I want you to get out of your diet? So first is I want you to stop drinking tap water. The second is I want you to get rid of seed oils. Seed oils. I got censored all over Instagram my entire account deleted by the fact-checkers for it I never said seed oils are bad I said industrial processed seed oils are bad so let's just cover this for 10 seconds you take a canola oil plant you press it it comes out like gum so how do you degum it they use hexane a neurotoxin to degum it then they take degummed neurotoxic oil and they heat it to 405 degrees. That makes it rancid. Now it smells so bad they can't bottle it, so they deodorize it with sodium chloride, which is a known carcinogen. Now it's deodorized, but it's cloudy, so they make it clear by bleaching it with chlorine. So they take a neurotoxin, they add a carcinogen, and then they bleach it with chlorine. And then they put it on the shelf and they serve it to you. You need four oils in your life. You need a coconut oil for cooking, a ghee butter, a grass-fed butter, or tallow. You need an olive oil and an avocado oil at room temperature. That's it. That's all you need, right? So four to cook and two to use for salads and dressings and everything else. If it's a palm oil, a sunflower oil, a safflower oil, guys, these are rancid, toxic, inflammatory, neuroinhibitory chemicals that we're putting into our body. So I want you to get those out of your body as well. Yes, sir. My dad has ankylosing spondylitis where the spine is fusing itself together. What would you say for something like that? Yeah. So ankylosing spondylitis is also known as candle wax spine, right? So it's actually where if you actually look at the spine on an x-ray, it'll look like candle wax. It'll look like those are dripping. This is actually not an issue with the bone. It's an issue right above the discal space with something called the periosteum. And if you actually Google amniotic stem cell and ankylosing spondylolisthesis, I know that's a big word, but I want you to put the word amniotic stem cell in front of it. You're going to see some of the most astounding clinical trials where they're actually able to repair the periosteal layer of the bone and stop this waxy dripping stop the spine from actually starting to disintegrate in on itself we always used to think that once these pathologies had started that they were irreversible we know now that that is absolutely not true right we don't just have to manage disease we can stop it and reverse it so if you like to see me after the break I'll give you some links to some of the research and some of the places in California that are actually treating that. Yeah, naturally. Woo. Yes, ma'am. Speaker 2 So two questions. Last year we were here and my husband stood up and he has Parkinson's disease. And I don't know if you were able to answer that question last year, I don't recall. But the second part of it is you talked about genetic diseases, things like that. But I had a son who passed away of adrenal leukodystrophy, which is an X-linked genetic disease. That is a true genetic disease. And then I have a son who's now 27 who tests positive for that but hasn't shown any symptoms. And I guess I'm just wondering what what can be done like if it's X linked versus genetic like you talked about and then what is possible for yeah so let me answer the first one real quick and then only into the second one so the first one with Parkinson's so Parkinson's is a neuro inflammatory disorder right I mean at the end of the day this is inflammation in the nerves so if we can reduce the inflammatory cascade right and we can heal the biline in the nerve, then we can stop or even reverse the symptoms of Parkinson's. We also know now that Parkinson's disease is linked to the gut bacteria. There are actually gases that are emitted from gut bacteria. When we are absent certain flora, we are absent certain gut flora. We're absent certain necessary gases in our gut. And these actually go up to the brain and they cause neural inflammation. You would not believe how much of our health and wellness is actually related to the health of our gut. When we destroy our gut with antibiotics, when we destroy our gut with artificial sweeteners, which by the way, you know, aspartame is about to be named by the World Health Organization as a known carcinogen. It's about to be outlawed in all European countries. It will stay in the United States for 10 years, right? Because we're the only governmental agency in the world that uses something called single dose toxicity to decide whether or not we can poison our population, right? We don't use cumulative dose toxicity. We use single dose toxicity. So Parkinson's, I'd love to give you some of the research on Parkinson's. There's something called vinyl resveratrol. There's another product called Stem Regen, which shows extreme promise in treating Parkinson's disorder. But there's also a product called TSO-1000, which repopulates the gut specifically with the flora that combats the neuroinflammatory condition in Parkinson's. So we did those two tests last year. I mean, not two, we each did your test and you said it's one time. Does that mean that, that results will never change? The results will never change. The genes you're born with are the genes you die with. That's why, that's why that test is so important because now you're supplementing for deficiency. Remember that DNA exists in the exact same form in every single cell in the human body. If it's broken in one cell, it's broken in every cell. So there's nothing more impactful for human beings than changing our DNA. So supplying our DNA with the raw material it needs to do its job, right? I mean, DNA only has two roles, replication and transcription. Create a perfect copy of itself and give commands to the cell like a CEO. So when the DNA is not giving the right command, the cell dysfunctions. When the DNA doesn't replicate perfectly right command, the cell dysfunctions. When the DNA doesn't replicate perfectly, this is where you get dysfunction. All cancer, all cancer is DNA replication run amok. If you didn't have DNA replication, was not impaired, you could not have cancer. And this is the same issue with the vaccine. We took a message from the nucleus of the cell called the messenger RNA, and we made a permanent copy of it. And we injected this into human beings. And now that permanent copy never stops giving the command to the cell. When the DNA gives a command, it eviscerates in two hours. When a synthetic messenger RNA enters the human body, it never turns off the signal. The cell never stops producing the nucleocapsid protein, right? So we're only now beginning to know the consequences of that. But I'd love to talk to you more about the second part. Yeah, I'd love to talk to you more about the Parkinson's offline so we can just get to a few more questions. And maybe about my son's disease too. No question. Yeah. Yes, ma'am. That was serious. mine is kind of you're like how do I lose belly fat I'm curious about rosacea yeah I was diagnosed with that and they told me there's no cure it's an autoimmune disease I call pull yes so autoimmune disease as a general category is actually not autoimmune disease it's autoimmune weakness all autoimmune disease is not it's a progressive dysfunction of the immune system because it's too weak right it's a weakening of the immune system and so autoimmune conditions generally are where the immune system is dysfunctional right it doesn't know the difference between friendly tissue and foreign tissue but what makes it incapable of recognizing friendly or foreign tissue whether it's rheumatoid or chagrins or any number of autoimmune conditions, is because the immune system is so weak. So we have a theory in modern medicine that we should weaken the immune system to keep it from attacking itself. We should actually strengthen the immune system so it can properly tell the difference between healthy tissue and foreign tissue. We know now, for example, Crohn's disease, which we call an autoimmune disease, is not the immune system dysfunctioning. It's the immune system properly doing its job, attacking bacteria in the luminal wall of the intestine. But we try to shut the immune system off instead of getting the bacteria out of the wall. Okay, so more sunshine, more sunshine, more dirt on the feet. No, so rosacea, more sunshine, more dirt on the feet, breathwork, sunshine, dirt on the feet, and cold showers. But rosacea is a secondary route of waste elimination for the the liver so you want to heal the liver with glutathione thank you so much yeah thanks oh yeah yes sir all right gary uh dude i like are you guys liking this i i'm all right as i know this guy got about two you know a few more minutes and i really want to add some value yes sir pulmonary emboli. So pulmonary embolism is now it depends on what's causing the pulmonary embolism, right? So I mean, deep vein thrombosis are a big cause of pulmonary embolism. I actually, Sage and I just actually flew to Dubai a few months ago when we got back. One of the passengers that we flew with actually dropped dead in the baggage claim area from a pulmonary embolism from what's called a deep vein thrombosis. I did 30 minutes of CPR on them while US Customs just stood around and just watched me. No one even knew what an AAD was for the wall. So when you travel, you'll wear compression gear, especially if you're on a flight over five hours. But one of the best ways to avoid deep vein thrombosis and pulmonary embolisms is to make sure that your blood stays at the right viscosity. This is why I think every single person needs to know more about your own biome than you do about your business. Like when I go up to you and you can tell me your income statement, your balance sheet, and your PNL, but you can't tell me your blood sugar, your hormone levels, or your nutrient deficiencies, that tells me where you're oriented, right? This is our temple. You guys should get information on your temple. And you should feed it with things that serve your temple. Don't steal from it, right? Because we often wonder why we're in this condition. And then you ask them, well, what has your sugar been like? What have your hormones been like? What nutrients are your body missing? They have no idea. They find out when disease or pathology or embolism strike. So the best way to avoid DVTs and pulmonary embolisms and embolisms of all kinds is to keep your hematocrit level, your blood viscosity at the right viscosity. And you can check that on a blood test. Okay. Yeah. Right now it's been blood thinners because there's no evidence of any deep vein thrombosis or anything else. So a lifetime of blood thinners as a defense mechanism, no interest in staying on those. So there's something called vinyloveritrol and there are other things you can do like regular blood donations to keep your blood thin. And let's talk about it at the break, because I've got to close things off right here. But guys, I appreciate you so much. Thank you so much for listening to me today. I hope you feel like you got some value out of today. I love you guys. Thank you, thank you, thank you. Woo!"

In [ ]:
whisper_vL3[:1000]

In [ ]:
with open('gary_vL3_transcript.txt','w+') as fh:
          fh.write(whisper_vL3)

In [ ]:
whisper_vL3_2 = "you you you you you you you you you you you Hold two to look after. Hold two to look after. Well, two to look after. It's in your screenshots. Three, two, one. Hey guys, welcome back to the 10,000 Step Challenge. We are going to do some investigation today to see how many of you guys were able to beat me this morning in your quest for 10,000 steps and today is going to be an amazing, amazing day. Remember I said my objective is to provide more value to you than you could possibly ever expect. I actually want to exceed your expectation. I want to give without the expectation of receipt and today is going to be one of those days. We have a huge, huge guest who has been on this 10,000 Step Challenge with me for several months. Has not gone public about it yet. He is on a massive weight loss journey. He's one of the top country music singers in the world. His music is just full of faith, full of wisdom. It is such powerful music. I'm just so excited to see it top of the charts right now. So stick around guys, because we're going to have Jelly Roll join us a little later in this simulcast to talk about what the 10,000 Step Challenge has done to change his life and what simple nutrients and supplementation has done to just dramatically change his life. So if you are not a VIP, today is the day to become a VIP. It is absolutely free for your first month. I'm going to be giving away thousands of dollars in gifts and prizes and incentives today, including a treadmill, including hydrogen water bottles, including hydrogen cleaning products. So I think what is 47 bucks to become a VIP? You're gonna win something that's either worth $250 up to $1000. So If there was ever a day to run your first 30 days free as a VIP today is the day. So you can interact with Joe and myself. I also have an amazing physician coming on this morning in just a few minutes. We just finished the podcast. Dr. Geoffrey. He is the founder of Kikur Acid. If you've ever used Kikur Acid, coffee and alkalizer are already in his Kikur Acid greens. We did an entire podcast today, Dr. Gormes. We did it on alkalinity in the body and the impact that simple nutrient deficiencies have on the trajectory of your health. And I talk about this all the time. I spent a lot of time yesterday talking about nutrient deficiencies. Remember the human needs, all of us need the same things. We need 91 essential minerals. We need eight essential amino acids. We need two essential fatty acids. And when we start to become deficient in these basics, we get all kinds of consequences. We get the expression of disease. His personal journey is amazing. The journey that he had with his father, losing his father to esophageal cancer based on just repeated esophageal, cancer based on just repeated esophageal, gastroesophageal reflux disease, GERD, or just acid reflux, which really ended up taking his father's life was what really propelled him to, you know, build Alkaline and create these products for coffee and for alkalizing the body. Remember, he agrees with me that, you know, that drinking alkaline water is not a way to make the body alkaline, but actually eating an alkaline diet can keep the pH range of the body in an alkaline state. So I'm going to open it all up for questions for two hours with the VIPs today. I'm going to bring the one and only jelly roll onto the podcast, onto the simulcast um for you guys that are on youtube if you're on facebook um if you're streaming live directly on zoom i really encourage you today get your free uh your free 30-day uh trial become a vip today and and get in the room with us after this first hour we're gonna have a ton of fun we're gonna do a ton of giveaways um we're gonna inspire a lot of people we're gonna hear from some top docs in the room with us after this first hour. We're going to have a ton of fun. We're going to do a ton of giveaways. We're going to inspire a lot of people. We're going to hear from some top docs in the industry. And we're going to hear from one of the greatest country music legends of all time, in my opinion. So let's get right to it. Let's see if any of you guys were able to beat my steps today. I really wanted to put you guys in pain, but I did not go that hard this morning. I actually got out of the house late and I had to be back for the podcast. It took about two hours to run this podcast before we got on today's live. So there might be a lot of you that beat me today, but I want to select a couple of winners to get a free hydrogen water bottle. I want to select a couple of you to get one of these hydrogen cleaning bottles because remember, we talk a lot about how we need to build this fence around our body, this imaginary fence, and filter things before they get to the temple rather than allowing our temple to be the filter. And I think, you know, having this mindset that we love ourselves enough um to make intelligent choices before things enter our body whether it's food or water or other compounds that we're putting on our skin that we're eating that might have hormone disruptors in them this this slow progression of what i call cumulative dose toxicity which is just robbing us of being in a state of optimal health and you know in the united states we use something called single dose toxicity, which is just robbing us of being in a state of optimal health. And, you know, in the United States, we use something called single dose toxicity to determine whether or not something is toxic enough to be served to the public. So believe it or not, there are certain amounts of pus that are allowed in meats. There's a certain amount of bacteria that are allowed in our foods. There are certain amounts of chemicals that are allowed in our foods. There are certain allow certain amounts of chemicals that are allowed to be embedded in plastics. Hormone disruptors like bisphenols, BPAs. By the way, you know BPAs is a special focus of mine because a lot of people don't know that BPAs, bisphenols, until the mid 60s were actually used as synthetic hormones in women's hormone therapy. Some cases they were even used to induce labor. And so these aren't just hormone disruptors, they are actually synthetic hormones. I don't know what scientist had so much time on his hands that he decided to take a synthetic hormone and mix it with plastic and realized that when you put synthetic hormones together with plastic, you create a more viscous surface so that things don't stick to the plastic. And then somehow the plastic industry adopted this synthetic hormone as a way of making their plastic more viscous. And the studies that are out on bisphenols and these BPAs in warm water bottles and cans of tomato paste that are highly acidic foods that leach these BPAs from the wall of plastic, showing that a can of tomato paste that is in a non-BPA free plastic lined can has enough BPAs in it to shift a woman's menstrual cycle. To take a woman from follicular to ovulation, ovulation to luteal, just think about the power of something that is so hormonally disruptive that can actually shift a woman's menstrual cycle just from a can of BPA laden plastic so we're really gonna go on this journey together guys these challenges are always gonna be free I'm really gonna encourage you guys to become VIPs and pay the $47 a month to get the extra value but at no time will I ever stop giving value absolutely free on these challenges we've got cold plunge challenges coming up we're gonna going to do food challenges. We're going to do all kinds of things. So I live for these. These are literally just chicken soup for my soul. So when we talk about some of these things that we can do to put this fence around our body, filtering our water, adding hydrogen to our water, I actually want to bring Dr. Geoffrey on in a few minutes to talk about how he uses hydrogen water when he does his gut healing protocol and I'm gonna make sure that we get this done before we actually move to the VIPs because I just think it's so important for you know all of my audience to understand the importance of gut health. But more importantly, I get a lot of questions and a lot of requests to talk about not just gut health, but Gary, how do I get off of PPIs, proton pump inhibitors? How do I get off of antacids? Because every time that I stop taking antacids or Tums or Rolaids or Nexium or Prilosec, the acid reflux comes back. And even though I know it's bad for me and it's destroying the downstream portion of my gut, I can't seem to get off of these. And we talked about a specific protocol today on the podcast for getting off of these. I'm not going to let today's simulcast ruin the podcast, but I do want to give you some tips and tricks, some really practical things that you could put to work in your life starting tomorrow that could start walking you towards having a healthier gut biome, improving the stomach acid, the contents of acid in your stomach so that we repair the digestive tract from the beginning all the way through to the end because as you know so goes the gut so goes the body i had a very very interesting podcast a few weeks ago i hope you guys have watched it with dr palmer who is a board certified md with harvard university and he's in their psychiatry department and he is actually treating the most drug resistant psychiatric illnesses with diet and lifestyle changes. Here is an MD-PhD at Harvard working in the psychiatric department who is actually working with the most drug-resistant patients in the world. When I say drug-resistant psychiatric illness, I am not talking about people that have a little depression or a little anxiety or a little bit of an impulse control disorder or they're suffering on and off from ADD and ADHD. I am talking about dramatic psychiatric illness, schizophrenia, paranoid schizophrenia, real mental illness that just tortures people inside their own minds, trapped inside of their own bodies, walking them out of this instead of chemicals and synthetics and pharmaceuticals, actually using diet and lifestyle changes and walking these people back to having a normal life. So it's just super, super inspiring. So, um, all right, let's, let's get right to it. You guys are already thrown. Don't, don't, don't start bragging about all the work you did this morning. You don't know how many steps I took. So Steve Thordew, 7,322. Dude, DMB did 12,641 steps this morning. I don't know who you are, DMB, but I think we may already have a winner. Somebody took 7,300. I see 19,000. Oh, 19,762 total for the day someone got up early and got after it so I was proud of myself until DMV came along shame on you DMV this is my show not yours um but there it is guys let's Let's see where we are. 7,904 steps today. So I did 7,904 steps today. So if you did more than 7,904 steps, I want you to put your name into the chat. This is on the honor system. So I need you to take a screenshot of your iPhone and just text that image into the chat so my staff can see it. Man, a lot of you guys beat me. Michael Olain got 8,992. K. Saray. Jessica got 8,500. Dude, Keith Evola crushing it with 10,011 steps. By the way, guys, this is the number of steps you've done before we got on here. My staff is saying some of those in the last 24 hours. It's the number of steps you did today. 13,593 this morning on a walk. Oh, you did 5.6 miles. You dirty little bird. Betty Criado.6 miles you dirty little bird betty criado man you just um ruined everybody else's day got up early and got after it great job debbie um jesse thomas 7600 jessica b 8500 um keith avola 10 000 steps like guys it's just just amazing because um whether you win a prize or not i mean you're all just you know benefiting tremendously from this um you know steady state cardiovascular exercise in my opinion is one of the most overlooked forms of exercise on the surface of the earth we do not really harness the power of just a simple walk. Remember, we talked yesterday about the lymphatic system and how it's a static system, right? Our arteries have the pressure of the heart, our veins have the pressure of the heart, our lymphatic system, the system that really eliminates waste, doesn't have the luxury of having the heart behind it to return waste back to an area where the body can get rid of it. And so walking, contracting our soleus and our gastroc muscles in our calf, using all of these quad muscles and moving lymphatic fluid from our lower extremities and our upper extremities and around our abdomen into an area of the body like the femoral veins where we can actually get rid of it is a major, major win. like the femoral veins where we can actually get rid of it is a major, major win. So it looks to me like Betty Criato, if I'm pronouncing that right, with 13,593 steps might have won today. So Debbie, amazing job. I am sending you an Echo water bottle. So if you paid the $47 to be a VIP VIP you just got five months of your VIP paid for so I'm sending you an echo water filter or an echo hydration bottle today and then we're going to give away a bunch more I want my staff to select the top five you're all going to get echo water bottles the next five people are going to get these cleaners for your countertops. Remember, this thing is amazing. It makes hypochloric acid. You buy this once for about 99 bucks. I use it every day. You fill it with water. You put a little scoop of sea salt in here, and the sea salt, when it interacts with hydrogen gas, becomes hypochloric acid. It is 80 times more virucidal and bactericidal than bleach and it's harmless to human beings you can actually spray it on your skin I'll suggest that but you could it's harmless to pets it's harmless to kids we talked yesterday about the toxicology report that came back from Swifters being one of the absolute worst things for moms and dads to use when you have young children and infants in the house because the chemicals that are in the Swifter are some of the worst forever chemicals. They're meant to be on the floor and on our tile for a reason. And when babies crawl around, they put their little feets and their really sensitive skin against the surfaces that have that. Sometimes they lick their hands. They're taking in a lot of these forever chemicals. And it's just absolutely unnecessary. We don't need the bleaches and the chlorides and the chlorines and the abrasive cleaners and all of these polyphenols and everything else to get rid of germs we can use simple things like apple cider vinegar baking sodas things like hypochloric acid and you can take a whole class of hormone and endocrine disruptors out of your life and actually save a ton of money doing it. So it's 11.15, right off camera is Dr. Geoffrey. I want to bring him on and I want to start opening things up for questions this morning. I want him to talk a little bit about his gut protocol. I want him to talk about the importance of alkalinity in the human body. I just want to just shower you guys with value this morning. And so welcome to our podcast, our simultaneous simulcast here. Dr. Joffrey, let's move this over. You guys got to tell me if we can both be on camera here. Morning. Morning. Good to see everybody. You can probably only see it from the waist down right now. That's good. I made him wear an ultimate human t-shirt. He is wearing his acid kicking t-shirt. I said, take that off. These are ultimate humans here. Right on. So, guys, it's a little weird just looking right into a camera. You can see yourself over here with a little delay. So what I want to do, you know, we just did an amazing podcast. Dr. Joffrey has an incredible story to tell, not just about his own journey with mold toxicity. And, you know, a lot of you guys, we've done an entire podcast on mold toxicity. It's one of those things, mold and mycotoxins are one of those things that just kind of hide in plain sight and create such a diffuse myriad of symptoms that people rarely are are inclined to actually start testing for mold toxicity if you're like us and you live in in southern Florida Miami and Malibu California are the mold capitals of the world you know we have such a temperature gradient and such a humidity gradient that we have a tendency to have lots of mold spores that you cannot see that are hiding behind your drywall. That's one of the reasons why I'm so big on air filtration and using air filters that have an ozone setting. So you can actually run ozone and kill mold spores behind the wall, even kill termites in your crawl spaces. But I want to get back to the 10,000 step challenge. I want to get back to movement and I000 step challenge. I want to get back to movement. And I want to talk a little bit about alkalinity in the body. What do they need to know about being alkaline, eating an alkaline diet? And talk a little bit about gut health. So the main purpose of eating alkaline is not to make your body more alkaline. That's the most important thing we need to know because that is a constant thing. When I'm talking about alkalinity, we're talking about the alkalinity of your blood. It's a constant number. It's tightly regulated. Just like we have a temperature, on average 98.6, it could be a little higher, it could be a little lower, but your body regulates that like a thermostat. The same thing happens with your blood pH. The pH of your blood is so important, if that pH dips down by more than a point, guess what happens? You die. So what you put into your body is critical. There's so many naysayers out there. They're like, oh, you know, this is always talking about there's no purpose of eating alkaline because the blood is always tightly regulated. It does it itself. Yes, I agree. It does do it itself. The whole purpose of eating these strength eating foods that are loaded with minerals, right, because minerals is what we need to alkalize the body to neutralize the acid that we get from common foods that we shouldn't be eating, right? So the whole purpose of putting these foods into the body is not to make the blood more alkaline, not to make the body more alkaline. It's to take the stress off your body's buffering system so your body doesn't have to do the regulating on its own. So let me give you an example. If it's freezing cold, I'm wearing a t-shirt, New York City is where I'm from, and I go outside, what happens is my body has to start to rob Peter to pay Paul. I gotta stay alive, so I'm gonna start to shiver. The shivering is what keeps my temperature at 98.6, but guess what? I gotta get out of the cold, or else I'm gonna be in trouble. I'm gonna die at some point, right? And if it keeps on going on, eventually the body is going to take the blood from the fingers and from the toes to bring it to the more important organs, the visceral, to keep you alive. Now what I can do is I can just put a coat on. The coat is the analogy of neutralizing the acid in the body. So the whole point of putting in foods that strengthen you from the inside out, I'm talking about green leafy vegetables like spinach and kale and watercress and romaine lettuce and cruciferous vegetables like broccoli and healthy plant-based keto fats like avocado, which I call God's butter, has the most important macros that your body can get every single day. Healthy nuts and seeds like macadamia nuts and Brazil nuts, right? Brazil nuts in selenium so important for your connective tissue these are just some examples that your body needs because these foods are loaded with minerals they're low in sugar they're high in minerals and it's these minerals that your body needs to get rid of these dangerous damaging toxins acids that take us out over time so that's the reason why we have to put these foods into our body. You know, it's funny how he and I got together. I started my wellness practice in Naples, Florida. His company, the Acid Kicking Coffee Alkalizer is produced in Naples, Florida. He lives in Naples, Florida. I was actually using his product for a few years before we ever even connected. And then when we connected, I heard about his story, his personal journey with mold toxicity, the journey that he went on with his father, you know, passing away from esophageal cancer, directly related to gastroesophageal reflux and acid reflux. And I think that there are so many of these silent issues going on in human beings. And we, because they are not present enough in our life to disrupt our life to the point where you have to come to a complete stop, we tend to ignore them because they become less and less of a priority because we're able to manage the symptoms with antacids or manage the symptoms with proton pump inhibitors. And all we're trying to do is just make this nagging, you know, acid reflux less troublesome and less of a nuisance in our life. Instead of thinking, well, how do I permanently put this in a rearview mirror? I permanently put this in a rearview mirror? You know, yesterday we talked a little bit about the difference between developing coping mechanisms for things like anxiety, depression, ADD, ADHD, OCD, and developing mechanisms to permanently get rid of them. You see, I would rather, you know, for all of my clients, I would rather you have a permanent solution and not have to learn to cope with a lot of these things and be able to put them permanently in the rear view mirror. And so many of you guys are looking at watching this right now and you're like, oh my God, I'm suffering from acid reflux. My husband, my wife, my father, what have you has been suffering from acid reflux for years. So today might be the day, the serendipitous reason why you're on this podcast today. I keep calling this a podcast. What is this thing? Is it a Zoom? Whatever it is. A live Zoom. Oh, it's a live. So the reason why you're on this live today, you may not even know why you're on here, but it could very well be to get this message. And I believe that there's a lot of serendipity in the world. And we talked about how our faith drives both of us. And I don't believe in coincidences, neither does he. We both believe in serendipity in the world. And we talked about how our faith drives both of us. And I don't believe in coincidences, neither does he. We both believe in serendipity. So maybe the reason you're on this podcast today is to get the information that we're about to tell you and share it with a loved one so you can permanently change the trajectory of their life. to talk a little bit about your gut healing protocol because this is basically free except maybe for some of the testing that you do for your gut but and some of the testing that you might be able to get done for food sensitivities and food allergies but what was astounding about the podcast that we just did was it was one of the few guests that actually gave a very specific protocol which I appreciated for saying what if myself or a loved one right now is constantly taking these antacids and proton pump inhibitors and every time they stop the acid reflux comes back so they just have not been able to get off of these and he walked me through a protocol that he's used in thousands of patients to get them permanently off of of these proton pump inhibitors can we can we talk a little bit about that absolutely by the way you guys finding this valuable if you are can you just throw a thumbs up into the chat or give me a thumbs up into the screen or give me a thumbs up into the screen? Um, cause without interacting with you, sometimes it's hard for me to know if you guys are really enjoying, I got some thumbs up there. Um, Nora, Jeremy, David, all firing the thumbs up. Okay, great. So, um, so let's talk about like one of the, one of the tips that you gave me, you know, 30 minutes before each meal to actually help improve the acid in the stomach. Yeah. Apple cider vinegar is one of my most favorite biohacks. And the reality of this, Gary, is that 50% of people, the research shows, who are actually tuning in right now have reflux. They're on some, not just reflux, they're on some form of PPI or over-the-counter medication to deal with it. Wow. So it's something that we have to take serious. Even if you think that it's not a serious thing. It can become something very serious. This is why I'm so passionate about this. I lost my father to esophageal cancer and that was from silent reflux that was developing over the years that was just not addressed. So this is why it's so important we have these conversations. But what Gary's talking about is 30 minutes before your biggest meal of the day for most of us that's dinner time, 20 to 30 minutes. What you want to do, if you don't... You should do this whether you have reflux or not, because the idea of this apple cider vinegar shot is to optimize digestion. Something I always say is that you're not what you eat, you're what you digest, absorb, and assimilate. So you could be eating the best organic, clean foods in the world, but if you're not breaking down those foods if you're not digesting the foods guess what you're not getting those foods into your cells which equals energy right health is energy but it comes down to the health of your cells the quality of your cells is the quality of your energy the quality of your life so by doing this whether you have reflux or not it's gonna have a massive impact on your digestive process and that in turn is gonna have so many downstream effects on your thyroid gland, your energy, your metabolism. So again, 20 to 30 minutes before your biggest meal of the day. If you don't have reflux, you can go to one tablespoon of apple cider vinegar and you wanna dilute that, very important, don't do it straight up. You wanna dilute that with a third cup of water, ideally warm water, because warm water will actually stimulate the gastrin secretions in your stomach, which makes more hydrochloric acid, right? That's the goal. Because maybe something we should say is that the cause of acid reflux is not too much acid in the stomach, contrary to common sense, right? Yeah. And it's actually too little acid. And by the way, that's supported by 16,000 research journal articles. So the goal as we age is to get stronger, right? But it's also to create more acid in the stomach. So this is a great biohack. So again, you take a tablespoon of apple cider vinegar if you don't have reflux. If you do have reflux, let's go a little more gentle. Let's do a half of a teaspoon so you can make sure it's not going to stir anything up. Add that to a third cup of warm water or regular water, you know, room temperature water, ideally hydrogen water. And then what you do is you drink that on an empty stomach. And then 20 to 30 minutes later, you're going to have your meal. And what you're going to find is that it's going to help you break down your food. You're going to digest your food better. You're going to create less food sensitivities, less inflammation in your gut, less leaky gut. You're going to feed the good guys, not the bad guys. So in other words, you're going to feed less food sensitivities, less inflammation in your gut, less leaky gut. You're going to feed the good guys, not the bad guys. So in other words, you're going to feed your healthy microbiome, the good probiotics, not the stealth pathogens that are most likely living inside your gut that are taking out the good guys. So there's a huge knock on effect to this one little thing that takes literally less than 30 seconds. Dr. Justin Marchegiani That is awesome. And then we talked about, so those are people that are maybe not actively taking proton pump inhibitors, not actively taking Tums or Prilosec or antacids every day. And then there's a category of people that have actually been taking these things for years. And if 50% of the world's population is on some kind of acid balancer, that means that somebody on this live has been on these for years or a loved one that you know has been on these for years, disrupting the entire downstream absorption of nutrients in their bodies and creating consequences that may be silent now. They're going to creep up later in life. So I want to talk about the folks that have been on these PPIs for a long period of time, Nexium, Prilosec, Tums, Rolaids, antacids, what have you. And, you know, the apple cider vinegar is not really enough for them. We have to actually restore the natural acid balance in the stomach and then slowly titrate them off. And, you know, one of the things that he suggested was a test called the mediator release test, an MRT. I'm actually extraordinarily familiar with this test. I'm a huge fan of the MRT test. There's another one called the ALCAT test. I believe that the same PhD developed both tests. What's interesting about this test called an MRT, mediator release test, is that it actually subtracts your existing state of inflammation to give you a true measure of what foods you are sensitive to and what foods you are allergic to. A lot of food sensitivity and allergy testing is very 1993 science. They do not subtract your current state of inflammation, so you get a lot of warnings on foods you actually are not sensitive to and you are not allergic to. They only barely move the needle but because your inflammatory state is so high it shows up on these tests as a very very allergic substance or something you need to completely avoid and in many cases that is not the case so one of the things he suggests is doing this MRT test this mediator release test you do not have to do this test through me or through him you can google MR this mediator release test. You do not have to do this test through me or through him. You can Google MRT mediator release test. It's something you can even do in your own home. You send it back and the report will come back to you and it will tell you exactly what foods you are sensitive or allergic to. And then once they know this, where do they go from there? Well, I think the most important thing about that is now you know what the inflammation levels are coming from, areas that you probably didn't have a clue of before i always like to say we don't die of old age we die of inflammation right um and you're gonna see me start using that on a regular basis guys yeah i just stole another one of his but it's crazy because when people get on a health kick they want to change their lifestyle they want to do a reset they start to put in foods that are healthy when i first did my first m test, some of the things that came up were foods I was eating on a regular basis. Avocado. Avocado. We love avocado. We just had avocado. It's God's butter. It's the most beautiful food in the world. But for me in that moment, it wasn't good for me. Now it's healthy for me, but my body couldn't digest the avocado and assimilate it. And because of that, it was creating inflammation in the gut. So what I had to do is stop the avocado for about three months or so while my gut had some chance to heal. So that when I added that back in, my gut was able to handle this once again. So it's going to give you a list of 170 different foods and chemicals of basically how the inflammation is being created inside your gut that you probably had no clue that was going on. And it's important because when you start to make better changes and better health decisions, now you're making this about your body. This is not just about, hey, I'm gonna follow this specific diet. Like I call it a strength eating diet, right? But this is about your bio-individuality. This is about what you need specifically in this moment to lower inflammation, to increase the nutrients, the micronutrients into your body, into your cells, and ultimately extend your longevity because you're really just coming up with a plan that's going to help your body heal from the inside out. Yeah. You know, I always say that what can't be measured can't be changed. You know, when we have data, and I always encourage people to get data on their bodies, get the DNA test to get data on what you need to supplement with, get a blood test so you know where you stand with your sugars, with your hormones, with your nutrient deficiencies. But if you are suffering from gut dysbiosis, issues with your gut, or reliance on PPIs or these antacids, then we need to figure out, first of all, what's going to inflame and irritate the gut. And then secondly, we need to start walking ourselves out of it. So after we get this information and we, when we take the pressure off of the inflammatory process, which is kind of stage one, where do we go from there in terms of repopulating the flora in the gut and healing the gut? Because, you know, people that have had this GERD for a long period of time they do have some damage probably to the distal part of the esophagus this can become something called Barrett's esophagitis which is a precursor for cancer and so how do we go about healing that he talks about one of my favorite peptides so I'm sure he's going to talk about that now of course yeah so yeah. So the GERD, the reflux, we have to understand is the symptom. That's the effect. The underlying cause is the dysbiosis, the imbalance in the microbiome. So I want everybody to look at their gut like a garden, right? You want lots of flowers. We don't want lots of weeds. The flowers are the probiotics. Pro means for life, probiotic for life. But the reality is that we get, research shows, probiotic for life. But the reality is that we get, research shows, one, one millionth, say it again, one, one millionth the amount of probiotics that our ancestors used to get. It's become critical. You have to supplement. There's five big supplements that I tell all my patients and clients you must take. Probiotics is one of those on the list. Kids need it. We need it. Elderly needs it. And the one that I like when we're starting out is a spore-based probiotic. It's not an alive probiotic. It's the spore, right? And that spore, the good thing about it is if you're taking an antibiotic, it's like it's not going to be destroyed by the antibiotic. A terrain-based probiotic like a lactobacillus will be. So it's got really a bunch of different effects. It's going to help strengthen your microbiome. It's going to build up the flowers. It's going to build up the healthy side of the microbiome that we want. It's also going to nudge out some of the bad guys. We have so many toxins that we're exposed to every single day. They're in the air that we breathe, the water that we drink, the foods that we eat. And guess what? Your gut is your first line of defense. So if we don't have enough of these healthy probiotics, it's going to take you out. It's going to cause leaky gut. It's going to make its way into your bloodstream. And from there, it's a systemic highway to the rest of your body, including your brain, right? So the probiotics are going to kill off some of the bad guys. But what they also do is they actually help to heal and seal your gut lining. We were talking about a study of 20 year olds and they took a spore based probiotic for 30 days and on average 44% healed their leaky gut. 44%, it's crazy, right? So just something like that by itself is gonna have such a powerful impact at getting the strength back into the gut. And you know, I talk a lot about different types of probiotics and how some of them are really missing the mark. Spore-based probiotics actually fall into a little bit of a different category. But when you take bacteria and you put them in a capsule, remember the whole idea of having a stomach, the reason why we have a stomach is to kill bacteria. That's the role of the stomach. There's not a single probiotic bacteria that can survive stomach acid. So why then would we take probiotics, put them in a capsule, not protect them from the pH of the stomach, swallow it, have it open in the stomach, and just let those bacteria bathe in stomach acid so that you just pass a bunch of dead bacteria into the stomach? This is why lots of you that have tried probiotics, and probably just about everybody on this live has, you've tried probiotics and you haven't really had any success. probiotics and you haven't really had any success it's because that probiotic manufacturer is not protecting the probiotic from the stomach acid and you're just passing dead bacteria you know into the gut and you're not getting all the benefits and you're reading the label going wow this has 13 strains of probiotics it has 55 billion what they call cfc who cares about that don't worry about the number yeah colony forming units and so And so, you know, sadly, most of the consumers, and there's not a lot of good education about that on probiotics out there, so we buy the most expensive, the one that's going to be refrigerated because somehow that seems like it should be the best one, and we buy the one with the highest number on the bottle, 55 billion colony forming units or CFUs. Not nine strains, but 13 strains, and then we swallow it, and they all bathe in the stomach acid and they die and you get no positive effects so spore-based probiotics are amazing and doing a test to find out what your food sensitivity sensitivities are and your food allergies are and then also adding digestive enzymes digestive enzymes are something that you actually eat midstream so you know I always tell clients to consume food, a little bit of your food, swallow the digestive enzyme, and then continue to eat after that. So we start taking some of the pressure off of the gut biome and we start handing it contents that have already gone through the first stages of digestion. So there's less gas, less bloating, less diarrhea, less constipation, less irritability while you're trying to repopulate the gut biome. And if we do nothing else for you today and teach you how to repopulate your gut biome, I promise you that alone is going to permanently change the trajectory of your life. And I really liked having him on because we had so many different synergies. We both believe in minerals like the Baja Gold Sea Salt. That to me is the cheapest life hack in the world. We talked about the number of clients we have that are mineral deficient. Most of us understand protein deficiencies and not getting enough calories, but we really don't understand that when we deprive the human body of basic raw materials like the 91 minerals that we could get by spending about $15 a year in a good mineral salt, we get the expression of those deficiencies. One of the other areas where we had a major crossing of the past that I want to share with you guys is on hydrogen water. You hear me talk about it all the time. at a major crossing of the past that I wanna share with you guys is on hydrogen water. You hear me talk about it all the time. I never leave home without a hydrogen water bottle. There are thousands of knockoffs out there. You've probably seen on social media, all of these companies, some of which I am suing right now, that are using my name and my likeness and videos like this to sell cheap Chinese knockoff water bottles. The proton pump in the bottom of this matters. This ionic membrane really matters in terms of creating high parts per million hydrogen water. So there's an amazing website, I think we showed it yesterday, called hydrogenstudies.com. If you wanna read the studies for yourself, go to hydrogenstudies.com. When want to read the studies for yourself go to hydrogenstudies.com when you get there you can actually search specifically for hydrogen water and you can specifically eliminate any animal studies and you can just look at the studies on human beings I promise you you will be so excited about the benefits of hydrogen water when you get off of that site you won't know what to do with yourself so you can actually look at sports performance. You can look at chronic illness like Parkinson's, autoimmune, gut dysbiosis, eczema, psoriasis, all kinds of inflammatory conditions. Read for yourself. Don't take my word for it. Go to the conclusion section of those research studies and just read the conclusion section on the impact of hydrogen. So can you talk a little bit about hydrogen water and some of the benefits of hydrogen water so they don't think I'm nuts? I think that already. It's alright now. Well, you've got to look at the gut again. Why do we need hydrogen water? It's the only water that I drink, by the way. Your gut should be making hydrogen every single day. It should make 10 liters of hydrogen, but the problem is... 10 liters of hydrogen gas. Hydrogen gas. Every day. of hydrogen. But the problem is... 10 liters of hydrogen gas. Hydrogen gas. Every day. Every day. But the problem is it's not happening. Sounds like a lot of... Yeah, it's research proven, though. Hydrogen is critical. And because we're not making it, why are we not making it? Because our guts have become so toxic, so acidic. Again, being the first line of defense, it just gets bombarded every single day, and things just don't start to work properly. That's why we support with digestive enzymes and the things we were just talking about to take stress off of the gut so the gut can heal itself. That's why we do it. It's the same thing with hydrogen. So we're not making enough hydrogen. So what we're doing, we're biohacking the body by using something like this. You press that button, it makes that hydrogen water. And what I love about hydrogen is that when you drink it, you're doing a few different things for the body. Number one, it's a powerful anti-inflammatory. I mean, this goes to the brain within 30 minutes, lowers inflammation of the body and the brain. This is why people deal with any sort of brain issue from ADD, ADHD with children, anxiety, depression. It's a big aspect of inflammation that usually comes from leaky gut, by the way, all the way to things like dementia, it's a big aspect of inflammation that usually comes from leaky gut by the way, all the way to things like dementia, Parkinson's. There's a study a few years ago in the New York Times they were showing that Alzheimer's parallels inflammation of the brain. I saw that, I saw that's a neuroinflammatory study that's on hydrogenstudies.com. Over a thousand studies. And the Parkinson's patients were having tremors they would drink this hydrogen water and they would stop. Why? Because it was lowering inflammation. When inflammation lowers, the body can actually do what it was designed to do, function properly, heal. What I also love about it is that if you've taken an antibiotic or if you're on an antibiotic, what do people always tell you to take when you're on an antibiotic? They say take a probiotic. Well, now we know you wanna take a spore-based probiotic because that really helps. But when you're drinking the hydrogen water, this is the most important thing you could do when you're on an antibiotic because it's the only therapeutic agent that is going to outpace the damaging effects that that antibiotic has on your microbiome. What you have to understand is that one dose of antibiotics will wipe out your microbiome, your gut wall, for two years. Two years. That's crazy, right right think about things like sugar and gluten and artificial sweeteners they're gonna poke holes in your gut but when you take an antibiotic which means against life that is like a napalm bomb going off in your gut so if you have that hydrogen it's gonna help it's gonna help protect that gut lining by building up the anaerobic microflora in your gut faster than the antibiotic is taking it out. The other thing that they did, again, so many studies, but just on colorectal cancer cells, when they combined the molecular hydrogen with something called 5-fluorosil, there was 100% apoptosis of that colorectal cancer cell. What that means in English, when that hydrogen hit that cancer cell, that cancer cell was dead. Why? Because it's such a powerful antioxidant. It's probably the most powerful antioxidant you can give your body because it's selective, which means one molecule of hydrogen will take out two free radicals of the hydroxyl-free radical. So basically, it's going to reverse that rusting and rotting and aging process that's going on inside our gut, our brain, and our body. Dr. It actually causes those hydroxyl-free radicals to become water molecules. Right? that rusting and rotting and aging process that's going on inside our gut, our brain, and our body. It actually causes those hydroxyl free radicals to become water molecules. That's right. Right? So, you know, hydrogen is H2 and these hydroxyl free radicals are O2. So they, they, they, it essentially makes these hydroxyls, OH, I'm sorry, OH, it turns these into a water molecule. So you take something that is pro-inflammatory and you actually turn it into um you know a source that's actually healthy for the body that's the power of hydrogen gas and hydrogen ions in the human body and um i know that you know ancestrally we got a you know a lot more hydrogen rich water if you actually scoop water out of a running stream you see that it actually has high amounts of hydrogen you'd also see that it has high amounts of trace minerals like you get in the Baja gold salt, because it'll have all the boring little things that nobody hears about, like molybdenum, selenium, manganese, boron, all of these trace minerals that are so important for cellular function. And, you know, we had a long talk today about how getting back to the basics, you know it it seems like a lot of fear-mongering and i really try not to say don't eat this it will kill you don't drink that it'll kill you don't do this you know it's going to be the end of life but the truth is all we're trying to do is navigate around all of the modern day nonsense and get us back to the basics um you know i've talked about the soil lineage study that was done decades ago in the 40s on things like spinach and broccoli and cauliflower and the nutrient density of the foods that we used to eat decades ago. Our grandparents and our great-grandparents ate a lot more nutrient-dense foods than we do today. And you talked about a study on spinach that I thought was really fascinating. The amount of spinach per pound that you would have to eat now to eat weight just a few ounces back then. In 2015, it would take you 60 servings of spinach to give you the same amount of mineral content, especially iron, that one serving gave you back in 1948. And now present time, that 60 servings is now at 75. So 75 servings of spinach to equal pop in trouble i actually just did a podcast with him at uh universal studios a couple a couple weeks ago i hope you guys saw it it's uh i went to universal studios and i did pop i and olive oil and i did um breath work on on access hollywood he was actually looking pretty good um so maybe he's eating 65 pounds of spinach a day i don't know but 65 servings equals a single serving so it is an absolute fallacy that we can get the nutrient density that we think we can from from our fruits and our vegetables i am i am a staunch um um supporter of the fact that we actually need to be supplementing in order to get those uh drink of greens that's what i like to say nutrients um so we have about 15 um more minutes i know i've commandeered this entire thing um before really taking a couple of questions i usually reserve the questions for the vips guys today please today is the day to become a vip your first month is free anyway i want you to jump into the room with dr geoffrey and i and jelly roll um i'll even see if i can get jelly roll to sing us a song um but he um he sent me a video while he was out um this morning on can you take too many prebiotics and or probiotics yes i want you to answer that you can take too much of anything you know we're talking about the of hormesis, which means you take a certain amount of something that's beneficial for the body. Then you go too much. It becomes toxic. Even too much water can kill you, right? So prebiotics, I like to get through my food. If you're eating at least 80% plant-based in your diet, you're going to get the prebiotics that you need. I'm talking about garlic and artichokes and onions. You know, there's so many different ways to get that. And prebiotics are basically the food for the probiotic. But again, if you're taking a spore-based probiotic, all you need to focus on is taking two of those capsules at your largest meal of the day. And that's whether you're eating fermented foods or not, because you are going to get some probiotics from the foods that you eat. One of the things I always like to tell my my clients and everyone is that you can't supplement your way to have a bad diet so we have to make sure that our diet is healthy so try to get some things like kimchi and sauerkraut and pickles because that's some healthy forms of probiotics but you still have to supplement with a probiotic as long as you're doing two largest meal of the day you you'll be golden. Awesome. Are there any other ones that were creeping up to the top, Gio? They asked what brands for probiotics. Oh, what brands for the probiotics? Go ahead. Okay. I don't have any affiliation to these brands, but I love Mega Spore Biotic. I love Thrive Probiotic. They're both spore-based probiotics. They have the top five spore-based ingredients. They're pretty much the same as ACT. So those are the two that I kind of go back and forth with. Once in a while, I will throw in a refrigerated type of probiotic just to get more of those terrain-based types of probiotics because that will strengthen. But as Gary was saying, it's not going to survive the road through that digestive system, the harsh terrain, like your bile salts and your acid in your stomach as much as the spore-based will. So it's really good to rotate, but I always like to kind of do more of the spore-based probiotic for that reason. Yeah, and I don't make a spore-based probiotic, so definitely one of those brands would be a great idea. I do make a terrain-based probiotic that does survive the stomach acid. So you can go to 10xhealthtest.com and you can actually buy the terrain-based probiotic. So that's great. So with something like that, you'd want to just rotate, do a spore-based for a month, do Gary's for a month and keep on rotating back and forth. Yeah. I mean, I never have got issues since I've done that. What else do we have? We've got about uh 12 minutes what are some good fermented foods to eat oh i love kefir fermented foods i personally get them i get fermented sauerkraut and and vegetables from an amish farm here in in southern florida so i get them in these sealed glass jars and they deliver them every uh about every 10 days or two weeks um but what are some great fermented i love pickles pickles are great um kimchi is a really good one i used to hate sauerkraut but if you actually prepare a sauerkraut right and you actually mix it with really basic foods more basic bland foods um i think it tastes amazing yeah so i've been eating a lot of sauerkraut and fermented vegetables lately this amish farm does an amazing job at their kefirs and their and their fermented foods what else we got always kombucha good I personally am NOT a big fan of kombucha because of the sugar content and kombucha um what's your opinion on itucha? It's so much fermentation. There's also a lot of alcohol byproduct. I mean, there was some talk showing that because the fermentation is so high, it's gone beyond the 0.8 legal limit. And then you're dealing with all the yeast. So you have to kind of, everyone's bio-individualistic, you know, try it out, see how you feel on it. Does this make me feel good it does this make me feel good does this make me feel you bad I think it's so important that we get in tune with our body it's something that I don't personally drink but or or or suggest or recommend yeah with that said I don't drink kombucha's either can they take apple cider vinegar while they take body health digestive enzymes absolutely absolutely yeah absolutely Can they take apple cider vinegar while they take body health digestive enzymes? Absolutely. Absolutely. Yeah. Absolutely. Yeah. Really cool. Oh, and that peptide we were talking about was BPC-157. Oh, gosh. That's another one. Yeah. We forgot to mention that. BPC-157. I personally think this is a miracle for healing the gut it's a gastric pentadecapeptide it's a peptide synthesized from gastric juice so for people that have had gastroesophageal reflux disease for people that have had um uh you know acid reflux for a long period of time or for any kind of gut dysbiosis leaky gut there's some pretty impressive clinical trials um that are in the public domain on bpc 157 body protection compound 157 get your doctor to prescribe it for you or you can um the fda just banned it in compound pharmacies but you can get it in nutritional grade form um you know i i've seen our clinical team work miracles with that. It's actually good for what I call nips and bibbles, little micro tears, rotator cuff, elbows, knees, hips, shoulders, low back. So it's great for recruiting platelets to the site of injury to help repair damaged tissue. It's great for post-surgical repair, according to the clinical studies. It's not my opinion. That's according to the randomized studies it's not my opinion that's according to the randomized trials on bbc 157 it's a pretty innocuous and is considered you know pretty safe peptide but if you have gut issues that's part of your gut healing protocol yeah i like to open one up and then put that in water and drink that on an empty stomach because it also heals ulcers so many people with esophageal reflux gird they get ulcers they get ulcers in their stomach, esophagus, even in their throat. So it helps to heal that tissue. And then like you said, the downstream issues, arthritis is a big one. It just helps that as well. So you can take them, swallow the capsule, and then you'll feel so many changes in your extremity joints. Like for me, all the adjusting over the years, I feel it on my joints and my hands. Literally within a few weeks, it's incredible. The pain level went down from like a 9 to a 2. So it's just a very powerful, as you said, innocuous supplement that really works. Go ahead. I was just asking, what's y'all's take on natural flavors? Natural flavors? Yeah. Well, it depends. Yeah, it depends on what you're eating. Sometimes these labels will say natural fruit flavors. That's sort of code word for ain't got no fruit. So that's really the code word for ain't got no fruit. This is a pandemic in flavored yogurts. You know, I see people all the time that tell me some of their favorite foods are these gogurts and different colored, different flavored yogurts. They'll be strawberry natural fruit flavored yogurt or blueberry natural fruit flavored yogurts. They are notorious for using multiple food dyes, high fructose corn syrup, and what they call natural fruit flavors to actually replace fruit. And when you see that very often it actually means that there's no fruit in that product and so one of the lateral shifts that I did recently you guys have probably seen this is where I just took this woman that just loved to eat these dam and fruit on the bottom yogurts and these there was another one go Gert and a few others and I and I spun the yogurt around and we looked at the nutritional content it had 54 grams of sugar it had four food dyes by that by the way food dyes in many cases are antimicrobial so when you claim that there's probiotics in a in a you know yogurt and then you mix that yogurt with food dyes, the food dyes kill the bacteria in the yogurt. So actually you're eating the yogurt because you think you're getting probiotics, but the food dye that's in there has actually killed the probiotic. And so had four food dyes. And then the yogurt was blueberry flavored, but it wasn't this natural flavoring, but there were no blueberries in it. And so I said, look, if that's the one that you like to eat, you like this blueberry flavored but it wasn't you know this natural flavoring um but there were no blueberries in it and so you know I said look um if that's the one that you like to eat you like this blueberry flavored yogurt we're going to actually go out we're going to get whole fat Greek yogurt we're going to put that in a bowl we're going to take a fistful of organic blueberries that you can buy in a little basket and we'll just take a teaspoon of monk fruit and we'll mix that up in there and you tell me if you can tell the difference we didn't change the flavor profile we didn't change the um cost in fact the cost went down because when you're buying them in those little um cellophane tear-off packs they're way more expensive per unit volume than if you buy a quart of whole fat greek yogurt and a basket of actual real organic blueberries and and a bag of monk fruit um monk fruit and you'll get a lot more servings out of it you'll drop the cost and then the nutritional profile skyrocketed because in the in this highly processed yogurt there was no fat it was very very high in added sugars and there was virtually no protein when you eat you know an organic whole fat g Greek yogurt, especially A2 yogurt from an A2 cow, less beta casein, then you get the same flavor profile. You actually save money and you skyrocket the nutritional content. So it's kind of guilt free. How crazy is that their selling point is low fat. Right. First of all, we want the fat, but we want the healthy fats. But they take out the fat, now the food tastes like cardboard, so they've got to bring something in to make it taste better. There comes the 54 grams of sugar. And then they'll say zero sugar. Then what are they bringing in next? The artificial sweeteners, which is even worse than sugar for your microbiome, for everything. I mean, artificial sweeteners, they just label finally after 40 years a potential carcinogen. Yeah. I mean, we've been talking about that for years. Yeah, totally. So you've got to be – anything that says low-fat be mindful be scared yeah I think we just got to stop blaming the butter for what the bread did that's right so true it depends on what your vitamin d3 levels are you know I mean usually 5,000 I use with about 80 micrograms of k2, a specific form of K2 called MK7. But this is why I think you should get data on your body. I think you might be surprised, especially if you're darker complected, how low your vitamin D3 levels really are. If you're Latino, African-American, at least, or if you're any of those populations that have darker skin complexions, you need considerably more sunlight to generate the same amount of I use of vitamin d3 25 D hydroxy as as someone with light skin and so huge idea to get your vitamin d3 levels checked you know functional range ranges for vitamin d3 tend to be 60 to 80 nanograms per deciliter most people are below 30 nanograms which is the is the bottom end of the scale and here's another one of those nutrients that when it's depleted it has all kinds of downstream consequences that you just can't imagine i mean it really does weaken the immune system it leads to osteopenia osteoporosis by the way that's not my opinion it's well documented there's even a um a study on women that it reduces the risk of breast cancer to have vitamin D3 in the 60 to 80 nanogram range if you want to see that study send me an email to info at the ultimate human.com I have my team actually send you that study women definitely need to adhere to a good vitamin D3 supplement protocol. So we got about three minutes left guys. Hopefully those of you that are not VIPs, you are going to the ultimatehuman.com forward slash VIP. Maybe my team could throw the link in the chat. It's in the chat. Today's the day to become the VIP. Your first month is free anyway, if you just did it for today's VIP chats, you can win a water bottle and a cleaner. And I'm about to get the announcement on what treadmill sponsor we got, because I'm also going to give away a treadmill to one of the winners for getting in their 10,000 step challenge three days in a row. And then we're going to, of course, we're going to have Jelly Roll, a country music singer on. We're going to talk about his health journey with me and with 10X Health. And we're going to talk about how important 10,000 steps a day is to him on his weight loss journey. And the doc's going to stick around. Will you stick around for a minute? Absolutely. When we go into the VIP chat, that would be great. And then we're going to just open it up to direct questions so we can take a deep dive. Because the idea is to add so much value to you that you are embarrassed and ashamed of yourself. And just kidding. Add so much value, you know, that you keep attending these challenges. And you help me get the word out about simple changes that people can do to totally change the trajectory of their life that are free or very, very inexpensive. Cole, how are we doing on the treadmill? What is the manufacturer? Can you give me a picture of it or something? All right, guys, we are heading over into the private VIP chat. I hope that you guys feel that we provided you value today. Thank you so much for your trust and confidence in joining the 10,000 Step Challenge. If all you did was do 10,000 steps today and yesterday and the day before, then my mission is accomplished because my mission is to get people moving, to fight the sedentary lifestyle being the leading cause of all-cause mortality. I've gotten so many incredible messages from you guys. I even got some very tear-jerking videos from a few of you moms. I deeply appreciate you guys being vulnerable and sending me those videos and just telling me what an impact it has had on you to just realize how easy it is to get 10,000 steps when you actually broke it up into, you know, easy bite-sized pieces. One of you actually texted me that you walked out and did the steps during your lunch break and got locked in the stairwell. So I don't take any responsibility for that, but I do the stairs next time, put a little wedge in the door. So you get back into the office credit for doing stairs. Yeah, no, I give them all the credit for doing, doing the stairs. I just don't want them to be unemployed after my 10,000 step challenge. Um, so I love you guys. God bless you. I will see all of the VIPs in three minutes in the VIP."
with open ('gary_vL3_2.txt','w+') as f:
    f.write(whisper_vL3_2)

In [ ]:
#whisper_vL3_3 = "If you want to strip fat off your body, there is nothing, no type of cardiovascular or weight training that comes anywhere close to... Gary Pratchett! He spent 20 years working in life insurance, predicting when people were going to die to the nearest month. And now he's on a mission to extend your life. Most people, they're walking around right now at about 55% of their true state of normal. There is an element missing from their body that would make the difference between them being an average person and being a superhuman. How? Everything that we put into our bodies gets converted into the usable form. If you cannot make this conversion, you have a deficiency. And it is this deficiency that leads to these conditions. They have accepted something as either a consequence of aging, stress, or their environment. This is not a consequence of aging, stress, or their environment. This is not a consequence of any of those things. I'm going to tell you exactly how to find out what it is that's missing so you could thrive in a way that you probably never thought possible. Let's use data-wise as an example. So LabCorp calls us and says, hey, we have a life-threatening alert on a patient. He had all of these conditions. I've been talking to doctors. None of them could fix any of my problems. He said, I'm surprised you can even sleep through the night without choking, gagging. He's like, slammed his hand down. How did you know that? And I said, if you don't do what we're going to ask you to do, you have a life expectancy of 10.4 years. And in 10 weeks, he had such a material change. And he was like, I had no idea I could feel this good. And his life expectancy? Almost tripled. Someone who's just heard that at home, where do they start? So. I just want to start this episode with a message of thanks. A thank you to everybody that tunes in to listen to this podcast. By doing so, you've enabled me to live out my dream, but also for many members of our team to live out their dreams too. It's one of the greatest privileges I could never have dreamed of or imagined in my life to get to do this, to get to learn from these people, to get to have these conversations, to get to interrogate them from a very selfish perspective, trying to solve problems I have in my life. So I feel like I owe you a huge thank you for being here and for listening to these episodes and for making this platform what it is. Can I ask you a favor? I can't tell you how much you can change the course of this podcast, the course of the guests we're able to invite to the show and to the course of everything that we do here just by doing one simple thing and that simple thing is hitting that subscribe button. Helps this channel more than I could ever explain. The guests on this platform are incredible because so many of you have hit that button and I know when we think about what we want to do together over the next year on this show, a lot of it is going to be fueled by the amount of you that are subscribed and that tune into this show every week. So thank you. Let's keep doing this. And I can't wait to see what this year brings for this show, for us as a community and for this platform. Gary. Yes. I, this is the first time I've started this podcast in this particular way, but I was, but this is where I wanted to start it. It's with a slightly curious question, which is there's people that have just clicked onto this podcast to listen now, right? YouTube, Spotify, Apple, wherever they're listening. Why should they stay and listen to the message that you have committed your life to spreading? Because everybody wants to be better, right? I mean, everybody wants to improve their health. And most people, especially young people, probably don't realize that they're walking around right now at about 55%, maybe 60% of their true state of normal. There is a nutrient, an amino acid, a substrate, a compound, an element missing from their body that if they knew what it was, would make the difference between them being an average person and being a superhuman. They have accepted something as either a consequence of aging or a consequence of stress or a consequence of their environment, something like brain fog or repeated poor sleep or weight gain or water retention or not the healthiest response to exercise or brain fog or poor short-term recall or any number of things that they've accepted as consequence of life, of aging, of stress, or what have you, that's not a consequence of any of those things. It's a consequence of missing raw material. And I'm going to describe that in detail and tell you exactly how to put that raw material back into your body and how to find out what it is that's missing so you could thrive in a way that you probably never thought possible. Who are you? I'm a human biologist and a researcher, biohacker. My background is in, of all things, mortality science. My undergraduate degrees were in biology. My postgraduate degrees are in human biology i'm fascinated by the human body um its form its function how it can improve its performance and i'm a researcher you know i spent 22 years as a mortality expert in the insurance industry which meant that if we got five years of demographic data on you and five years of medical records we could tell the insurance company how long you had to live to the month. To the month. To the month. And I get a lot of flack about that. People say, oh, if you could predict death to the month, you'd win a Nobel Prize or, you know, only God can tell you when you're gonna die. I mean, and that's very true. I mean, obviously if we do a mortality prediction on it, it doesn't mean December 12th, you know, 2065, you're gonna drop dead on that day. But it is very accurate science. You know, if you think of the most successful financial institutions on the planet as a category are insurance companies. If you look at the failures during the 2008-2009 financial crisis, which was a global crisis, you know, in the US, we had 364 banks fail. You didn't have a single life insurance company fail. No other financial institution, no other investment enterprise, hedge fund, angel investor, venture capital firm would ever put tens of millions of dollars at risk on any kind of investment on one single variable. Yet every time a life insurance company bets on your life or issues an annuity or a reverse mortgage or any number of other financial services instruments that are based on mortality, they're only betting on one factor. They don't care where you are on a mortality curve. They want to know how many more months does this person have left on earth. And it is some of the most accurate science on the planet. And they have perfected this. And the good news is that this science is very accurate, and I'm going to share a lot of the details of that today. But the downside of this industry and the reason why I left it was that during my tenure in this career, I was not allowed to have any contact with the patient or any contact with the treating physician. Now, I'm not licensed to practice medicine. I'm a human biologist. But if I was reading a medical record and saw a life-threatening drug interaction, I could not contact the patient. And this database where this information is drawn from, if this database could see the light of day, I believe it would permanently change the face of humanity. It would upend modern medicine in a way that would be completely catastrophic. Because you see, they have information that no other database has, no other research study, no other financial institution, no other university has. And that is that they know the day, the date, the time, the location, and the cause of death for hundreds of millions of people that they have all of these records on. You know, if I'm a cardiologist and you come in to see me and I put a heart stent in your heart, well, when you leave my office, I don't know what happens to you two months later, two years later, 12 years later. I don't know if you died as a consequence of something that went wrong with the procedure I did, or if you died of something completely different. I don't know exactly what kind of impact on your life that procedure had or didn't have, unless it's contained in a very short-term clinical study. And not that I want to go into the whole science of mortality, but if you had access to this database, you would realize that the most common ailments that we suffer from are not diseases and pathology that are happening to us. They are things that are happening within us. You know, I always say that if I was to boil my entire career down to a single sentence, it would be that the presence of oxygen is the absence of disease. And nothing is more impactful than that statement. The presence of oxygen is the absence of disease. I have yet to find a single disease etiological pathway, something that's happening in the human body, causing pathology, causing disease, dysfunctionological pathway, something that's happening in the human body, causing pathology, causing disease, dysfunction, that doesn't have its roots in a lack of blood oxygen or its roots is not aggravated by something called hypoxia, lack of oxygen. In fact, all human beings die of the same thing. We all leave this earth the same way, right? We all die of hypoxia, lack of oxygen to the brain. That's the definition of death. Only we tend to think of it as an event, a gunshot wound, a bus, car, a stroke, heart attack. But the truth is we are on a hypoxic curve, meaning we are accelerating quickly or accelerating slowly towards the grave. voluminous data-driven industry was that when you deplete certain raw materials in the human body, you get the expression of that deficiency. So what I mean is, you know, we've all heard that disease, you know, travels in families, or you might have genetically inherited hypertension, or genetically inherited hypothyroid, genetically inherited drug and alcohol addiction, diabetes, depression, anxiety. These things tend to run in families. So we've accepted the myth in most cases that these are genetically inherited diseases. But think about this. The next time one of your listeners gets told that, oh, you have genetically inherited hypertension, what we call idiopathic hypertension, or genetically inherited hypothyroid, or if you have this genetically inherited disease, or that genetically inherited disease, stop your physician and say, wait a minute, we've mapped the entire human genome. So we know every gene in the human body. Can you tell me what disease I inherited from my ancestor that's actually causing this condition and watch their face go blank because in the majority of cases we have accepted that disease travels in families when it's actually not the disease being passed from generation to generation it is the inability for their body to refine a certain raw material which causes a deficiency which leads to that disease so in other words there's not a single compound known to mankind, not one. No mineral, vitamin, amino acid, protein, carbohydrate, no nutrient of any kind, no element known to man that enters the human body and is used in the format that we put it in. Everything that we put into our bodies gets taken in by the body and converted into the usable form. If you cannot make this conversion, you have a deficiency. And it is this deficiency that leads to these conditions. And so if you go hunting for that deficiency and you supplement, not for the sake of supplementing, but you supplement for the sake of deficiency, magic things happen in human beings. There is a superhuman inside of every person listening to this podcast. And if they actually were able to test themselves, do a genetic test, and look at what's called their methylation, how their body refines raw materials, and they were able to replace the deficiency that is holding their body back from creating adequate levels of neurotransmitter and achieving normal mood, moving contents through the gut and achieving normal gut function, elevating emotional state, deepening their sleep, quieting their mind, all of these conditions that society has labeled different pathologies and disease, then they would begin to thrive in a way that they just never thought possible. It's so blatantly clear how passionate you are about this subject matter, which begs the question, you know, I remember when I was reading through your story, I uncovered that you'd read hundreds of peer-reviewed papers. I think the quote was something along the lines of, if there's a peer-reviewed paper on the subject matter of biohacking and disease and longevity and those kinds of things, you've basically read it. So this begs the question to me, like why is one individual so unbelievably passionate about this subject matter? And I would like you to take me as early as you possibly can to answer that question in your own life. Where did your obsession, passion, where was it fostered and nurtured so that you spent and committed your life to this subject matter? Where did that begin? So it actually began, I grew up on a very large tobacco farm. My father was a Navy captain. My mom was a flight attendant. She worked for the airlines. But we had a farm that we leased out to other farmers and surrounding this 300 acre tobacco farm were all kinds of animal farms. There were cattle farms, there were chicken farms, there was horse farms. And I was an only child. My nearest neighbor was miles away. In fact, from my home, you couldn't even see another house from my parents' house until I was in my early 20s. And so for me to to play with another child i had to get on a bike and spend half an afternoon biking to their house so i got very familiar with the farm and i was always just fascinated by the veterinarians that would come onto these farms and fix horses and fix cattle and fix sheep and fix chickens and i always thought it was so cool that these animals could be laying on the ground, seemingly about to die, and the veterinarian would come on board and do a bunch of stuff. And the next thing you know, this horse was up, and a few days later, it was galloping back in the field. Or you had all kinds of strange things that would happen on this farm, and there was always someone arriving to just fix it. And I started to get fascinated with medicine, I guess, in that way. I always found it as a way to take something that was sick and helpless and sort of help it get back to normal function. And I found out in the eighth grade, sometime during my eighth grade year, that I was clinically photographic. So I have a clinical level of photographic recall, which is different than just having a visual memory. I'm clinically photographic. So I have a clinical level of photographic recall, which is different than just having a visual memory. I'm clinically photographic. So I have a voluminous capacity to recall things that I read, even if I don't understand them, which is why I never read for pleasure. I'm very cautious about what I read. I'm very cautious about what I look at because I record everything. So I can't be flying on American Airlines and take the seatback magazine out and read the seatback magazine because three months later, I'll tell you where the sales center is for a condo project in Buenos Aires. So I can fill my brain with senseless things like that and record it and regurgitate it, or I can fill it full of things that fascinate me. And so I naturally gravitated towards science. Is that what they call a photographic memory? Photographic memory is usually someone that can remember seven numbers in sequence. They have extremely good recall for things that they visualize. Clinically photographic is being able to regurgitate voluminous amounts of information with incredible accuracy. So I still remember, you know, section 15.2, so paragraph B of my first employment contract that I signed 28 years ago. Really? Really. Yeah. I remember that. It was actually with a trading firm that I signed it with. So, you know, when I read peer-reviewed papers in scientific journals, it doesn't make me more intelligent than someone because very often I can recall information that I don't understand. So I can regurgitate voluminous amounts of information. So subjects that rely on rote memorization like chemistry, biology, neurobiology, microbiology, a lot of these sciences don't actually make sense, right? That you just have to memorize how they operate. Chemistry in a lot of ways't actually make sense, right? That you just have to memorize how they operate. Chemistry in a lot of ways doesn't make sense, right? You just have to remember what happens when you put these two elements together, they create this. When you pull a carbon bond or you double a carbon bond here or hydrogen bond there that the molecule shifts in this way. And so I gravitated towards science. I got my undergraduate degree in biology. How did you do? I did very well. I actually found it quite the breeze. You know, when I was an undergrad, my biology and chemistry were considered the toughest majors. And I went to a branch campus, University of Maryland. A lot of my roommates were political science, not busting on political science majors at all, or philosophy or psychology. And they got to party all the time. And, you know, I had really intense classes, you know, morphology of thalafites and chemistry and, and, you know, biomechanics and, and science and, and a lot of plant biology courses and human biology courses. But I was able to have the same kind of social life that they were because I would, you know, record a lecture and then I would regurgitate the lecture when I needed it during a test. Is it a gift or a curse? Because with all things in life that appear to be gifts, there's often some kind of hidden curse. Well, the curse is that I can't, I rarely read for pleasure. It's very difficult to read for pleasure because I record all of that information. So when I'm going in to read a book, a lot of people read a book that they didn't really like and it doesn't stay with them. Or they read a fiction novel, it doesn't really give them any benefit. It doesn't feed them, but they do it to relax or they do it for entertainment. For me, it would take up storage, right? I would record that information. So there's no need for me to have that information in my possession. I find other ways to relax and meditate and unwind, but reading for pleasure, just not one of them. So I guess that could be a curse. But then I went to grad school for human biology. I went to another four years of school for human biology. So I had eight years of science. And then I was either going to go into chiropractic or into orthopedic medicine. I really liked the idea of orthopedics because, again, an orthopedic surgeon was somebody that fixed people. And an internist was somebody that just managed disease you know they took a obese hypertensive diabetic patient and they just kind of managed their case throughout their lifetime but i felt like an orthopedic was somebody that you know you came in and you had a broken leg and they you know fixed your broken leg and and you were back to normal i like the idea of fixing things and kind of returning function back to people. And I took some time off, you know, after I got my human biology degree and I ended up going into a rare area of science, this mortality science for life insurance and secondary life insurance, where you would take medical records and demographic data, and you would use this in what's called a probabilistic model to predict life expectancy. And I was fascinated by it because I love the big data nature of it. And for years, I actually subscribed to the fact that this was just data, and that I had no influence on this person's outcome. I didn't put them in this position, nor am I responsible for getting them out of this position. But eventually, I think, as both my faith grew and my awareness of the fact that there were human death for the monetary gain of an investor versus taking this information and helping people live healthier, happier, longer, more fulfilling lives. Because by studying medical records, and I read eight to 10 hours a day, six days a week, thousands and thousands and thousands of medical records and extracted from those how it would impact somebody's outcome of their life, how it would impact the length of or the shortness of the duration of their life, their health span or their lifespan. And once you realize there's human beings on the other side of the spreadsheet, you start to feel like you're sitting behind a thick glass wall, just watching blind people walk into traffic. you're sitting behind a thick glass wall just watching blind people walk into traffic and towards the tail end i used to artificially kind of manipulate the record i would say well what if i what if i artificially fix this person's d3 deficiency what if i cured the anemia what if i actually um noticed that they had a gene mutation called mthfr instead of trying to fix them with folic acid i fixed them with methylfolate And you would just see the life expectancy jump. In the model? In the model. Yeah. Right. And this model was based on very large data. So it was very likely to have a real life impact. And when I say we predicted death to the month, it was based on that patient's current position. And it doesn't mean that they couldn't walk out the next day and get hit by a bus or die in a commercial airline disaster or something like that. But there were standard deviations that would account for those kinds of risks. What the insurance company wanted to quantify or the annuity company or the reverse mortgage company wanted to quantify was based on this person's – everything that we know about this person right now, the function of their liver, the slightly hypoxic anemic profile that they have, the deficiencies that they have in vitamin D3, the suppressed immune system, the elevated visceral fat, the body mass index, and the bone mineral density, based on all of these things, when do these things all meet and actually cause a catastrophic event? And there were things called mortality debits and comorbidity debits, and we put all these together, and I really liked the data of it. But realizing that there were human beings on the other side, the spreadsheet just woke me up, and I said, you know, I have this chasm of knowledge, even though I'm not a physician, I have a fundamental understanding of human physiology. I'm fascinated by human physiology and ways that we can improve lifespan and healthspan. And so why wouldn't I take this gift and get into an industry like wellness or functional medicine where I could actually make a difference? Where instead of predicting death, we could extend life. And in doing so, help people have healthier, happier, longer lives, more fulfilling lives, get rid of a lot of the ailments that people suffer from. I mean, when you start to peel back the way that modern medicine defines a lot of conditions that we have, take depression or anxiety or ADD or ADHD, because a lot of your listeners are probably entrepreneurs. And a lot of entrepreneurs have very active minds and they've been told they have attention deficit disorder, right, or attention deficit hyperactivity disorder. Well, an attention deficit disorder is not an attention deficit at all. We've misdefined these things. It's an attention overload disorder, because you see in the human brain, we don't just create thought. We also dismantle thought. It is just as important for you to be able to create a new thought or feeling as it is to dismantle it, right? And if you don't degrade thought, called catecholamines, if you don't degrade them, then there's a gene that governs this, catechol-O-methyltransferase. If you create thought at a faster rate than you degrade thought, then the mind gets very clouded. And so attention deficit disorder is attention overload disorder. It's too many windows open at the same time. So modern medicine says, well, if the mind's racing, let's put an amphetamine into the body, race the central nervous system to match the pace of the mind. And this is a very poor choice, right? Because eventually this will burn you out, can actually change the neuroplasticity of the brain. Rather than put the right amino acids back into the body, the right B-complex blend, the right methylated folates, so that the mind can actually begin to quiet. What would you say to someone that says that ADHD is also in some ways a result of some early trauma? Well, you know, trauma is always fascinating to me. Trauma can trigger methylation. Trauma can interrupt the methylation cycle, right? But the idea in modern medicine that you have some kind of trauma, you have a disrupted relationship with your mother, for example, and that somehow we're going to go and put neuroplasticity altering chemicals into the brain and it's going to fix this 30-year broken relationship you have with your mother to me it doesn't yeah yeah make any sense right and so it's not at all to poo poo trauma to to put trauma down depression really exists anxiety really exists but but if you actually look at how we define these conditions take depression for example we define depression at least in America, we define depression as an inadequate supply of serotonin, right? So if you are low in serotonin, you're by definition depressed. So then you would think that the solution would be to raise serotonin, right? If we define depression as low serotonin, you'd think that the solution would be to raise serotonin, but that's not what we do. We take people that are depressed and we put them on SSRIs, selective serotonin reuptake inhibitors. And what these do is they ration what little serotonin these people have. So by definition, it never raises serotonin. So by definition, it never ends depression. I mean, I have clients come in to see me all the time and on our clinical team, and then I'll say, well, how long have you been on antidepressants? They say 15 years, 18 years. And my first question is, well, how long have you been on antidepressants? They say, 15 years, 18 years. My first question is, when did you think it was gonna kick in, right? So if we understand that serotonin is actually methylated in the gut, this process that I'm talking about is called methylation. We actually make serotonin from an amino acid called tryptophan, the one that's famous for making you sleepy after Thanksgiving dinner. I know you guys don't have Thanksgiving in the UK, but I'm from America. So because we eat a lot of turkey on Thanksgiving and turkey has a lot of tryptophan. So when you take tryptophan, methylate it into the neurotransmitter serotonin, of which 90% resides in your gut. Methylate is basically processing. Yeah, the processing. It's like the refining process. Crude oil gets refined into gasoline. Amino acids get methylated into neurotransmitters. And so this process of methylation, when it's broken, and it can be relatively easy to fix when it's broken, it means that we have an impaired ability to create, we have an impaired ability to refine a raw material and it leads to this deficit. Well, serotonin, for example, 90% of it resides in your gut. So if you don't have it here, you can't have it here. So depression rarely begins in the outside environment. It usually begins in the gut. Now, it may be trauma that led to the deficiency, but the fix is not in a chemical or synthetic or pharmaceutical blocking the brain's capacity to uptake these neurotransmitters. The fix is in restoring adequate levels to the body so it can naturopathically make its way back up the vagus nerve and arrive to the brain. Similar things are true with anxiety. I mean, if you actually have ever suffered from or know somebody who's suffered from anxiety, if you ask them three questions, you can find out very quickly that their anxiety is not coming from a cluster of symptoms. It's not coming from a cluster of symptoms. It's not coming from their outside environment. It is coming from within them. It's coming from their physiology, right? I mean, if you know someone who's suffered from anxiety and you say, well, have you had anxiety on and off throughout your lifetime? The most of the time they'll say yes. And then if you say, can you point to the specific trigger that causes it? Very often they'll say no. I mean, yes, I know some of my triggers, but I could be sitting in a podcast just like this in a very calm environment. There's no threats around. And all of a sudden I get overwhelmed by anxiety. I can be driving home from work on an otherwise innocuous day and I can be overwhelmed by anxiety. Well, that is not coming from your outside environment, right? This is coming from a process called methylation and it is caused from excess catecholamines entering the brain and an inability to down-regulate these. So the body's entering this mild fight or flight response without the presence of a fear. See, remember that as sophisticated as we like to think our brains are, it's really not. Our brain is very primal. You know what the brain cares about? The brain cares about survival. And so it doesn't care how fat or skinny you are, how pretty or ugly you are. It doesn't care about your skin, your hair. It cares about survival. And so when we understand that the brain does not know the difference between perception and reality, we start to understand how it can play tricks on us. So I always use the example that let's say you drove home tonight and you got out of your car. When you got home, you got out of your car and somebody was standing in front of you with a knife. It's a very real threat, right? You'd have a fight or flight response. Your pupils would dilate, your heart rate would increase, your extremities would flood with blood, your hearing would get very acute, your brain would flood with catecholamines, you are getting ready to fight or flight. But you could also be laying on the 30th floor of a condo building in bed and start thinking about getting eaten by a shark. There is zero chance of a shark getting out of the ocean, going up a 30th floor elevator, right? Coming into your condo and biting you in that bed. But you can have the exact same response. If you're watching a movie or something. Exactly. So one is entirely real. One is entirely perceived. The physiologic response is identical. So now once we understand this, now we begin to understand how I can feel the presence of a fear, which is what anxiety is. It's a fear of something happening in the future. Usually it's not going to happen. Usually it hasn't happened in the past and is not likely to happen. But this fear starts to build up. You start to get very anxious, it can actually change your heart rate, to the point where you can panic attacks can land you in a hospital. Or it can be mild enough that it just causes you anxiousness and mild anxiety, but there's no presence of a fear. And so you start trying to correlate it to your outside environment starts to drive you crazy because you go, well, I don't get it. I'm on vacation with my wife and my spouse and my kids. And I'm in a resort of a lifetime. I've been here a thousand times. I love this place. There's no reason I should feel like this. But all of a sudden you have this feeling of anxiousness, anxiety. So these are lack of raw material in the human body. My mission is to try to help people by taking a genetic test once in their lifetime, find out where is methylation broken, and then stop supplementing just for the sake of supplementing and start supplementing for this deficiency so your body can thrive. In the case of people that are listening to this now, and they can pinpoint the moments where they've got an anxiety. So say they've, I remember I had one guest on the podcast, maybe two years ago. And after he became famous, he developed social anxiety. So whenever he would be around a lot of people, he would feel that sense of anxiety. And then from that sort of catalystic moment, then when he's at home, he'd get the same rush of anxiety, but he would point to that catalystic moment of becoming famous. And then some things had happened in his life. And then he'd get anxiety at home when nothing was going on. Right. In that situation, what's the new- So there you go. So now you've interrupted methylation because there's one where there's the presence of fear and there's one where there isn't. There's the absence of a fear. So to be very specific, anxiety, true anxiety does exist, but you can point to the specific trigger that causes it. So for example, if you have a fear of heights and you walk to the edge of a 30th floor balcony and look over it, you're going to feel anxiety. If you're claustrophobic and you step on a really crowded elevator, you're going to feel anxiety. But if you're claustrophobic and you're sitting at home and you start to become overwhelmed with anxiety. This is actually not coming from that trigger. This is coming from your physiology. And the way that we deal with stress, right? And like cortisol, when you measure cortisol levels, cortisol is not really a measure of how much stress is in your life. It's a measure of your body's reaction to stress. So why are some people more resilient to stress and don't have anxiety attacks? And why are other people not as resistant? Again, this is not to say that if and don't have anxiety attacks and why are other people not as resistant again this is not to say that if you didn't have a violent attack in your life or or a terrible car accident that sometimes when you've had a you know a vehicle accident you approach an intersection the the the memory of having been t-boned you know recently and badly injured is going to give you anxiety but the majority of people are not suffering from that type of hyper specific situational anxiety suffering from something called generalized anxiety or idiopathic anxiety which means of unknown origin so for somebody who's very famous and gets into a crowd and doesn't know who's coming at them that's a very i wouldn't even define that as anxiety, that's a very primal instinctual reaction to a real fear. Just like walking to the edge of a 30th floor balcony. What's not a primal reaction to a real fear is when there is no presence of a fear, especially if that incident has never happened and you aren't even sure what you are afraid of or why you are anxious or why you have anxiety, then this is coming from your physiology. So how would you treat that? You'd look at the different, there are five major actionable genes that I like to look at in their, what's called their suballeles. And when you find out what they're deficient in, you start to supplement with things like SAMe, S-adenosylmethionine, methylated forms of vitamins, L-methionine, the proper balance of B-complex, methylated forms of folic acid or folate called methylfolate. And what happens is now the body has the capacity to degrade these neurotransmitters that are causing this fight or flight. This group of neurotransmitters called catecholamines and the anxiousness that follows. And you'll find that the majority of people that suffer from idiopathic anxiety or generalized anxiety because of low serotonin, they also have gut issues. You show me a person that's truly depressed and I'll show you somebody that's also suffering from severe gut issues, either gas or bloating or diarrhea, constipation, irritability, cramping, because the same neurotransmitters that affect these emotional states also are responsible for the motility of the gut, the speed of the gut. This is the most overlooked thing in all of bariatric medicine, because people that believe that they have all of these allergies, well, I'm allergic to wheat, soy, corn, dairy, blueberries, bananas, gluten. Yes, sometimes those individual allergies do exist, but the majority of time, even if you talk to somebody who says, yeah, I get bloated or I deal with gas or cramping or diarrhea or constipation or irritability, I deal with all of these gut issues, irritable bowel syndrome, Crohn's disease, ulcerative colitis, all these names that we give to conditions of the gut. When you ask them, well, what are you allergic to? And they give you this laundry list of things. And then you ask them another question and say, well, if you're really allergic to corn, is there ever a time that you can eat corn and not have a reaction? The majority of time people will say yes. Okay, well, right there, you know, you don't have an allergy. Allergies are not transient. Allergies are consistent, right? You don't wake up Monday morning and being allergic to milk, and then you're unallergic on Wednesday afternoon and then re-allergic on Saturday morning. But what happens when people have gut issues that they can't explain is they always correlate it to what they last ate. And it's hard to make this connection. They're like, well, wait a second. I ate the same thing Monday and I was fine. And I ate the identical food on Wednesday and I blew up like a tick. So this is not an allergy. This has to do with the motility of the gut. So if you don't know what gene mutation you have that is causing a deficiency, then you don't know what to supplement with to restore gut motility. But once you do, the gut goes back to its to supplement with to restore gut motility. But once you do The gut goes back to its normal pace. What's gut motility? It's the pace of the gut So if you remember henry ford was actually not made famous for the automobile He was made famous for something called the assembly line So the assembly line was just a glorified conveyor belt, right? And when you walked into his factory, they put a part on it on one end, and about every six feet, somebody stood and tinkered with that part. So it went to me, I tinkered with it, it moved to the guy to my right, he did something to it, moved to the guy to his right. And by the time it reached the end of that conveyor belt, it's fully assembled. This is very analogous to how the human intestinal tract works. It's 30 feet long. It's a giant conveyor belt. You put parts on it at one end as they exit the stomach in a very acidic environment, and it moves slowly towards the rectum. And before it exits the rectum, it's in a relatively alkaline environment. So instead of having people standing along a conveyor belt, you have bacteria that are graded by pH. The sequence is very important. So imagine what would happen if Henry Ford walked into his factory one day and doubled the speed of the conveyor belt. The entire assembly line would break down, not because there's anything wrong with the parts, the contents, not because there's anything wrong with the people that are working there, the bacteria, but because you changed the speed. What if he went in there one day and reversed the conveyor belt? What if he just ran it in the opposite direction? It would screw the whole thing up, right? So by changing the pace of the gut, the speed of the gut, the conveyor belt, I've ruined this sequence of events. And I spend a lifetime trying to figure out what's wrong with the parts, what's wrong with the workers, what's wrong with the conveyor belt itself. Nothing. It's how quickly or slowly it's running because the motility, this peristaltic activity is affected. that all of a sudden these strange allergies eviscerate and all of this gas and bloating and diarrhea and constipation and irritability and all of this inability to equate things that I'm eating back to what is going on in my gut seem to go away. It's true with all kinds of conditions. You know, we have subscribed in this world to the fact that we are so affected by disease and pathology. And once I get you to subscribe to the fact that you have a disease and you can get you to subscribe to a lifetime of medication you know this is true in hypertension you know it's another huge you know we think genetically inherited disease and a lot of families have you know grandfather had high blood pressure um dad had high blood pressure now the son has high blood pressure so they go up it's genetic genetically inherited high blood pressure okay well the son has high blood pressure. So they go up, it's genetic, genetically inherited high blood pressure. Okay. Well, what gene is being passed down from these generations to cause this? Well, they don't know. Okay. Well, we know all of the genes. So if we don't know what gene it is, then let's look at the methylation genes. Let's look at the genes in the human body that refine raw materials. Because if you have a certain deficiency, for example, in hypertension, very often, not always, but very often it's caused by elevated levels of an amino acid called homocysteine. You have it in your bloodstream right now. I have it in my bloodstream. Every listener to this podcast has homocysteine in their blood. Because in the process of normally refining this amino acid into something called methionine, we keep this level at a reasonable level. But when you can't break homocysteine down, it rises. When homocysteine rises, it's very irritating to what's called the endothelial lining of the blood vessel. It irritates the lining of the blood vessel. Well, when you irritate a blood vessel, it clamps down. Well, if you make the pipes smaller in a fixed system, pressure goes up. You have 63,000 miles of blood vessel in your body. It doesn't take much narrowing to drive pressure up. And so these people end up with hypertension, high blood pressure, and they go to the doctor and the doctor does an EKG. It's normal. An EEG, it's normal. Stress EKG, it's normal. Cardiac cath, normal. Heart and lung sounds, normal. Die contrast study, normal. The entire cardiac workup's normal. And then they still start pounding on the heart with medication because they can't figure out why the blood pressure's up. But they never actually looked at what raw material was missing in their body, not allowing them to bring the homocysteine down and allowing the vascular system to relax. As the vascular system relaxes, pressure returns to normal. You spent 22 years at that insurance company looking at the database that you described. And as you're saying that, much of the sort of medical profession, and I think most of society- Yeah, and I don't want to attack the medical profession. I really, you know, I'm in no place- I am a huge lover and believer in modern medicine. It's saved people very close to me. I mean, and trust me, if I hit a windshield at 20 miles an hour, I want a surgeon, I want painkillers, I'm going to the ER. But there's been this kind of- it's not a preventative approach to disease that we've taken over the last couple of decades. It really is, you know, to put a band-aid on something that's that's emerged and as you said at the start of this conversation these predicaments these um diseases emerged decades before we even see the symptoms oftentimes um at the end of your 22 years at the insurance company you started to look at these things which you call modifiable risk factors. Yes. What is a modifiable risk factor? And what are the most common modifiable risk factors in your view? Well, I mean, one of the most common ones that I saw was modifiable risk factor is, it's a risk factor that you have. And if you changed it, it would have modified it, it would have a demonstrative impact on the trajectory of your life. So for example, anemia, right? Low hemoglobin, low red blood cell count, low oxygen transport in the blood will exacerbate just about any condition that you have, right? So if you are hypertensive and anemic, if you are diabetic and anemic, if you are morbidly obese and anemic, meaning you have low blood oxygen because you have low red blood cells, low hemoglobin. What if you could modify the anemia? Well, if you could modify the anemia, carry more oxygen in the blood, you'd be much more resilient to all of these conditions, right? So the same thing is true with dementia and Alzheimer's and cognitive function. As you impair cognitive function, you increase the incidence of all-cause mortality. So a modifiable risk factor would be something like looking at the levels of vitamin D3 in the blood, in the body. So it's estimated that roughly 50% of the world's population is clinically deficient in vitamin D3. Why is that important? Well, vitamin D3 is the only vitamin that a human being can make on our own. There's hundreds of vitamins in your bloodstream right now. You're only capable of making one, and it's vitamin D3, cholecalciferol. We make it from sunlight and cholesterol. You don't even need to eat to make this vitamin. There is not a single cell in the entire human body that does not have a receptor site for this vitamin it also acts like a hormone it's a calcium transport molecule it's enormously impactful in your immune system clinical deficiency in vitamin d3 was at one time the second leading cause of morbidity in covet you know when they said covet disproportionately affected minorities if you ever heard that, it's true. So how did COVID disproportionately affect minorities? It wasn't like the virus didn't start going after certain minority populations. Well, it actually did, but it wasn't because they were not minorities that had to do with the pigment of their skin. The darker the pigmentation of your skin, the lower the vitamin D3. The lower the vitamin D3, the more compromised the immune system. The more compromised the immune system. The more compromised the immune system, the more susceptible you are to attack. That suggests that if you have darker skin like I do, that you should spend more time in the sunlight. Yes, it does. It absolutely does. I mean, you and I, just based on our skin tone, if you and I in a pair of board shorts went out and spent 30 minutes in sunlight, I would manufacture about 25,000 IUs, international units of vitamin D3, you would manufacture about 3,500. Fuck, you know, that's not fair. It's not fair. We can run fast. But you also look better older and you don't age, you don't get wrinkles as much, and you have better oily skin. Great basketball. There's some trade offs. So when you look at African American populations, Latino populations, Middle Eastern populations, the percentage of vitamin D3 deficiency approaches 85%. The truth is most of us are not getting enough sun. It's not that we're getting too much. So let's talk about the depletion of this one nutrient. Can I just ask then before we move on to the depletion of that nutrient, because I want to understand why that is. My guess was that because people with darker skins have grown up in sunlight, we spent more time in the sunlight, I'm guessing, which meant that we adjusted our production of vitamin D3 to suit the environment we lived in. Yeah, you actually have a better barrier, right? You are less susceptible to skin damage from sunlight than someone with lighter skin, right? And so if you actually look, because I used to be a population mortality expert, you know, the longest life expectancies were centered right around the equator of the earth. If you look at most of the blue zones, they're gonna be sort of closer to the equator of the earth. They're gonna be closer to places where the sun shines longer periods of time throughout the year. The blue zones? The blue zones are, are sorry the blue zones are areas of the world where people live um extraordinarily long life expectancies right mediterranean diets so the longest life expectancies are centered around the equator so for every 20 degrees latitude, because this is longitude, every 20 degrees latitude, there's a precipitous drop in life expectancy until you get to the poles, where when I started in this industry, the lowest, the shortest life expectancy on earth existed. So when I was born in 1970, a true Eskimo had a 56 year life expectancy. 56 years. An Eskimo had 56. True Eskimo. So these are at the polls. Now why? Because they would go months without seeing the sun. And when they did see the sun, they were layered up. So they had brittle bone disease, they had autoimmune disease, they had immunocompromised conditions, all kinds of conditions that go with severe depletion in vitamin D3. And so I would see this in the medical record all the time, right? I would see medical record after medical record. You know, a patient has been, had a decade or longer of clinical deficiency in vitamin D3. Well, when you deplete this nutrient and you don't supplement for it, by the way, vitamin D3 is so easy to supplement for. Everybody should be taking vitamin D3 unless you're getting adequate sunlight. 5,000 IUs of D3 with 80 micrograms of K2. And you deplete this nutrient. Eventually, you will develop rheumatoid arthritis-like symptoms. Now, you don't have rheumatoid arthritis, but you have the same identical symptoms when you're depleted in vitamin D3. You start waking up sore and achy like you had a workout the night before when you haven't. The soles of your feet and your ankles are tender when you get out of bed in the morning to walk to the bathroom and take your first pee. Your knees and hips and ankles start to stiffen up. Then it goes to cross the shoulders and eventually it's hard to make a really tight fist. Well, if you go to the wrong physician, family medicine practitioner that's seeing a lot of high volume of patients, you start to give them that description. They go, you know what? You've got rheumatoid arthritis. I'm going to hit you with something called corticosteroids. I'm going to put you on some prednisone first, and then we're going to transition you to a corticosteroid like a methotrexate or one of these, and you're going to be fine. It's going to push this off into the future. Well, we knew that if you started corticosteroids, that at first they had an anti-inflammatory effect, but you had roughly six years and one day until you were having a joint replacement. Because first they're anti-inflammatory, but then they eat the joint like a termite. And so it was so accurate that if you were a 60-year-old female and you were misdiagnosed with rheumatoid arthritis because you had a vitamin D3 deficiency, I would artificially advance your age six years. I would schedule the joint replacement. That wasn't required, by the way. I would schedule the joint replacement. And then I would begin to reduce what's called your ambulatory profile, how well you ambulate, how well you move. And we know now that sitting is the new smoking, right? Sedentary lifestyle is the leading cause of all-cause mortality. And so as I start to reduce your ambulatory profile, I bring in all of the diseases that exacerbate with reduced motility. So if I back this up, you were diagnosed with a condition that you didn't have because you had a deficiency in a very simple nutrient. So then you were put on a medication that wasn't required. This caused you to have surgery that you didn't need to replace a joint that wasn't initially going This caused you to have surgery that you didn't need to replace a joint that wasn't initially going to be defective. This reduced your mobility and it brought diseases forward that you never would have caught. And you died early from a condition you never would have had because you had a deficiency in a nutrient that was misdiagnosed, mistreated, and led to your early demise. And I can give you hundreds of examples like this and this is why if you look at the 2016 harvard study the 2019 johns hopkins study medical error is the third leading cause of death medical error medical error is that misdiagnosis misdiagnosis and medical error the average american over 50 52 is on five prescription medications. These are synthetic chemical pharmaceuticals, right? And in a lot of cases, we don't know the long-term side effects of these. In a lot of these cases, we actually don't even know the mechanism by which they work. If you actually open the PDR that comes, the disclaimer that comes with a lot of these medications, you'll see that they say mechanism of action unknown. Well, it's a sleep medication. How does it make me sleep? I don't know. Just makes you sleep, right? In fact, most people that are suffering from sleep issues are suffering some sleep issues, not because they can't sleep, not because they're not tired, because their mind is keeping them awake, right? And if you have a gene mutation called COMT, catechol-O- omethyltransferase, then what happens to you is you lay down to go to sleep and there's nothing more frustrating than one spouse having this gene mutation and one not. Right? Oh my God, I think you're talking about me and my partner. I'm the one that, I need some like sound or something to go to sleep because she's just out like a light. There you go. See, and nothing's more frustrating because- So annoying, I sometimes sleep in another room because- Okay, so the reason why you're not falling asleep is because when your environment quiets, your mind wakes up. And if you actually hone in on the kind of things you're thinking about, you're thinking about the most innocuous little nonsense. Right? I mean, it's like, did I get everything on my grocery list? Did my belt match my shoes today? Did I return that email? It's nothing that couldn't wait till the next day 100 yeah and so and sometimes you can even catch yourself going what am i doing thinking about the color of dishes i'm gonna have if i threw a party which i'm probably not gonna throw it's just like how did i get down this rabbit hole right and um this is because you know the you are not quieting the mind at the pace that you are exciting the mind. So in other words, when you lay down to go to sleep and your environment quiets, your mind starts to wake up. So you need to be distracted so that your mind doesn't continue to run. But if you just put the right amino acid balance, in fact, I would bet my career you have this gene mutation. I will pay for your test if you don't. It's $599. It's a cheek swab you do once in your lifetime and it will tell you if you have this gene mutation, COMT. And if you do, I'll tell you exactly what supplement to take and it will become a permanent thing of your past. What supplement can I take? Don't end your sentence there, Gary. So it's called L-methionine. Oh, I've heard of L. And you take L-methionine, you take methylfolate, and depending on where the gene break is and how severe it is, you may add something called SAMe, S-adenosylmethionine. And these sound like fancy names. They're just fancy names for vitamins and amino acids. All of these are in your bloodstream right now. So when you look at the sequence of breaks that somebody has, then you can tell them exactly what supplement to take. When they take the supplement, their body's no longer deficient, right? Deficiencies over time express themselves as all kinds of things. You know, there's a lot of people that can't reach elevated emotional states for any prolonged period of time. This is because mood and emotional states are recipes, just like a chef bakes a recipe. But if you went to a bakery chef and you said, you can bake whatever you want, you just can't use butter. Okay, it doesn't sound like a big deal. It's just one ingredient. But think of the number of cookies, pastries, pies, brownies that it would affect. It would affect so many different recipes just by removing one ingredient. This is the same thing that happens in human beings. If you couldn't use serotonin to assemble mood, okay, now any emotional state that requires that neurotransmitter as a part of its structure, you can't assemble. And as it relates to that mutation, when people hear the term mutation, they think of it as being something that's happened in me. And I'm maybe, you know, one of a few that have that mutation. Mutation sounds like a... Yeah, it does sound, it does sound like an alien's about to start like growing out of your stomach. Yeah, but it also sounds like, you know, my sister won't have it and it's just me. Okay. So a gene mutation means that, you know, we get a copy of our genes from each parent. Yeah. Right. And then these, the copy of our genes from each parent, right? And then the copy of these genes is passed down to us. And we know, you know, most of us are familiar with basic genetics, eye color, skin color, detached ear lobes. But there's nothing you can do with that information. So beyond your ancestry, you know, whether you've got Irish heritage or Native American Indian or what have you, I mean, those are, if you did a 23andMe, you'd be able to find your ancestry, whether you've got Irish heritage or Native American Indian or what have you. I mean, those are, if you did a 23andMe, you'd be able to find your ancestry, which is also your genetic history. That's not actionable information. When you look at the genes of methylation, right, and there are several of them. I look at five majors and a few minors. When you look at the genes of methylation, these are the genes that code for how materials are taken into the body are refined into the usable form. And these are not mutations that you're going to suffer from. But what they do is they cause deficiencies to arise in the body. And it's this deficiency that leads to the expression of these conditions. And so when you can't adequately methylate neurotransmitters, you have a deficiency in a certain neurotransmitter, any mood, any emotional state that requires that neurotransmitter, you can't assemble. If you have a deficiency in the ability to quiet the mind, then you have excess neurotransmitters in the brain, which cause it to stay in an awakened state. So that gene that is responsible for my loud mind when I go to sleep, do you have any idea the amount of people that have that particular gene? Well, 44% of the world's population has a gene mutation called MTHFR, Well, 44% of the world's population has a gene mutation called MTHFR, methylene tetrahydrofolate reductase. This is a genetic mutation that impairs the ability to convert folate into methylfolate or folic acid into methylfolate. And that might not sound like a big deal, but folic acid is one of the most prevalent nutrients in the human diet. And the thing about folic acid is that folic acid is an entirely man-made chemical. Folic acid is not found anywhere on the surface of the earth. You can't find folic acid anywhere naturally in nature. And yet we give folic acid to pregnant women when they get pregnant because we tell pregnant women that folic acid prevents neural tube defects. Well, that's patently false. Folic acid doesn't prevent anything. Folic acid has to be converted first into tetrahydrofolate, then dihydrofolate, but eventually it becomes something called methylfolate, and that prevents a neural tube defect. Well, what if like 44% of the population and 44% of women, this woman has this gene mutation and you give her folic acid, the man-made version of folate, and she can't convert it into methylfolate. This is where postpartum depression develops. And I have not seen a single peer-reviewed published clinical study linking elevated levels of hormones in female pregnancy to postpartum depression. But if you give a woman that can't methylate folic acid 1400% of the daily allowance of folic acid, and she can't break it down, then she goes nuts. And then eventually the pregnancy ends, the depression goes away, and so she blames it on the pregnancy, not on the vitamin. And we do this over and over and over again. We have, in the modern industrial world, we try to synthesize what occurs naturally in nature in a laboratory. You know, there's three types of B12, right? Adenosylcobalamin, hydroxycobalamin, methylcobalamin. These forms of B12 occur naturally in nature. There's one that we make synthetically in a laboratory. It's called cyanocobalamin. We make it from hydrogen cyanide. It is useless in the human body. The human body actually has to take it and convert it into the active form of B12 called hydroxycobalamin. And so a lot of times the supplements that we're taking us are causing more harm than good. And we're taking them because a doctor said, well, I'm pregnant. My OBGYN told me to take this synthetic chemical called folic acid, which it's always beyond me why anyone would think that something that we make in a laboratory that's entirely synthetic, that isn't natural, that isn't found anywhere on the surface of the earth could ever be required for optimal health. I mean, just on its surface, that sounds strange to me. But yet we do, and we don't understand this process of methylation. So once you understand where your genetic mutations are where the genetic inability to refine raw materials is you can go about supplementing targeted supplementing for that deficiency so you would recommend people take a genetic test to find out these answers right i'd recommend everyone take that test once in their lifetime you know i i'm i'm in the camp that we're, as humans, we're not broken. I totally agree with you. Do you know what I mean? And so whenever there's something going on in my body or, you know, a way that I am or a habit I have that I'm like, why do I do that? I'm always trying to figure out the sort of, I guess, the ancestral reason why that might have helped me to survive or what's going on. So when I hear mutation, it sounds a little bit like broken to me. Yeah, Yeah. So I shouldn't use that. I mean, we use the term mutation, but basically, you know, in our ancestry, right, we pass on our entire genetic code, which a lot of that is hair color, eye color, skin color, you know, our heritage, but we also pass on how our body refines certain raw materials. And it's not to say that it's good or bad or broken or fixed, but there are certain families and human beings that have an inability to refine certain raw materials. And if we could be aware of this, it is astounding what happens to human beings when you just give their body the raw material to do its job. You know, lots of people that have attention deficit disorder have trouble sleeping or also have trouble with prioritization. They just don't know it. They'll say things like, I work really well under pressure, right? Most entrepreneurs say, I work really well under pressure. Do you say that? Oh my God. Okay. I only work when there is pressure. I was that kid in school that would only do the test when there was an hour left to go or the homework or whatever. Yeah. So many entrepreneurs are that way. And when you say you work really well under pressure, physiologically what you're saying is I lack the ability to set priorities internally. So I use external pressure to set my priorities. And why is that? It's because when you don't have the right methylated nutrients in the body, you lend equal weight to every thought that comes into your mind. Right? So you could be working on, you know, the deal, the joint venture business closing of a lifetime, right? And you got the contract, and you have 45 minutes to get it back to your lawyer, you know, the deals permanently going to go away. This is a life-changing deal. You've waited for this your entire life. It's a very meaningful event. You're working your way through this legal document and you look over at your phone and there's an Instagram message. And you're like, oh, that's my neighbor's cousin's kid fishing in a lake. I wonder if he catches a fish. So you start looking at it and you're like so you still cast and oh he didn't catch it that time now meanwhile this has gone from 45 minutes to 35 minutes and now it's down to 25 minutes and all of a sudden you go oh you know crap you turn the phone off which didn't mean anything anyway and you focus all of your attention and because you're very intelligent and you and you execute and you can hyper focus you hyper focus on this and you worm your way through it and one minute before the deadline you hit the send button lawyer gets it the deal goes through and you go i work really well under pressure what's that got that process you described there is many people describe that as procrastination it's procrastination but it's procrastination because you give equal weight to all of these different variants that come into your field. So you actually give equal priority to very disequal events. But also, you know, sometimes if I'm looking at that contract, the contract clause is a little bit difficult. And my attempt to maybe escape discomfort would be, I'll just go do the dishes. And so I had Nir Eyal on the podcast. He wrote a book called Indistractable. And he says we're creatures that, we're not pleasure-seeking creatures. We're creatures that are avoiding discomfort, essentially. I think that aging is the aggressive pursuit of comfort. Could not agree with you more. Let's use Dana White as an example. Okay. Because, you know, the reason why I came across your work is because of a clip that, you know, involved Dana White. And I'm sure you, I know you get this a lot because I've seen you be asked about this in interviews, but for context for anybody listening, I don't know where I was or, ah, now I know what happened. My friend sent into a WhatsApp group a clip of Dana White talking about you. And that's the clip that made me go down the rabbit hole. I watched that clip. I then did some research. I then watched a series of videos of you online talking about health. And I watched you asking audience members to stand up and name the health issue they were suffering. And you on the spot told them what was missing from their life, their diet, whatever it might have been. You kind of diagnosed them in a way of, and then I reached out to you on Instagram and that's why you're here. But if we go back to the start of that, it was that story that Dana White told that had me so compelled to reach out to you. For anybody that doesn't know, and there will be some people that don't know, Dana White is the president of the UFC, which is the big fighting tournament where everybody kicks each other's heads in. So what's your take on the Dana White story? So Dana White is an example. I mean, he's just a celebrity example, but he is a shining example of the vast number of people, men and women in his age category that have given up on the capacity to thrive. They've accepted that they have hypothyroid, hypertension. They wake up sore and achy in the mornings that they don't have a response to exercise. They have a little bit of spare tire, their brain foggy. They're on three or four medications. In his case, he was on seven medications at the time, three of which were for blood pressure. He was on, you know, I think a thyroid medication was also on, he's been very public about this by the way. And again, I have to say, I'm not licensed to practice medicine. So it was my clinical team that came up with the diagnosis and I communicated it to Dana. I do train physicians to read blood work and genetic testing, but I can't practice medicine. But the point is that when I met Dana, all he wanted to do was for me to predict his life expectancy. And I hadn't done that in almost seven years. I left that industry for a reason. I don't do it anymore. The test that I do does not predict life expectancy. The genetic test and the blood work that I do will not tell you how long you're going to live. I have no interest in predicting death anymore. I only have an interest in extending life. And when Dana was only interested in me predicting his death, so I said, okay, for Dana White, I'll come out, I'll meet with you. I'll do a blood test on you, a gene test on you. I'll pull all your medical records and I'll give you your life expectancy. But what I did was went out and got his blood work and his gene test. And I was actually in bed at 1.30 in the morning when the lab was running his blood work. And I've had seven life threatening alert calls in the middle of the night, because when you drop blood work off at the lab, the lab runs it through the night. If they find a life-threatening alert, they call the account holder, right? So I owned a company, I was on the account. So LabCorp calls us at one o'clock, 1.30 in the morning, says, hey, we have a life-threatening alert on a patient. I was like, Whoa, what's the patient's name? They said, last name is white. I said, Dana White. They said, Yeah, well, what's the life threatening alert? They said triglycerides are almost 800. Now triglycerides a measure of blood fat, okay, it shouldn't be above 149. At 200 or 300. This is a cataclysmic level in the blood, especially in a fasted state. Okay, we pulled his blood in a fasted state. They weren't 400. They weren't 500. They weren't 600. They weren't 700. They were like 768. So they were, I mean, this is an enormous number. And so I said, okay, I need to get the blood work over to the doctor. And when they sent the blood work into the portal, I then saw that he was insulin resistant. He was hyperinsulinemic. He was pre-diabetic. He had skyrocketing levels of cholesterol. He was hypertriglyceridemic. He was hyperhomocystinemic. This homocysteine that I told you elevates and causes the blood vessels to constrict. I mean, he had all of these conditions. I literally at that moment booked a flight for 7.30 or 8.30 in the morning to head out and see him. And because I said, I need to go see him in person. And I remember, I think his assistant called me and I was at the airport and she said, "'Hey, Dana wants to know if his life expectancy's in." I go, well, I'm on my way to see him. And she goes, oh God, is it like that? I said, yeah, it's like that. And so I flew out to see him and I sat down with Dana. And when we talked about the blood work, I didn't even explain the levels. I explained the symptom. I did not know that he was on a CPAP machine, but I said, I am surprised that you can actually sleep through the night because he was so hypoxic. Red blood cell count, hemoglobin levels. I'm surprised that you can even sleep through the night without like just waking up, choking, gagging. He's like, dude, I'm on a CPAP machine. I wake up every night i throw up in the middle of the night i throw up so much i'm losing my voice um and i said this level of caudication triglycerides in the bloodstream i'm surprised you can even bend down and tie your shoes that's not painful to tie your shoes not that it's not restrictive to tie your shoes it's not painful like it doesn't feel like the skin's going to peel off your legs and he went what the i mean he slammed his hand down he was like how did you you know how did you know that and i said dana your level of brain fog and fatigue right now has got to be at a crushing level of fatigue i don't know how the only thing getting you through the day is your own stubborn willpower and i'm surprised you can remember anything from one minute to the next and his whole staff was like dude he's so forgetful he passes out in meetings he's sleeping in the planes he he's gagging, snoring. These were not things I necessarily knew about him. So I began to describe all the outcomes of these kinds of conditions. And I said, look, if you don't do what we're going to ask you to do for the next 10 weeks, based on this blood work and the medical records that we pulled for the previous 10 years and the demographic data we pulled for 10 years, you have a life expectancy of 10.4 years. You know, for a 52-year-old man to realize that he's not going to make it out of his 60s, a big realization. And he flicked a switch, a level of discipline that, you know, I haven't seen in a patient in a long time. He goes, dude, I'll do whatever you tell me to do. So we wrote a prescription ketogenic diet. I I'm a fan of the keto diet. I don't think everybody needs to be on the keto diet, but, um, um, by prescription ketogenic diet, we wrote a keto diet right down to the grocery list, keto reset diet. And I said, if it's literally, if it's not on here, you can't eat it, Dana. This is your grocery list. You go to the store, you buy this. You send your chef to the store to buy this. You make this. If it's not this recipe, if it's not on here, you literally can't eat it. Your only leeway is water and the supplements. And we started a process of balancing hormones, controlling his glycemic index, of using amino acids to bring down his level of homocysteine to actually try to fix the insulin resistance to reduce his triglycerides and in 10 weeks he had such a material change in his blood work i forget how much weight he had lost i think he had lost almost 28 or 30 pounds at times he's over 40 pounds now. By the end of the fifth month, he was completely off of every prescription medication he was on. He's down 44 pounds. He lost the, you know, he's no longer using the CPAP machine. He no longer is pre-diabetic. He no longer has insulin resistance. He no longer has life-threatening levels of triglyceride. In fact, they're normal. His kidney function improved. His liver function improved. His immune system strengthened. He feels like a 35-year-old man again. His skin tone all improved. His blood pressure returned to normal. He's not on any blood pressure medication. So his blood pressure returned to normal. And he was like, dude, I had no idea I could feel this good. I feel freaking amazing. And his life expectancy? Almost tripled. Almost tripled? Almost tripled almost tripled almost tripled just under 30 years when i heard the story about dana white and i saw he had gone from respectfully being a man that had a little bit of weight to having this these six-pack abs on instagram of course the six-pack isn't the the outcome it's as you said, it's the stuff going on inside him. That's really the transformation. It left me with the question, like, okay, I heard the keto bit, but what can someone who's just heard that at home, where do they start with getting, extending their life by triple and getting the... So, you know, he also started something called the superhuman protocol. And superhuman protocol is using magnetism, oxygen, and light, right? So the only things that we really get from Mother Nature, the big benefit we get from Mother Nature is we get magnetism from the earth, we get oxygen from the air, we get light from the sun. The truth is most of us are not contacting the surface of the earth that much anymore. So he bought $150,000 worth of equipment, a PMF mat, an oxygen, what's called a Hypermax oxygen to do exercise with oxygen therapy, and a red light therapy bed. And I had him use that equipment every single day, seven days a week. But if your listeners want to do it for free, you can take off your shoes and contact the surface of the earth. And I'm talking about bare feet on soil, dirt, grass, sand, because earthing and grounding is a very real thing. We actually discharge into the earth. We actually, human beings build up a charge. Do you know that pH, the acid alkaline scale, pH stands for potential hydrogen. It's a charge. It's a complete fallacy that you can get alkaline by drinking alkaline water. That's the biggest marketing myth ever sold to the public. But you can get alkaline by contacting the surface of the earth. So if you don't have 150 grand, which I don't expect anybody listening to this podcast to spend 150 grand, but he did. I said, you need a PMF mat so you can be alkaline. You need to spend 10 minutes a day breathing 95% O2 under mild exercise, and you need to lay in a red light therapy bed. So in the absence of the superhuman protocol, you can become superhuman by contacting the earth and by learning to do breath work. Let's talk about breath work. I spend eight minutes every day doing a very specific series of breath work, and I'll teach it to you now. You said your wife has certified him? Yeah, my partner, she's a breath work practitioner. Oh, your partner. I've done breathwork with her. I've done breathwork with a few people, but no one's ever had the profound impact on me through breathwork that she has. I've never shouted her out before, so I probably should. Her Instagram is at M-E-L-O-A-I for anybody that's interested in breathwork. People do not realize the power of something that is so accessible, so free, and so easy to do, right? They want things to be more complicated, but it's not. And when I said the presence of oxygen is the absence of disease, it's absolutely true. Remember that every elevated emotional state that a human being can experience actually has in its molecular structure, oxygen is a component of that emotion. So if you look at the difference between passion, elation, joy, arousal, libido, and anger, for example, it's usually only one neurotransmitter and the presence of oxygen. The reason why no human being has ever woken up laughing is because you don't have the oxidative state to experience laughter right out of deep sleep. But can you wake up angry? Yes, because anger doesn't require oxygen. So every morning contact the surface of the earth and then spend eight minutes doing, I do a Wim Hof style of breath work. I give credit where credit's due. He's the father of breath work as far as I'm concerned. So I do three rounds of 30 deep breaths, like obnoxiously deep breaths. And I start by trying to take my belly button and pull my belly button out towards the wall imagine there's a string pulling your belly button towards the wall and then you you fill from the lobes of the lung to the apex of the lung and then you exhale and just relax God knows what they think we're doing out there. Right outside this podcast. They're like, a bunch of freaks. I knew it was a cult. I knew he was a cult leader. So you do three rounds of 30 breaths. On the 30th breath, you exhale and you hold. Allow the carbohydrate receptor to reset. When you don't feel you can hold anymore, you take a deep breath in. You hold again and you let it out slow. And you start again. I would suggest that you start with three rounds of five breaths. Then work to 10, 15, 20, 25, and 30. If you get lightheaded, this is a good sign that the oxygen tension is changing in your brain. If your fingers and toes get tingly, this is a good sign that the oxygen tension is changing in your brain if your fingers and toes get tingly this is a good sign that you're changing the oxygen tension if you feel some kind of heat temperature change in your neck these are all great signs you will get to the point where you can actually hold your breath for two or three minutes sometimes four minutes between rounds of breath work um and then the last thing is to expose yourself to natural sunlight. First thing in the morning, the first 45 minutes of the day, God gives us a very, very special type of light. It's called first light. There's no UVA. There's no UVB rays in this light. So that it's not the damaging rays from the sun. It still generates vitamin D3. It has a positive effect on cortisol, on vitamin D3. First light is the best way to reset your circadian rhythm. So by contacting the surface of the earth, doing breath work and getting first light, you can get to the same place that Dana White did with 150 grand in equipment. What about oxygen masks? Because I'll be honest, when I read about the Dana story, I went on Amazon soon after and I was like, I'm just going to buy an oxygen canister. Good idea, bad idea? So what you want to do is, you know, you get an oxygen concentrator, which takes 21% oxygen from, which is what the concentration at sea level, it turns it into 95% O2 and it fills this bag and it can refill this bag over and over and over again. Okay. I use one called the Hypermax. You can see it on my Instagram and you plug it in, you turn it on, it fills this bag. And then you go in, you put an oxygen mask on, and you exercise for 10 minutes, only 10 minutes. Cycle for three minutes, sprint for 30 seconds, cycle for three minutes, sprint for 30 seconds, cycle three minutes, sprint 30 seconds, and you're done. And what this does is it raises something called the partial pressure, the storage of oxygen in your blood. is it raises something called the partial pressure, the storage of oxygen in your blood. The only two-time Nobel laureate prize winner in medicine, Dr. Otto Warburg, won both of his Nobel prizes for his work in exercise with oxygen therapy. You want to be a superhuman, do mild exercise every day while breathing 95% O2. It's important that you're exercising. And then after that, you move into a red light therapy bed, photobiomodulation. So, you know, if you don't have access to a hypermax oxygen machine, just do the breath work. Get the breath in, you know, exchange the oxygen tension in the tissues and expose yourself to first light. What about cold water plunging? So I'm a huge fan of cold water plunging, but probably not for the reasons why you think. You know, I also sit on the board of the NFL Alumni Association Athletica as a health service director. There was a time when we used to think that putting athletes in cold water after exercise was good because of its anti-inflammatory effects. We know now that that's only about 15% of the benefit. The majority of the benefit comes from something called a cold shock protein. If you really want to be fascinated, Google cold shock proteins. These are reserve proteins that are reserve proteins that are in your liver they're dumped into the bloodstream in an effort to save your life when you put yourself in cold water they scour the body of free radical oxidation they increase the rate of protein synthesis muscle repair they are free you get them when you put yourself in cold water i don't know what the celsius conversion is but I use 50 degrees for three minutes minimum, six minutes maximum. Cold. Yes. It's actually not that cold. I mean, you know, I see people getting in 37, 38 degree water. There's no evidence that I've read that shows that colder is better. You get a peripheral vasoconstriction, so it forces all the oxygen into the core and up to the brain. And you get an activation of something called brown fat, right? Thermogenesis comes from brown fat. And for the women that are listening, for some reason, I seem to ensnare the women when I say this. Remember that the definition of a calorie is a measure of heat, right? I mean, the definition of a calorie is the amount of energy it takes to raise one cubic centimeter of water, one degree centigrade. So if a calorie is a measure of heat, then this means that when heat's leaving your body, calories are leaving your body. So if there is nothing, nothing, no amount of exercise hits cardio, no type of cardiovascular or weight training that comes anywhere close to immersing yourself in cold water in terms of what will strip fat off your body fast. If you want to strip fat off your body, get in cold water three to six minutes a day. That's fascinating. Because the oxygen rushes to my head, that's why it has a really profound impact on mood. That's why it has a very profound impact on mood. Because if you think about it, what's the reason why we need deep sleep? What happens in deep sleep that's so special? There's a secondary oxygen transfer. We transfer oxygen from the periphery, from the extremities to the brain. Remember, the brain's a non-metabolic organ so in other words it's unlike a muscle if i pick up a weight and start to work out my muscle my arm my body will send more blood more amino acids more oxygen to that muscle because it's working well if i'm sitting at my computer and i'm watching reruns of the simpsons or i'm sitting in my computer and i'm solving the most complex joint venture agreement, partnership agreement with all kinds of mathematical equations, my brain gets the same amount of nutrients, same amount of blood flow, same amount of oxygen. So it eats the same meal, whether or not it's in a dead sprint or whether or not it's just chilling on the couch, except in deep sleep and when you're in cold water because it's forcing the oxygen up to the brain. You said earlier about comfort. Yes. I was speaking to someone yesterday about this thing called, he referred to it as the comfort crisis and how as we've become more, I would say civilized, but I don't know if that's the right terminology, as we've become more advanced technologically as humans, we can make our lives increasingly more comfortable. Correct. Sounds like a good thing. Terrible. It accelerates aging in every form. I mean, aging is the aggressive pursuit of comfort. We have got to stop telling grandma not to go outside. It's too hot. Not to go outside. It's too cold. Just to lay down. Just to relax. To eat at the very first pang of hunger. This is collapsing all of our own natural defense mechanisms. You know, if we don't load our bones, they don't strengthen. If you don't tear a muscle, it doesn't grow. If you don't challenge the immune system, it weakens. And so stress is very often very good for the body. Thermal stress, weight-bearing exercise, breath work, these things put stressors into the body that are very, that have a very positive effect at strengthening you. We want to regulate everything now. We regulate our temperature. We go from a, you know, a temperature-controlled office to a temperature-controlled car to a temperature-controlled home. You know, we don't, we don't thermoregulate anymore. I mean, you know, usually when you, when I ask people to start taking cold showers, they take their first cold shower, they never do it again. Why? Because they don't want to be uncomfortable. And so when you learn to deal and become comfortable with being uncomfortable, this is like a metaphor for life. It's almost like yoga. If you've ever done really intense yoga and you're holding a yoga pose and you're trying to remain calm and focus on your breath while your body's in intense pain. Now you're not in any risk, but your ass feels like it's going to peel off your legs and your hamstrings are firing and you're sweating and you're shaking and you're doing this thing that's called the candlestick, but it's really painful. And if you can maintain calm and breathe through a situation like that, what happens four hours later when you get a nasty Instagram message? Nothing. It doesn't shift your mood. And if we don't learn to control our emotional state, we will never control our future. You know, MIT did an incredible clinical study that showed that the amygdala of the brain, which is where we experience emotion, is the sole gateway to an area of the brain called the hippocampus, which is where we experience emotion, is the sole gateway to an area of the brain called the hippocampus, which is where we hold our memories. So just imagine that the emotional center of the brain is the sole gateway to the memory of the brain. This is why if you've ever had an argument with your spouse, you can always recall with incredible accuracy every other time they've made you feel this way. You know, you did this on September 21st. You did this when we were on the boat with my boys. You did this at, you know, our Christmas holiday party four Christmases ago because that emotion is linked to that memory. So you can recall that memory very accurately. Well, our memory, our hippocampus is what projects into the prefrontal cortex. It determines our future. It's our conscience. So this means if emotion is the only gateway to memory and memory projects to our conscience, which is our future, this means that your current emotional state determines your future. That's a biophysiologic fact. So like, for example, if you had an argument with your spouse on the way to work and you get out of the car and you slam the door and you walk into the office, when you break the plane of the door of that office, the only memories you can recall about the office at that moment are negative. You're gonna walk through the door of the office, you can be like, they don't respect me around here. I'm gonna have a stern talk into management today. You know, my office better not, you know, nobody better be at my desk. And you know what, Mary better not run into me today because she doesn't respect me. You can just start going through all the negative things about the office. The office didn't do anything to you. How do I prevent that? You learn to control your emotion. How? Well, first you start by putting the right nutrients into the body that allows you to achieve elevated emotional states. And you learn to do things like when you feel like you are beginning to lose control of your emotional state, you actually break that cycle. I usually do it with breath work. And so, first it begins by having the right raw materials, but this is just taking you back to the cold punch. If you can start your day in an elevated emotional state, if anybody listening to this has ever really done a cold punch, tell me if you were ever in a bad mood getting out of a cold punch. Just try to be in a bad mood getting out of a cold punch. They say if you want to cure depression, push somebody in cold water. You know? And it's so true. You're in such an elevated emotional state. You're like, wow. Now you go cruising into the day and get a little negative, you know, Instagram message and your spouse calls you and tells you she forgot what you wanted to get at the grocery store and you get to work and you got a little problem at the office. These things roll off your back instead of shifting your state, which now shifts your memory, which now changes the trajectory of your prefrontal cortex, which affects your future. I do a lot of traveling, Gary. I travel all over the world all the time. Oh, it's one of my favorites. Yeah, you travel a lot? Tons. You've got a bit of a system for traveling because when I travel, I feel like shit. Oh, I feel amazing when I travel. And I post all about it on my Instagram. You know, all I do is teach on Instagram. But you know, I went Miami, Atlanta, New York, London, Stade, Switzerland, Dubai, Dubai, Miami, Miami, Vegas. I got up at four o'clock this morning to come here. I mean, I landed here at I think 7.15 this morning. And you're flying out of here. I'm flying out of here now. In five here. I'm flying out of here now. In five minutes. I'm flying out of here in five minutes and I'm going on a red eye back to Miami. And you feel good when you travel and I feel like shit. What do you think we're doing differently? If you were to guess. Okay, so there's three things that you can do when you travel. And first and foremost, and I don't know why anybody talks about this, is that everybody talks about waking with the sun, which I'm a big believer in, or forcing yourself to stay up to try to get onto a new time zone. When you change time zones, the single most important thing that you can do is preserve your sleeping window. Do not eat during your normal sleeping window. Let me tell you what I mean. Let's say that you're on the East Coast, you live in New York, and you go to bed at 10 and you get up at 6 a.m. Okay. You go to bed at 10 p.m., get up at 6 a.m. Most people go to bed at midnight, get up at 6 a.m. So let's say you go to bed at 10, you get up at 6 a.m. And now you go to London. Okay. So now London is, depending on the time of the year, six hours ahead. If you eat during 10 p.m. to 6 a.m. New York time, there is zero chance you will adjust to that time zone. We are more tied to our digestion in terms of our circadian rhythm than we are to the sleep-wake cycle of the sun. So in other words, if I fly to London and I start eating when it's 3.30 in the morning my time, my body body goes what the heck are you doing we're having steak and eggs and a champagne it's 3 30 in the morning right your circadian rhythm is screwed up so shift your sleeping window and preserve that sleeping window in your new time zone and do not eat during those times so in other words in london that would be um 6 a.m to noon so be 6 a.m. to noon. So between 6 a.m. and noon, I'm not going to eat. I'll have coffee or I'll have water or fluids, but I will not start to eat until noon. For how long do I preserve that window? Well, depending on how long you're going to be there, usually I'm only, if you're there a week or less, preserve your sleeping window the entire week that you're there. When you're there for more than 7 to 10 days, then you need to really adjust to that time zone. What happens physiologically if I don't? If you don't, you will irrevocably mess up your circadian rhythm. I mean, just imagine it. So let's say you lived in New York and you didn't travel. You go to bed at 10 and you wake up at 6 a.m. Try for three nights setting alarm for 3.30 in the morning, get up, need a big breakfast at 3.30 and try to go at 6 a.m. Try for three nights setting alarm for 3.30 in the morning, get up, eat a big breakfast at 3.30 and try to go back to bed. And watch what happens to your sleep cycle, right? You'll destroy it. So we don't, we preserve our sleeping window. The other thing is I fast on domestic flights. I don't care what time or where I'm going. If I fly anywhere in the continental United States, I fast on airplanes. So I allow myself, you know, water,. I hydrate and I have black coffee on international flights. I just came back from Dubai. It was 16 hours. I ate on that plane. And then for flights that are more than an hour, every hour on the hour, I get up out of my seat. I don't care how weird it looks. I go to the back of the plane. I do 25 air squats. I go either into the bathroom or in the back of the plane. I do 25 deep breaths. On a 10-hour flight, I'll do 250 deep breaths and 250 air squats over the course of a 10-hour flight. You feel amazing. And then I eat fats and proteins on flights. Carbohydrates at altitude are terrible for you, and it's usually where all the salt hides. Remember that there are essential fatty acids, meaning they're essential for life. There's two of them. If you don't get these fatty acids, you'll die. There's nine essential amino acids. They're proteins. They're essential for life. If you don't get these nine essential amino acids, you'll die. There is no such thing as an essential carbohydrate. Why are carbs so bad in the air? Carbs are bad in the air because as soon as you divert blood from your brain to digestion, let's not forget it's a 30-foot long tube. The higher the consumption of carbohydrate, the more blood floods to your gut. So you're going to feel like crying. So now you're seated and your gut is flooded with blood. It's all come from your brain. So now you're tired. You're not focused. You're exhausted. And you're seated. So this isn't a good place to be tired because you're not going to get good sleep. So I set a huge priority on energy. Energy is a huge priority to me. So when I look at food, I look at it two ways. It's going to serve me, it's going to steal from me. When I'm flying first class back to Miami tonight, it's a red eye. I'm going to actually use that time to sleep. I'm not going to waste it on eating because I know that, first of all, I'm past my feeding window now because I preserved my East Coast time feeding window. And this is what keeps the train running on time. What is energy? You said energy though, what is energy? Energy is oxygen in your blood. Everything that you perceive about energy is nothing more than oxygen in your blood. If you told me, Gary, I had a lot of energy today. Physiologically, what you're saying is, I had a lot of oxygen in my blood today. Oxygen equals energy. At the start of this conversation, I asked you a question. I said, why should people listen to your message? We're now at the end of the conversation. What have I missed that is pertinent, important to your message that we haven't discussed? You know, I feel like I could talk about this forever. Like to me, I feel like the podcast is just getting going. You know, I believe in human beings and the ability of the body to heal itself. I believe in the power of the mind and frequency and the power of this to heal this. I guess my message would be, and I don't think that you've missed anything. My message would be that optimal health is found in the basics, not in the complicated, fancy neurotropics or some rare root that's buried deep in the Amazon jungle. It's found in the basics. The further we get away from the basics, magnetism, oxygen, light, whole foods, the more unhealthy we become. And, you know, my message, what I try to teach on Instagram and what the message that I get to the I get to the world is not that you need a lot of fancy equipment. Sure, if you can afford it, you can have it, but if you get back to the basics with Mother Nature and back to the basics with our foods, a good rule of thumb is if your great-grandmother wouldn't recognize it, don't eat it. Then you'll find a state of optimal health that's beyond anything that you thought imaginable. There's a superhuman inside of everybody listening to this podcast. Gary, we have a closing tradition on this podcast where the last guest asks a question for the next guest, not knowing who they're asking it for. The question that's been left for you is, what is the unobvious thing that you struggle with? What is the unobvious thing that I struggle with? What is the unobvious thing that I struggle with? You know, I believe that I'm so committed to my craft and I'm so committed to being authentic that I struggle sometimes like everyone else when I have an extra burden of guilt whenever I want to just have any kind of marginal enjoyment, right? Like if I just want to have a cocktail or I want to eat some birthday cake, which I know is not going to hurt me and it's not going to throw me off, but I really struggle with that. I feel like I'm letting the whole world down when i do that um and i know that it's not and it's ridiculous and you shouldn't be that you know being that discipline actually is a you know it's not sustainable over over a long period of time um i think that like a lot of people listening to this podcast i listening to this podcast, I'm very hard on myself. I'm my own worst enemy sometimes. And I have an insatiable appetite to do what I'm doing. And I think that if I just wasn't so hard on myself, I probably would find it a lot easier. Thank you so much. Everybody listening to this should go check out 10x Health. Much of the information you've discussed here lives within that ecosystem and your Instagram and your website and those channels, which I've explored, you know, in depth are incredible resources to understand how to start your journey to living a more healthy life. And that's, that's everything that you espouse. You're incredible, Gary. Thank you so much. And you know, it's very, very rare that I find someone online. then dm them and I nagged you on dm for a couple of months to get you here but but I think everybody listening to this can understand why um you're doing incredible work incredibly important work an incredibly important time to shift a narrative and um as I said to you um I think before we start recording or in an interval I realized that this is literally just the start for you in the journey and the mission that you're on. That's so clear to me. Thank you. I feel the same way. I've now been a Huel drinker for about four years, roughly, so much so that I ended up investing in the company. And I play a role on the board of the company, but they also very kindly sponsor this podcast. And to be honest, I've never said this before, but Huel believed in this podcast before anybody else. The CEO, Julian, told me before we even launched the podcast how successful it would be and that Huel would back it. And I absolutely have a huge amount of gratitude for them for that support, but an even greater sense of gratitude for the fact that they've helped me stay nutritionally complete throughout the chaos and hecticness of my tremendously busy business schedule. So if you haven't tried out Huel, which I hope most of you have at least given it a go by now, try it out. It's an unbelievable way to try and stay nutritionally on course if you have a hectic, busy schedule. And let me know what you think. Send me a tweet and a DM, tag me. Let me know what you think. Thank you."

In [ ]:
whisper_vL3_3 = " If you want to strip fat off your body, there is nothing, no type of cardiovascular or weight training that comes anywhere close to... Gary Pratchett! He spent 20 years working in life insurance, predicting when people were going to die to the nearest month. And now he's on a mission to extend your life. Most people, they're walking around right now at about 55% of their true state of normal. There is an element missing from their body that would make the difference between them being an average person and being a superhuman. How? Everything that we put into our bodies gets converted into the usable form. If you cannot make this conversion, you have a deficiency. And it is this deficiency that leads to these conditions. They have accepted something as either a consequence of aging, stress, or their environment. This is not a consequence of aging, stress, or their environment. This is not a consequence of any of those things. I'm going to tell you exactly how to find out what it is that's missing so you could thrive in a way that you probably never thought possible. Let's use data-wise as an example. So LabCorp calls us and says, hey, we have a life-threatening alert on a patient. He had all of these conditions. I've been talking to doctors. None of them could fix any of my problems. He said, I'm surprised you can even sleep through the night without choking, gagging. He's like, slammed his hand down. How did you know that? And I said, if you don't do what we're going to ask you to do, you have a life expectancy of 10.4 years. And in 10 weeks, he had such a material change. And he was like, I had no idea I could feel this good. And his life expectancy? Almost tripled. Someone who's just heard that at home, where do they start? So. I just want to start this episode with a message of thanks. A thank you to everybody that tunes in to listen to this podcast. By doing so, you've enabled me to live out my dream, but also for many members of our team to live out their dreams too. It's one of the greatest privileges I could never have dreamed of or imagined in my life to get to do this, to get to learn from these people, to get to have these conversations, to get to interrogate them from a very selfish perspective, trying to solve problems I have in my life. So I feel like I owe you a huge thank you for being here and for listening to these episodes and for making this platform what it is. Can I ask you a favor? I can't tell you how much you can change the course of this podcast, the course of the guests we're able to invite to the show and to the course of everything that we do here just by doing one simple thing and that simple thing is hitting that subscribe button. Helps this channel more than I could ever explain. The guests on this platform are incredible because so many of you have hit that button and I know when we think about what we want to do together over the next year on this show, a lot of it is going to be fueled by the amount of you that are subscribed and that tune into this show every week. So thank you. Let's keep doing this. And I can't wait to see what this year brings for this show, for us as a community and for this platform. Gary. Yes. I, this is the first time I've started this podcast in this particular way, but I was, but this is where I wanted to start it. It's with a slightly curious question, which is there's people that have just clicked onto this podcast to listen now, right? YouTube, Spotify, Apple, wherever they're listening. Why should they stay and listen to the message that you have committed your life to spreading? Because everybody wants to be better, right? I mean, everybody wants to improve their health. And most people, especially young people, probably don't realize that they're walking around right now at about 55%, maybe 60% of their true state of normal. There is a nutrient, an amino acid, a substrate, a compound, an element missing from their body that if they knew what it was, would make the difference between them being an average person and being a superhuman. They have accepted something as either a consequence of aging or a consequence of stress or a consequence of their environment, something like brain fog or repeated poor sleep or weight gain or water retention or not the healthiest response to exercise or brain fog or poor short-term recall or any number of things that they've accepted as consequence of life, of aging, of stress, or what have you, that's not a consequence of any of those things. It's a consequence of missing raw material. And I'm going to describe that in detail and tell you exactly how to put that raw material back into your body and how to find out what it is that's missing so you could thrive in a way that you probably never thought possible. Who are you? I'm a human biologist and a researcher, biohacker. My background is in, of all things, mortality science. My undergraduate degrees were in biology. My postgraduate degrees are in human biology i'm fascinated by the human body um its form its function how it can improve its performance and i'm a researcher you know i spent 22 years as a mortality expert in the insurance industry which meant that if we got five years of demographic data on you and five years of medical records we could tell the insurance company how long you had to live to the month. To the month. To the month. And I get a lot of flack about that. People say, oh, if you could predict death to the month, you'd win a Nobel Prize or, you know, only God can tell you when you're gonna die. I mean, and that's very true. I mean, obviously if we do a mortality prediction on it, it doesn't mean December 12th, you know, 2065, you're gonna drop dead on that day. But it is very accurate science. You know, if you think of the most successful financial institutions on the planet as a category are insurance companies. If you look at the failures during the 2008-2009 financial crisis, which was a global crisis, you know, in the US, we had 364 banks fail. You didn't have a single life insurance company fail. No other financial institution, no other investment enterprise, hedge fund, angel investor, venture capital firm would ever put tens of millions of dollars at risk on any kind of investment on one single variable. Yet every time a life insurance company bets on your life or issues an annuity or a reverse mortgage or any number of other financial services instruments that are based on mortality, they're only betting on one factor. They don't care where you are on a mortality curve. They want to know how many more months does this person have left on earth. And it is some of the most accurate science on the planet. And they have perfected this. And the good news is that this science is very accurate, and I'm going to share a lot of the details of that today. But the downside of this industry and the reason why I left it was that during my tenure in this career, I was not allowed to have any contact with the patient or any contact with the treating physician. Now, I'm not licensed to practice medicine. I'm a human biologist. But if I was reading a medical record and saw a life-threatening drug interaction, I could not contact the patient. And this database where this information is drawn from, if this database could see the light of day, I believe it would permanently change the face of humanity. It would upend modern medicine in a way that would be completely catastrophic. Because you see, they have information that no other database has, no other research study, no other financial institution, no other university has. And that is that they know the day, the date, the time, the location, and the cause of death for hundreds of millions of people that they have all of these records on. You know, if I'm a cardiologist and you come in to see me and I put a heart stent in your heart, well, when you leave my office, I don't know what happens to you two months later, two years later, 12 years later. I don't know if you died as a consequence of something that went wrong with the procedure I did, or if you died of something completely different. I don't know exactly what kind of impact on your life that procedure had or didn't have, unless it's contained in a very short-term clinical study. And not that I want to go into the whole science of mortality, but if you had access to this database, you would realize that the most common ailments that we suffer from are not diseases and pathology that are happening to us. They are things that are happening within us. You know, I always say that if I was to boil my entire career down to a single sentence, it would be that the presence of oxygen is the absence of disease. And nothing is more impactful than that statement. The presence of oxygen is the absence of disease. I have yet to find a single disease etiological pathway, something that's happening in the human body, causing pathology, causing disease, dysfunctionological pathway, something that's happening in the human body, causing pathology, causing disease, dysfunction, that doesn't have its roots in a lack of blood oxygen or its roots is not aggravated by something called hypoxia, lack of oxygen. In fact, all human beings die of the same thing. We all leave this earth the same way, right? We all die of hypoxia, lack of oxygen to the brain. That's the definition of death. Only we tend to think of it as an event, a gunshot wound, a bus, car, a stroke, heart attack. But the truth is we are on a hypoxic curve, meaning we are accelerating quickly or accelerating slowly towards the grave. voluminous data-driven industry was that when you deplete certain raw materials in the human body, you get the expression of that deficiency. So what I mean is, you know, we've all heard that disease, you know, travels in families, or you might have genetically inherited hypertension, or genetically inherited hypothyroid, genetically inherited drug and alcohol addiction, diabetes, depression, anxiety. These things tend to run in families. So we've accepted the myth in most cases that these are genetically inherited diseases. But think about this. The next time one of your listeners gets told that, oh, you have genetically inherited hypertension, what we call idiopathic hypertension, or genetically inherited hypothyroid, or if you have this genetically inherited disease, or that genetically inherited disease, stop your physician and say, wait a minute, we've mapped the entire human genome. So we know every gene in the human body. Can you tell me what disease I inherited from my ancestor that's actually causing this condition and watch their face go blank because in the majority of cases we have accepted that disease travels in families when it's actually not the disease being passed from generation to generation it is the inability for their body to refine a certain raw material which causes a deficiency which leads to that disease so in other words there's not a single compound known to mankind, not one. No mineral, vitamin, amino acid, protein, carbohydrate, no nutrient of any kind, no element known to man that enters the human body and is used in the format that we put it in. Everything that we put into our bodies gets taken in by the body and converted into the usable form. If you cannot make this conversion, you have a deficiency. And it is this deficiency that leads to these conditions. And so if you go hunting for that deficiency and you supplement, not for the sake of supplementing, but you supplement for the sake of deficiency, magic things happen in human beings. There is a superhuman inside of every person listening to this podcast. And if they actually were able to test themselves, do a genetic test, and look at what's called their methylation, how their body refines raw materials, and they were able to replace the deficiency that is holding their body back from creating adequate levels of neurotransmitter and achieving normal mood, moving contents through the gut and achieving normal gut function, elevating emotional state, deepening their sleep, quieting their mind, all of these conditions that society has labeled different pathologies and disease, then they would begin to thrive in a way that they just never thought possible. It's so blatantly clear how passionate you are about this subject matter, which begs the question, you know, I remember when I was reading through your story, I uncovered that you'd read hundreds of peer-reviewed papers. I think the quote was something along the lines of, if there's a peer-reviewed paper on the subject matter of biohacking and disease and longevity and those kinds of things, you've basically read it. So this begs the question to me, like why is one individual so unbelievably passionate about this subject matter? And I would like you to take me as early as you possibly can to answer that question in your own life. Where did your obsession, passion, where was it fostered and nurtured so that you spent and committed your life to this subject matter? Where did that begin? So it actually began, I grew up on a very large tobacco farm. My father was a Navy captain. My mom was a flight attendant. She worked for the airlines. But we had a farm that we leased out to other farmers and surrounding this 300 acre tobacco farm were all kinds of animal farms. There were cattle farms, there were chicken farms, there was horse farms. And I was an only child. My nearest neighbor was miles away. In fact, from my home, you couldn't even see another house from my parents' house until I was in my early 20s. And so for me to to play with another child i had to get on a bike and spend half an afternoon biking to their house so i got very familiar with the farm and i was always just fascinated by the veterinarians that would come onto these farms and fix horses and fix cattle and fix sheep and fix chickens and i always thought it was so cool that these animals could be laying on the ground, seemingly about to die, and the veterinarian would come on board and do a bunch of stuff. And the next thing you know, this horse was up, and a few days later, it was galloping back in the field. Or you had all kinds of strange things that would happen on this farm, and there was always someone arriving to just fix it. And I started to get fascinated with medicine, I guess, in that way. I always found it as a way to take something that was sick and helpless and sort of help it get back to normal function. And I found out in the eighth grade, sometime during my eighth grade year, that I was clinically photographic. So I have a clinical level of photographic recall, which is different than just having a visual memory. I'm clinically photographic. So I have a clinical level of photographic recall, which is different than just having a visual memory. I'm clinically photographic. So I have a voluminous capacity to recall things that I read, even if I don't understand them, which is why I never read for pleasure. I'm very cautious about what I read. I'm very cautious about what I look at because I record everything. So I can't be flying on American Airlines and take the seatback magazine out and read the seatback magazine because three months later, I'll tell you where the sales center is for a condo project in Buenos Aires. So I can fill my brain with senseless things like that and record it and regurgitate it, or I can fill it full of things that fascinate me. And so I naturally gravitated towards science. Is that what they call a photographic memory? Photographic memory is usually someone that can remember seven numbers in sequence. They have extremely good recall for things that they visualize. Clinically photographic is being able to regurgitate voluminous amounts of information with incredible accuracy. So I still remember, you know, section 15.2, so paragraph B of my first employment contract that I signed 28 years ago. Really? Really. Yeah. I remember that. It was actually with a trading firm that I signed it with. So, you know, when I read peer-reviewed papers in scientific journals, it doesn't make me more intelligent than someone because very often I can recall information that I don't understand. So I can regurgitate voluminous amounts of information. So subjects that rely on rote memorization like chemistry, biology, neurobiology, microbiology, a lot of these sciences don't actually make sense, right? That you just have to memorize how they operate. Chemistry in a lot of ways't actually make sense, right? That you just have to memorize how they operate. Chemistry in a lot of ways doesn't make sense, right? You just have to remember what happens when you put these two elements together, they create this. When you pull a carbon bond or you double a carbon bond here or hydrogen bond there that the molecule shifts in this way. And so I gravitated towards science. I got my undergraduate degree in biology. How did you do? I did very well. I actually found it quite the breeze. You know, when I was an undergrad, my biology and chemistry were considered the toughest majors. And I went to a branch campus, University of Maryland. A lot of my roommates were political science, not busting on political science majors at all, or philosophy or psychology. And they got to party all the time. And, you know, I had really intense classes, you know, morphology of thalafites and chemistry and, and, you know, biomechanics and, and science and, and a lot of plant biology courses and human biology courses. But I was able to have the same kind of social life that they were because I would, you know, record a lecture and then I would regurgitate the lecture when I needed it during a test. Is it a gift or a curse? Because with all things in life that appear to be gifts, there's often some kind of hidden curse. Well, the curse is that I can't, I rarely read for pleasure. It's very difficult to read for pleasure because I record all of that information. So when I'm going in to read a book, a lot of people read a book that they didn't really like and it doesn't stay with them. Or they read a fiction novel, it doesn't really give them any benefit. It doesn't feed them, but they do it to relax or they do it for entertainment. For me, it would take up storage, right? I would record that information. So there's no need for me to have that information in my possession. I find other ways to relax and meditate and unwind, but reading for pleasure, just not one of them. So I guess that could be a curse. But then I went to grad school for human biology. I went to another four years of school for human biology. So I had eight years of science. And then I was either going to go into chiropractic or into orthopedic medicine. I really liked the idea of orthopedics because, again, an orthopedic surgeon was somebody that fixed people. And an internist was somebody that just managed disease you know they took a obese hypertensive diabetic patient and they just kind of managed their case throughout their lifetime but i felt like an orthopedic was somebody that you know you came in and you had a broken leg and they you know fixed your broken leg and and you were back to normal i like the idea of fixing things and kind of returning function back to people. And I took some time off, you know, after I got my human biology degree and I ended up going into a rare area of science, this mortality science for life insurance and secondary life insurance, where you would take medical records and demographic data, and you would use this in what's called a probabilistic model to predict life expectancy. And I was fascinated by it because I love the big data nature of it. And for years, I actually subscribed to the fact that this was just data, and that I had no influence on this person's outcome. I didn't put them in this position, nor am I responsible for getting them out of this position. But eventually, I think, as both my faith grew and my awareness of the fact that there were human death for the monetary gain of an investor versus taking this information and helping people live healthier, happier, longer, more fulfilling lives. Because by studying medical records, and I read eight to 10 hours a day, six days a week, thousands and thousands and thousands of medical records and extracted from those how it would impact somebody's outcome of their life, how it would impact the length of or the shortness of the duration of their life, their health span or their lifespan. And once you realize there's human beings on the other side of the spreadsheet, you start to feel like you're sitting behind a thick glass wall, just watching blind people walk into traffic. you're sitting behind a thick glass wall just watching blind people walk into traffic and towards the tail end i used to artificially kind of manipulate the record i would say well what if i what if i artificially fix this person's d3 deficiency what if i cured the anemia what if i actually um noticed that they had a gene mutation called mthfr instead of trying to fix them with folic acid i fixed them with methylfolate And you would just see the life expectancy jump. In the model? In the model. Yeah. Right. And this model was based on very large data. So it was very likely to have a real life impact. And when I say we predicted death to the month, it was based on that patient's current position. And it doesn't mean that they couldn't walk out the next day and get hit by a bus or die in a commercial airline disaster or something like that. But there were standard deviations that would account for those kinds of risks. What the insurance company wanted to quantify or the annuity company or the reverse mortgage company wanted to quantify was based on this person's – everything that we know about this person right now, the function of their liver, the slightly hypoxic anemic profile that they have, the deficiencies that they have in vitamin D3, the suppressed immune system, the elevated visceral fat, the body mass index, and the bone mineral density, based on all of these things, when do these things all meet and actually cause a catastrophic event? And there were things called mortality debits and comorbidity debits, and we put all these together, and I really liked the data of it. But realizing that there were human beings on the other side, the spreadsheet just woke me up, and I said, you know, I have this chasm of knowledge, even though I'm not a physician, I have a fundamental understanding of human physiology. I'm fascinated by human physiology and ways that we can improve lifespan and healthspan. And so why wouldn't I take this gift and get into an industry like wellness or functional medicine where I could actually make a difference? Where instead of predicting death, we could extend life. And in doing so, help people have healthier, happier, longer lives, more fulfilling lives, get rid of a lot of the ailments that people suffer from. I mean, when you start to peel back the way that modern medicine defines a lot of conditions that we have, take depression or anxiety or ADD or ADHD, because a lot of your listeners are probably entrepreneurs. And a lot of entrepreneurs have very active minds and they've been told they have attention deficit disorder, right, or attention deficit hyperactivity disorder. Well, an attention deficit disorder is not an attention deficit at all. We've misdefined these things. It's an attention overload disorder, because you see in the human brain, we don't just create thought. We also dismantle thought. It is just as important for you to be able to create a new thought or feeling as it is to dismantle it, right? And if you don't degrade thought, called catecholamines, if you don't degrade them, then there's a gene that governs this, catechol-O-methyltransferase. If you create thought at a faster rate than you degrade thought, then the mind gets very clouded. And so attention deficit disorder is attention overload disorder. It's too many windows open at the same time. So modern medicine says, well, if the mind's racing, let's put an amphetamine into the body, race the central nervous system to match the pace of the mind. And this is a very poor choice, right? Because eventually this will burn you out, can actually change the neuroplasticity of the brain. Rather than put the right amino acids back into the body, the right B-complex blend, the right methylated folates, so that the mind can actually begin to quiet. What would you say to someone that says that ADHD is also in some ways a result of some early trauma? Well, you know, trauma is always fascinating to me. Trauma can trigger methylation. Trauma can interrupt the methylation cycle, right? But the idea in modern medicine that you have some kind of trauma, you have a disrupted relationship with your mother, for example, and that somehow we're going to go and put neuroplasticity altering chemicals into the brain and it's going to fix this 30-year broken relationship you have with your mother to me it doesn't yeah yeah make any sense right and so it's not at all to poo poo trauma to to put trauma down depression really exists anxiety really exists but but if you actually look at how we define these conditions take depression for example we define depression at least in America, we define depression as an inadequate supply of serotonin, right? So if you are low in serotonin, you're by definition depressed. So then you would think that the solution would be to raise serotonin, right? If we define depression as low serotonin, you'd think that the solution would be to raise serotonin, but that's not what we do. We take people that are depressed and we put them on SSRIs, selective serotonin reuptake inhibitors. And what these do is they ration what little serotonin these people have. So by definition, it never raises serotonin. So by definition, it never ends depression. I mean, I have clients come in to see me all the time and on our clinical team, and then I'll say, well, how long have you been on antidepressants? They say 15 years, 18 years. And my first question is, well, how long have you been on antidepressants? They say, 15 years, 18 years. My first question is, when did you think it was gonna kick in, right? So if we understand that serotonin is actually methylated in the gut, this process that I'm talking about is called methylation. We actually make serotonin from an amino acid called tryptophan, the one that's famous for making you sleepy after Thanksgiving dinner. I know you guys don't have Thanksgiving in the UK, but I'm from America. So because we eat a lot of turkey on Thanksgiving and turkey has a lot of tryptophan. So when you take tryptophan, methylate it into the neurotransmitter serotonin, of which 90% resides in your gut. Methylate is basically processing. Yeah, the processing. It's like the refining process. Crude oil gets refined into gasoline. Amino acids get methylated into neurotransmitters. And so this process of methylation, when it's broken, and it can be relatively easy to fix when it's broken, it means that we have an impaired ability to create, we have an impaired ability to refine a raw material and it leads to this deficit. Well, serotonin, for example, 90% of it resides in your gut. So if you don't have it here, you can't have it here. So depression rarely begins in the outside environment. It usually begins in the gut. Now, it may be trauma that led to the deficiency, but the fix is not in a chemical or synthetic or pharmaceutical blocking the brain's capacity to uptake these neurotransmitters. The fix is in restoring adequate levels to the body so it can naturopathically make its way back up the vagus nerve and arrive to the brain. Similar things are true with anxiety. I mean, if you actually have ever suffered from or know somebody who's suffered from anxiety, if you ask them three questions, you can find out very quickly that their anxiety is not coming from a cluster of symptoms. It's not coming from a cluster of symptoms. It's not coming from their outside environment. It is coming from within them. It's coming from their physiology, right? I mean, if you know someone who's suffered from anxiety and you say, well, have you had anxiety on and off throughout your lifetime? The most of the time they'll say yes. And then if you say, can you point to the specific trigger that causes it? Very often they'll say no. I mean, yes, I know some of my triggers, but I could be sitting in a podcast just like this in a very calm environment. There's no threats around. And all of a sudden I get overwhelmed by anxiety. I can be driving home from work on an otherwise innocuous day and I can be overwhelmed by anxiety. Well, that is not coming from your outside environment, right? This is coming from a process called methylation and it is caused from excess catecholamines entering the brain and an inability to down-regulate these. So the body's entering this mild fight or flight response without the presence of a fear. See, remember that as sophisticated as we like to think our brains are, it's really not. Our brain is very primal. You know what the brain cares about? The brain cares about survival. And so it doesn't care how fat or skinny you are, how pretty or ugly you are. It doesn't care about your skin, your hair. It cares about survival. And so when we understand that the brain does not know the difference between perception and reality, we start to understand how it can play tricks on us. So I always use the example that let's say you drove home tonight and you got out of your car. When you got home, you got out of your car and somebody was standing in front of you with a knife. It's a very real threat, right? You'd have a fight or flight response. Your pupils would dilate, your heart rate would increase, your extremities would flood with blood, your hearing would get very acute, your brain would flood with catecholamines, you are getting ready to fight or flight. But you could also be laying on the 30th floor of a condo building in bed and start thinking about getting eaten by a shark. There is zero chance of a shark getting out of the ocean, going up a 30th floor elevator, right? Coming into your condo and biting you in that bed. But you can have the exact same response. If you're watching a movie or something. Exactly. So one is entirely real. One is entirely perceived. The physiologic response is identical. So now once we understand this, now we begin to understand how I can feel the presence of a fear, which is what anxiety is. It's a fear of something happening in the future. Usually it's not going to happen. Usually it hasn't happened in the past and is not likely to happen. But this fear starts to build up. You start to get very anxious, it can actually change your heart rate, to the point where you can panic attacks can land you in a hospital. Or it can be mild enough that it just causes you anxiousness and mild anxiety, but there's no presence of a fear. And so you start trying to correlate it to your outside environment starts to drive you crazy because you go, well, I don't get it. I'm on vacation with my wife and my spouse and my kids. And I'm in a resort of a lifetime. I've been here a thousand times. I love this place. There's no reason I should feel like this. But all of a sudden you have this feeling of anxiousness, anxiety. So these are lack of raw material in the human body. My mission is to try to help people by taking a genetic test once in their lifetime, find out where is methylation broken, and then stop supplementing just for the sake of supplementing and start supplementing for this deficiency so your body can thrive. In the case of people that are listening to this now, and they can pinpoint the moments where they've got an anxiety. So say they've, I remember I had one guest on the podcast, maybe two years ago. And after he became famous, he developed social anxiety. So whenever he would be around a lot of people, he would feel that sense of anxiety. And then from that sort of catalystic moment, then when he's at home, he'd get the same rush of anxiety, but he would point to that catalystic moment of becoming famous. And then some things had happened in his life. And then he'd get anxiety at home when nothing was going on. Right. In that situation, what's the new- So there you go. So now you've interrupted methylation because there's one where there's the presence of fear and there's one where there isn't. There's the absence of a fear. So to be very specific, anxiety, true anxiety does exist, but you can point to the specific trigger that causes it. So for example, if you have a fear of heights and you walk to the edge of a 30th floor balcony and look over it, you're going to feel anxiety. If you're claustrophobic and you step on a really crowded elevator, you're going to feel anxiety. But if you're claustrophobic and you're sitting at home and you start to become overwhelmed with anxiety. This is actually not coming from that trigger. This is coming from your physiology. And the way that we deal with stress, right? And like cortisol, when you measure cortisol levels, cortisol is not really a measure of how much stress is in your life. It's a measure of your body's reaction to stress. So why are some people more resilient to stress and don't have anxiety attacks? And why are other people not as resistant? Again, this is not to say that if and don't have anxiety attacks and why are other people not as resistant again this is not to say that if you didn't have a violent attack in your life or or a terrible car accident that sometimes when you've had a you know a vehicle accident you approach an intersection the the the memory of having been t-boned you know recently and badly injured is going to give you anxiety but the majority of people are not suffering from that type of hyper specific situational anxiety suffering from something called generalized anxiety or idiopathic anxiety which means of unknown origin so for somebody who's very famous and gets into a crowd and doesn't know who's coming at them that's a very i wouldn't even define that as anxiety, that's a very primal instinctual reaction to a real fear. Just like walking to the edge of a 30th floor balcony. What's not a primal reaction to a real fear is when there is no presence of a fear, especially if that incident has never happened and you aren't even sure what you are afraid of or why you are anxious or why you have anxiety, then this is coming from your physiology. So how would you treat that? You'd look at the different, there are five major actionable genes that I like to look at in their, what's called their suballeles. And when you find out what they're deficient in, you start to supplement with things like SAMe, S-adenosylmethionine, methylated forms of vitamins, L-methionine, the proper balance of B-complex, methylated forms of folic acid or folate called methylfolate. And what happens is now the body has the capacity to degrade these neurotransmitters that are causing this fight or flight. This group of neurotransmitters called catecholamines and the anxiousness that follows. And you'll find that the majority of people that suffer from idiopathic anxiety or generalized anxiety because of low serotonin, they also have gut issues. You show me a person that's truly depressed and I'll show you somebody that's also suffering from severe gut issues, either gas or bloating or diarrhea, constipation, irritability, cramping, because the same neurotransmitters that affect these emotional states also are responsible for the motility of the gut, the speed of the gut. This is the most overlooked thing in all of bariatric medicine, because people that believe that they have all of these allergies, well, I'm allergic to wheat, soy, corn, dairy, blueberries, bananas, gluten. Yes, sometimes those individual allergies do exist, but the majority of time, even if you talk to somebody who says, yeah, I get bloated or I deal with gas or cramping or diarrhea or constipation or irritability, I deal with all of these gut issues, irritable bowel syndrome, Crohn's disease, ulcerative colitis, all these names that we give to conditions of the gut. When you ask them, well, what are you allergic to? And they give you this laundry list of things. And then you ask them another question and say, well, if you're really allergic to corn, is there ever a time that you can eat corn and not have a reaction? The majority of time people will say yes. Okay, well, right there, you know, you don't have an allergy. Allergies are not transient. Allergies are consistent, right? You don't wake up Monday morning and being allergic to milk, and then you're unallergic on Wednesday afternoon and then re-allergic on Saturday morning. But what happens when people have gut issues that they can't explain is they always correlate it to what they last ate. And it's hard to make this connection. They're like, well, wait a second. I ate the same thing Monday and I was fine. And I ate the identical food on Wednesday and I blew up like a tick. So this is not an allergy. This has to do with the motility of the gut. So if you don't know what gene mutation you have that is causing a deficiency, then you don't know what to supplement with to restore gut motility. But once you do, the gut goes back to its to supplement with to restore gut motility. But once you do The gut goes back to its normal pace. What's gut motility? It's the pace of the gut So if you remember henry ford was actually not made famous for the automobile He was made famous for something called the assembly line So the assembly line was just a glorified conveyor belt, right? And when you walked into his factory, they put a part on it on one end, and about every six feet, somebody stood and tinkered with that part. So it went to me, I tinkered with it, it moved to the guy to my right, he did something to it, moved to the guy to his right. And by the time it reached the end of that conveyor belt, it's fully assembled. This is very analogous to how the human intestinal tract works. It's 30 feet long. It's a giant conveyor belt. You put parts on it at one end as they exit the stomach in a very acidic environment, and it moves slowly towards the rectum. And before it exits the rectum, it's in a relatively alkaline environment. So instead of having people standing along a conveyor belt, you have bacteria that are graded by pH. The sequence is very important. So imagine what would happen if Henry Ford walked into his factory one day and doubled the speed of the conveyor belt. The entire assembly line would break down, not because there's anything wrong with the parts, the contents, not because there's anything wrong with the people that are working there, the bacteria, but because you changed the speed. What if he went in there one day and reversed the conveyor belt? What if he just ran it in the opposite direction? It would screw the whole thing up, right? So by changing the pace of the gut, the speed of the gut, the conveyor belt, I've ruined this sequence of events. And I spend a lifetime trying to figure out what's wrong with the parts, what's wrong with the workers, what's wrong with the conveyor belt itself. Nothing. It's how quickly or slowly it's running because the motility, this peristaltic activity is affected. that all of a sudden these strange allergies eviscerate and all of this gas and bloating and diarrhea and constipation and irritability and all of this inability to equate things that I'm eating back to what is going on in my gut seem to go away. It's true with all kinds of conditions. You know, we have subscribed in this world to the fact that we are so affected by disease and pathology. And once I get you to subscribe to the fact that you have a disease and you can get you to subscribe to a lifetime of medication you know this is true in hypertension you know it's another huge you know we think genetically inherited disease and a lot of families have you know grandfather had high blood pressure um dad had high blood pressure now the son has high blood pressure so they go up it's genetic genetically inherited high blood pressure okay well the son has high blood pressure. So they go up, it's genetic, genetically inherited high blood pressure. Okay. Well, what gene is being passed down from these generations to cause this? Well, they don't know. Okay. Well, we know all of the genes. So if we don't know what gene it is, then let's look at the methylation genes. Let's look at the genes in the human body that refine raw materials. Because if you have a certain deficiency, for example, in hypertension, very often, not always, but very often it's caused by elevated levels of an amino acid called homocysteine. You have it in your bloodstream right now. I have it in my bloodstream. Every listener to this podcast has homocysteine in their blood. Because in the process of normally refining this amino acid into something called methionine, we keep this level at a reasonable level. But when you can't break homocysteine down, it rises. When homocysteine rises, it's very irritating to what's called the endothelial lining of the blood vessel. It irritates the lining of the blood vessel. Well, when you irritate a blood vessel, it clamps down. Well, if you make the pipes smaller in a fixed system, pressure goes up. You have 63,000 miles of blood vessel in your body. It doesn't take much narrowing to drive pressure up. And so these people end up with hypertension, high blood pressure, and they go to the doctor and the doctor does an EKG. It's normal. An EEG, it's normal. Stress EKG, it's normal. Cardiac cath, normal. Heart and lung sounds, normal. Die contrast study, normal. The entire cardiac workup's normal. And then they still start pounding on the heart with medication because they can't figure out why the blood pressure's up. But they never actually looked at what raw material was missing in their body, not allowing them to bring the homocysteine down and allowing the vascular system to relax. As the vascular system relaxes, pressure returns to normal. You spent 22 years at that insurance company looking at the database that you described. And as you're saying that, much of the sort of medical profession, and I think most of society- Yeah, and I don't want to attack the medical profession. I really, you know, I'm in no place- I am a huge lover and believer in modern medicine. It's saved people very close to me. I mean, and trust me, if I hit a windshield at 20 miles an hour, I want a surgeon, I want painkillers, I'm going to the ER. But there's been this kind of- it's not a preventative approach to disease that we've taken over the last couple of decades. It really is, you know, to put a band-aid on something that's that's emerged and as you said at the start of this conversation these predicaments these um diseases emerged decades before we even see the symptoms oftentimes um at the end of your 22 years at the insurance company you started to look at these things which you call modifiable risk factors. Yes. What is a modifiable risk factor? And what are the most common modifiable risk factors in your view? Well, I mean, one of the most common ones that I saw was modifiable risk factor is, it's a risk factor that you have. And if you changed it, it would have modified it, it would have a demonstrative impact on the trajectory of your life. So for example, anemia, right? Low hemoglobin, low red blood cell count, low oxygen transport in the blood will exacerbate just about any condition that you have, right? So if you are hypertensive and anemic, if you are diabetic and anemic, if you are morbidly obese and anemic, meaning you have low blood oxygen because you have low red blood cells, low hemoglobin. What if you could modify the anemia? Well, if you could modify the anemia, carry more oxygen in the blood, you'd be much more resilient to all of these conditions, right? So the same thing is true with dementia and Alzheimer's and cognitive function. As you impair cognitive function, you increase the incidence of all-cause mortality. So a modifiable risk factor would be something like looking at the levels of vitamin D3 in the blood, in the body. So it's estimated that roughly 50% of the world's population is clinically deficient in vitamin D3. Why is that important? Well, vitamin D3 is the only vitamin that a human being can make on our own. There's hundreds of vitamins in your bloodstream right now. You're only capable of making one, and it's vitamin D3, cholecalciferol. We make it from sunlight and cholesterol. You don't even need to eat to make this vitamin. There is not a single cell in the entire human body that does not have a receptor site for this vitamin it also acts like a hormone it's a calcium transport molecule it's enormously impactful in your immune system clinical deficiency in vitamin d3 was at one time the second leading cause of morbidity in covet you know when they said covet disproportionately affected minorities if you ever heard that, it's true. So how did COVID disproportionately affect minorities? It wasn't like the virus didn't start going after certain minority populations. Well, it actually did, but it wasn't because they were not minorities that had to do with the pigment of their skin. The darker the pigmentation of your skin, the lower the vitamin D3. The lower the vitamin D3, the more compromised the immune system. The more compromised the immune system. The more compromised the immune system, the more susceptible you are to attack. That suggests that if you have darker skin like I do, that you should spend more time in the sunlight. Yes, it does. It absolutely does. I mean, you and I, just based on our skin tone, if you and I in a pair of board shorts went out and spent 30 minutes in sunlight, I would manufacture about 25,000 IUs, international units of vitamin D3, you would manufacture about 3,500. Fuck, you know, that's not fair. It's not fair. We can run fast. But you also look better older and you don't age, you don't get wrinkles as much, and you have better oily skin. Great basketball. There's some trade offs. So when you look at African American populations, Latino populations, Middle Eastern populations, the percentage of vitamin D3 deficiency approaches 85%. The truth is most of us are not getting enough sun. It's not that we're getting too much. So let's talk about the depletion of this one nutrient. Can I just ask then before we move on to the depletion of that nutrient, because I want to understand why that is. My guess was that because people with darker skins have grown up in sunlight, we spent more time in the sunlight, I'm guessing, which meant that we adjusted our production of vitamin D3 to suit the environment we lived in. Yeah, you actually have a better barrier, right? You are less susceptible to skin damage from sunlight than someone with lighter skin, right? And so if you actually look, because I used to be a population mortality expert, you know, the longest life expectancies were centered right around the equator of the earth. If you look at most of the blue zones, they're gonna be sort of closer to the equator of the earth. They're gonna be closer to places where the sun shines longer periods of time throughout the year. The blue zones? The blue zones are, are sorry the blue zones are areas of the world where people live um extraordinarily long life expectancies right mediterranean diets so the longest life expectancies are centered around the equator so for every 20 degrees latitude, because this is longitude, every 20 degrees latitude, there's a precipitous drop in life expectancy until you get to the poles, where when I started in this industry, the lowest, the shortest life expectancy on earth existed. So when I was born in 1970, a true Eskimo had a 56 year life expectancy. 56 years. An Eskimo had 56. True Eskimo. So these are at the polls. Now why? Because they would go months without seeing the sun. And when they did see the sun, they were layered up. So they had brittle bone disease, they had autoimmune disease, they had immunocompromised conditions, all kinds of conditions that go with severe depletion in vitamin D3. And so I would see this in the medical record all the time, right? I would see medical record after medical record. You know, a patient has been, had a decade or longer of clinical deficiency in vitamin D3. Well, when you deplete this nutrient and you don't supplement for it, by the way, vitamin D3 is so easy to supplement for. Everybody should be taking vitamin D3 unless you're getting adequate sunlight. 5,000 IUs of D3 with 80 micrograms of K2. And you deplete this nutrient. Eventually, you will develop rheumatoid arthritis-like symptoms. Now, you don't have rheumatoid arthritis, but you have the same identical symptoms when you're depleted in vitamin D3. You start waking up sore and achy like you had a workout the night before when you haven't. The soles of your feet and your ankles are tender when you get out of bed in the morning to walk to the bathroom and take your first pee. Your knees and hips and ankles start to stiffen up. Then it goes to cross the shoulders and eventually it's hard to make a really tight fist. Well, if you go to the wrong physician, family medicine practitioner that's seeing a lot of high volume of patients, you start to give them that description. They go, you know what? You've got rheumatoid arthritis. I'm going to hit you with something called corticosteroids. I'm going to put you on some prednisone first, and then we're going to transition you to a corticosteroid like a methotrexate or one of these, and you're going to be fine. It's going to push this off into the future. Well, we knew that if you started corticosteroids, that at first they had an anti-inflammatory effect, but you had roughly six years and one day until you were having a joint replacement. Because first they're anti-inflammatory, but then they eat the joint like a termite. And so it was so accurate that if you were a 60-year-old female and you were misdiagnosed with rheumatoid arthritis because you had a vitamin D3 deficiency, I would artificially advance your age six years. I would schedule the joint replacement. That wasn't required, by the way. I would schedule the joint replacement. And then I would begin to reduce what's called your ambulatory profile, how well you ambulate, how well you move. And we know now that sitting is the new smoking, right? Sedentary lifestyle is the leading cause of all-cause mortality. And so as I start to reduce your ambulatory profile, I bring in all of the diseases that exacerbate with reduced motility. So if I back this up, you were diagnosed with a condition that you didn't have because you had a deficiency in a very simple nutrient. So then you were put on a medication that wasn't required. This caused you to have surgery that you didn't need to replace a joint that wasn't initially going This caused you to have surgery that you didn't need to replace a joint that wasn't initially going to be defective. This reduced your mobility and it brought diseases forward that you never would have caught. And you died early from a condition you never would have had because you had a deficiency in a nutrient that was misdiagnosed, mistreated, and led to your early demise. And I can give you hundreds of examples like this and this is why if you look at the 2016 harvard study the 2019 johns hopkins study medical error is the third leading cause of death medical error medical error is that misdiagnosis misdiagnosis and medical error the average american over 50 52 is on five prescription medications. These are synthetic chemical pharmaceuticals, right? And in a lot of cases, we don't know the long-term side effects of these. In a lot of these cases, we actually don't even know the mechanism by which they work. If you actually open the PDR that comes, the disclaimer that comes with a lot of these medications, you'll see that they say mechanism of action unknown. Well, it's a sleep medication. How does it make me sleep? I don't know. Just makes you sleep, right? In fact, most people that are suffering from sleep issues are suffering some sleep issues, not because they can't sleep, not because they're not tired, because their mind is keeping them awake, right? And if you have a gene mutation called COMT, catechol-O- omethyltransferase, then what happens to you is you lay down to go to sleep and there's nothing more frustrating than one spouse having this gene mutation and one not. Right? Oh my God, I think you're talking about me and my partner. I'm the one that, I need some like sound or something to go to sleep because she's just out like a light. There you go. See, and nothing's more frustrating because- So annoying, I sometimes sleep in another room because- Okay, so the reason why you're not falling asleep is because when your environment quiets, your mind wakes up. And if you actually hone in on the kind of things you're thinking about, you're thinking about the most innocuous little nonsense. Right? I mean, it's like, did I get everything on my grocery list? Did my belt match my shoes today? Did I return that email? It's nothing that couldn't wait till the next day 100 yeah and so and sometimes you can even catch yourself going what am i doing thinking about the color of dishes i'm gonna have if i threw a party which i'm probably not gonna throw it's just like how did i get down this rabbit hole right and um this is because you know the you are not quieting the mind at the pace that you are exciting the mind. So in other words, when you lay down to go to sleep and your environment quiets, your mind starts to wake up. So you need to be distracted so that your mind doesn't continue to run. But if you just put the right amino acid balance, in fact, I would bet my career you have this gene mutation. I will pay for your test if you don't. It's $599. It's a cheek swab you do once in your lifetime and it will tell you if you have this gene mutation, COMT. And if you do, I'll tell you exactly what supplement to take and it will become a permanent thing of your past. What supplement can I take? Don't end your sentence there, Gary. So it's called L-methionine. Oh, I've heard of L. And you take L-methionine, you take methylfolate, and depending on where the gene break is and how severe it is, you may add something called SAMe, S-adenosylmethionine. And these sound like fancy names. They're just fancy names for vitamins and amino acids. All of these are in your bloodstream right now. So when you look at the sequence of breaks that somebody has, then you can tell them exactly what supplement to take. When they take the supplement, their body's no longer deficient, right? Deficiencies over time express themselves as all kinds of things. You know, there's a lot of people that can't reach elevated emotional states for any prolonged period of time. This is because mood and emotional states are recipes, just like a chef bakes a recipe. But if you went to a bakery chef and you said, you can bake whatever you want, you just can't use butter. Okay, it doesn't sound like a big deal. It's just one ingredient. But think of the number of cookies, pastries, pies, brownies that it would affect. It would affect so many different recipes just by removing one ingredient. This is the same thing that happens in human beings. If you couldn't use serotonin to assemble mood, okay, now any emotional state that requires that neurotransmitter as a part of its structure, you can't assemble. And as it relates to that mutation, when people hear the term mutation, they think of it as being something that's happened in me. And I'm maybe, you know, one of a few that have that mutation. Mutation sounds like a... Yeah, it does sound, it does sound like an alien's about to start like growing out of your stomach. Yeah, but it also sounds like, you know, my sister won't have it and it's just me. Okay. So a gene mutation means that, you know, we get a copy of our genes from each parent. Yeah. Right. And then these, the copy of our genes from each parent, right? And then the copy of these genes is passed down to us. And we know, you know, most of us are familiar with basic genetics, eye color, skin color, detached ear lobes. But there's nothing you can do with that information. So beyond your ancestry, you know, whether you've got Irish heritage or Native American Indian or what have you, I mean, those are, if you did a 23andMe, you'd be able to find your ancestry, whether you've got Irish heritage or Native American Indian or what have you. I mean, those are, if you did a 23andMe, you'd be able to find your ancestry, which is also your genetic history. That's not actionable information. When you look at the genes of methylation, right, and there are several of them. I look at five majors and a few minors. When you look at the genes of methylation, these are the genes that code for how materials are taken into the body are refined into the usable form. And these are not mutations that you're going to suffer from. But what they do is they cause deficiencies to arise in the body. And it's this deficiency that leads to the expression of these conditions. And so when you can't adequately methylate neurotransmitters, you have a deficiency in a certain neurotransmitter, any mood, any emotional state that requires that neurotransmitter, you can't assemble. If you have a deficiency in the ability to quiet the mind, then you have excess neurotransmitters in the brain, which cause it to stay in an awakened state. So that gene that is responsible for my loud mind when I go to sleep, do you have any idea the amount of people that have that particular gene? Well, 44% of the world's population has a gene mutation called MTHFR, Well, 44% of the world's population has a gene mutation called MTHFR, methylene tetrahydrofolate reductase. This is a genetic mutation that impairs the ability to convert folate into methylfolate or folic acid into methylfolate. And that might not sound like a big deal, but folic acid is one of the most prevalent nutrients in the human diet. And the thing about folic acid is that folic acid is an entirely man-made chemical. Folic acid is not found anywhere on the surface of the earth. You can't find folic acid anywhere naturally in nature. And yet we give folic acid to pregnant women when they get pregnant because we tell pregnant women that folic acid prevents neural tube defects. Well, that's patently false. Folic acid doesn't prevent anything. Folic acid has to be converted first into tetrahydrofolate, then dihydrofolate, but eventually it becomes something called methylfolate, and that prevents a neural tube defect. Well, what if like 44% of the population and 44% of women, this woman has this gene mutation and you give her folic acid, the man-made version of folate, and she can't convert it into methylfolate. This is where postpartum depression develops. And I have not seen a single peer-reviewed published clinical study linking elevated levels of hormones in female pregnancy to postpartum depression. But if you give a woman that can't methylate folic acid 1400% of the daily allowance of folic acid, and she can't break it down, then she goes nuts. And then eventually the pregnancy ends, the depression goes away, and so she blames it on the pregnancy, not on the vitamin. And we do this over and over and over again. We have, in the modern industrial world, we try to synthesize what occurs naturally in nature in a laboratory. You know, there's three types of B12, right? Adenosylcobalamin, hydroxycobalamin, methylcobalamin. These forms of B12 occur naturally in nature. There's one that we make synthetically in a laboratory. It's called cyanocobalamin. We make it from hydrogen cyanide. It is useless in the human body. The human body actually has to take it and convert it into the active form of B12 called hydroxycobalamin. And so a lot of times the supplements that we're taking us are causing more harm than good. And we're taking them because a doctor said, well, I'm pregnant. My OBGYN told me to take this synthetic chemical called folic acid, which it's always beyond me why anyone would think that something that we make in a laboratory that's entirely synthetic, that isn't natural, that isn't found anywhere on the surface of the earth could ever be required for optimal health. I mean, just on its surface, that sounds strange to me. But yet we do, and we don't understand this process of methylation. So once you understand where your genetic mutations are where the genetic inability to refine raw materials is you can go about supplementing targeted supplementing for that deficiency so you would recommend people take a genetic test to find out these answers right i'd recommend everyone take that test once in their lifetime you know i i'm i'm in the camp that we're, as humans, we're not broken. I totally agree with you. Do you know what I mean? And so whenever there's something going on in my body or, you know, a way that I am or a habit I have that I'm like, why do I do that? I'm always trying to figure out the sort of, I guess, the ancestral reason why that might have helped me to survive or what's going on. So when I hear mutation, it sounds a little bit like broken to me. Yeah, Yeah. So I shouldn't use that. I mean, we use the term mutation, but basically, you know, in our ancestry, right, we pass on our entire genetic code, which a lot of that is hair color, eye color, skin color, you know, our heritage, but we also pass on how our body refines certain raw materials. And it's not to say that it's good or bad or broken or fixed, but there are certain families and human beings that have an inability to refine certain raw materials. And if we could be aware of this, it is astounding what happens to human beings when you just give their body the raw material to do its job. You know, lots of people that have attention deficit disorder have trouble sleeping or also have trouble with prioritization. They just don't know it. They'll say things like, I work really well under pressure, right? Most entrepreneurs say, I work really well under pressure. Do you say that? Oh my God. Okay. I only work when there is pressure. I was that kid in school that would only do the test when there was an hour left to go or the homework or whatever. Yeah. So many entrepreneurs are that way. And when you say you work really well under pressure, physiologically what you're saying is I lack the ability to set priorities internally. So I use external pressure to set my priorities. And why is that? It's because when you don't have the right methylated nutrients in the body, you lend equal weight to every thought that comes into your mind. Right? So you could be working on, you know, the deal, the joint venture business closing of a lifetime, right? And you got the contract, and you have 45 minutes to get it back to your lawyer, you know, the deals permanently going to go away. This is a life-changing deal. You've waited for this your entire life. It's a very meaningful event. You're working your way through this legal document and you look over at your phone and there's an Instagram message. And you're like, oh, that's my neighbor's cousin's kid fishing in a lake. I wonder if he catches a fish. So you start looking at it and you're like so you still cast and oh he didn't catch it that time now meanwhile this has gone from 45 minutes to 35 minutes and now it's down to 25 minutes and all of a sudden you go oh you know crap you turn the phone off which didn't mean anything anyway and you focus all of your attention and because you're very intelligent and you and you execute and you can hyper focus you hyper focus on this and you worm your way through it and one minute before the deadline you hit the send button lawyer gets it the deal goes through and you go i work really well under pressure what's that got that process you described there is many people describe that as procrastination it's procrastination but it's procrastination because you give equal weight to all of these different variants that come into your field. So you actually give equal priority to very disequal events. But also, you know, sometimes if I'm looking at that contract, the contract clause is a little bit difficult. And my attempt to maybe escape discomfort would be, I'll just go do the dishes. And so I had Nir Eyal on the podcast. He wrote a book called Indistractable. And he says we're creatures that, we're not pleasure-seeking creatures. We're creatures that are avoiding discomfort, essentially. I think that aging is the aggressive pursuit of comfort. Could not agree with you more. Let's use Dana White as an example. Okay. Because, you know, the reason why I came across your work is because of a clip that, you know, involved Dana White. And I'm sure you, I know you get this a lot because I've seen you be asked about this in interviews, but for context for anybody listening, I don't know where I was or, ah, now I know what happened. My friend sent into a WhatsApp group a clip of Dana White talking about you. And that's the clip that made me go down the rabbit hole. I watched that clip. I then did some research. I then watched a series of videos of you online talking about health. And I watched you asking audience members to stand up and name the health issue they were suffering. And you on the spot told them what was missing from their life, their diet, whatever it might have been. You kind of diagnosed them in a way of, and then I reached out to you on Instagram and that's why you're here. But if we go back to the start of that, it was that story that Dana White told that had me so compelled to reach out to you. For anybody that doesn't know, and there will be some people that don't know, Dana White is the president of the UFC, which is the big fighting tournament where everybody kicks each other's heads in. So what's your take on the Dana White story? So Dana White is an example. I mean, he's just a celebrity example, but he is a shining example of the vast number of people, men and women in his age category that have given up on the capacity to thrive. They've accepted that they have hypothyroid, hypertension. They wake up sore and achy in the mornings that they don't have a response to exercise. They have a little bit of spare tire, their brain foggy. They're on three or four medications. In his case, he was on seven medications at the time, three of which were for blood pressure. He was on, you know, I think a thyroid medication was also on, he's been very public about this by the way. And again, I have to say, I'm not licensed to practice medicine. So it was my clinical team that came up with the diagnosis and I communicated it to Dana. I do train physicians to read blood work and genetic testing, but I can't practice medicine. But the point is that when I met Dana, all he wanted to do was for me to predict his life expectancy. And I hadn't done that in almost seven years. I left that industry for a reason. I don't do it anymore. The test that I do does not predict life expectancy. The genetic test and the blood work that I do will not tell you how long you're going to live. I have no interest in predicting death anymore. I only have an interest in extending life. And when Dana was only interested in me predicting his death, so I said, okay, for Dana White, I'll come out, I'll meet with you. I'll do a blood test on you, a gene test on you. I'll pull all your medical records and I'll give you your life expectancy. But what I did was went out and got his blood work and his gene test. And I was actually in bed at 1.30 in the morning when the lab was running his blood work. And I've had seven life threatening alert calls in the middle of the night, because when you drop blood work off at the lab, the lab runs it through the night. If they find a life-threatening alert, they call the account holder, right? So I owned a company, I was on the account. So LabCorp calls us at one o'clock, 1.30 in the morning, says, hey, we have a life-threatening alert on a patient. I was like, Whoa, what's the patient's name? They said, last name is white. I said, Dana White. They said, Yeah, well, what's the life threatening alert? They said triglycerides are almost 800. Now triglycerides a measure of blood fat, okay, it shouldn't be above 149. At 200 or 300. This is a cataclysmic level in the blood, especially in a fasted state. Okay, we pulled his blood in a fasted state. They weren't 400. They weren't 500. They weren't 600. They weren't 700. They were like 768. So they were, I mean, this is an enormous number. And so I said, okay, I need to get the blood work over to the doctor. And when they sent the blood work into the portal, I then saw that he was insulin resistant. He was hyperinsulinemic. He was pre-diabetic. He had skyrocketing levels of cholesterol. He was hypertriglyceridemic. He was hyperhomocystinemic. This homocysteine that I told you elevates and causes the blood vessels to constrict. I mean, he had all of these conditions. I literally at that moment booked a flight for 7.30 or 8.30 in the morning to head out and see him. And because I said, I need to go see him in person. And I remember, I think his assistant called me and I was at the airport and she said, "'Hey, Dana wants to know if his life expectancy's in." I go, well, I'm on my way to see him. And she goes, oh God, is it like that? I said, yeah, it's like that. And so I flew out to see him and I sat down with Dana. And when we talked about the blood work, I didn't even explain the levels. I explained the symptom. I did not know that he was on a CPAP machine, but I said, I am surprised that you can actually sleep through the night because he was so hypoxic. Red blood cell count, hemoglobin levels. I'm surprised that you can even sleep through the night without like just waking up, choking, gagging. He's like, dude, I'm on a CPAP machine. I wake up every night i throw up in the middle of the night i throw up so much i'm losing my voice um and i said this level of caudication triglycerides in the bloodstream i'm surprised you can even bend down and tie your shoes that's not painful to tie your shoes not that it's not restrictive to tie your shoes it's not painful like it doesn't feel like the skin's going to peel off your legs and he went what the i mean he slammed his hand down he was like how did you you know how did you know that and i said dana your level of brain fog and fatigue right now has got to be at a crushing level of fatigue i don't know how the only thing getting you through the day is your own stubborn willpower and i'm surprised you can remember anything from one minute to the next and his whole staff was like dude he's so forgetful he passes out in meetings he's sleeping in the planes he he's gagging, snoring. These were not things I necessarily knew about him. So I began to describe all the outcomes of these kinds of conditions. And I said, look, if you don't do what we're going to ask you to do for the next 10 weeks, based on this blood work and the medical records that we pulled for the previous 10 years and the demographic data we pulled for 10 years, you have a life expectancy of 10.4 years. You know, for a 52-year-old man to realize that he's not going to make it out of his 60s, a big realization. And he flicked a switch, a level of discipline that, you know, I haven't seen in a patient in a long time. He goes, dude, I'll do whatever you tell me to do. So we wrote a prescription ketogenic diet. I I'm a fan of the keto diet. I don't think everybody needs to be on the keto diet, but, um, um, by prescription ketogenic diet, we wrote a keto diet right down to the grocery list, keto reset diet. And I said, if it's literally, if it's not on here, you can't eat it, Dana. This is your grocery list. You go to the store, you buy this. You send your chef to the store to buy this. You make this. If it's not this recipe, if it's not on here, you literally can't eat it. Your only leeway is water and the supplements. And we started a process of balancing hormones, controlling his glycemic index, of using amino acids to bring down his level of homocysteine to actually try to fix the insulin resistance to reduce his triglycerides and in 10 weeks he had such a material change in his blood work i forget how much weight he had lost i think he had lost almost 28 or 30 pounds at times he's over 40 pounds now. By the end of the fifth month, he was completely off of every prescription medication he was on. He's down 44 pounds. He lost the, you know, he's no longer using the CPAP machine. He no longer is pre-diabetic. He no longer has insulin resistance. He no longer has life-threatening levels of triglyceride. In fact, they're normal. His kidney function improved. His liver function improved. His immune system strengthened. He feels like a 35-year-old man again. His skin tone all improved. His blood pressure returned to normal. He's not on any blood pressure medication. So his blood pressure returned to normal. And he was like, dude, I had no idea I could feel this good. I feel freaking amazing. And his life expectancy? Almost tripled. Almost tripled? Almost tripled almost tripled almost tripled just under 30 years when i heard the story about dana white and i saw he had gone from respectfully being a man that had a little bit of weight to having this these six-pack abs on instagram of course the six-pack isn't the the outcome it's as you said, it's the stuff going on inside him. That's really the transformation. It left me with the question, like, okay, I heard the keto bit, but what can someone who's just heard that at home, where do they start with getting, extending their life by triple and getting the... So, you know, he also started something called the superhuman protocol. And superhuman protocol is using magnetism, oxygen, and light, right? So the only things that we really get from Mother Nature, the big benefit we get from Mother Nature is we get magnetism from the earth, we get oxygen from the air, we get light from the sun. The truth is most of us are not contacting the surface of the earth that much anymore. So he bought $150,000 worth of equipment, a PMF mat, an oxygen, what's called a Hypermax oxygen to do exercise with oxygen therapy, and a red light therapy bed. And I had him use that equipment every single day, seven days a week. But if your listeners want to do it for free, you can take off your shoes and contact the surface of the earth. And I'm talking about bare feet on soil, dirt, grass, sand, because earthing and grounding is a very real thing. We actually discharge into the earth. We actually, human beings build up a charge. Do you know that pH, the acid alkaline scale, pH stands for potential hydrogen. It's a charge. It's a complete fallacy that you can get alkaline by drinking alkaline water. That's the biggest marketing myth ever sold to the public. But you can get alkaline by contacting the surface of the earth. So if you don't have 150 grand, which I don't expect anybody listening to this podcast to spend 150 grand, but he did. I said, you need a PMF mat so you can be alkaline. You need to spend 10 minutes a day breathing 95% O2 under mild exercise, and you need to lay in a red light therapy bed. So in the absence of the superhuman protocol, you can become superhuman by contacting the earth and by learning to do breath work. Let's talk about breath work. I spend eight minutes every day doing a very specific series of breath work, and I'll teach it to you now. You said your wife has certified him? Yeah, my partner, she's a breath work practitioner. Oh, your partner. I've done breathwork with her. I've done breathwork with a few people, but no one's ever had the profound impact on me through breathwork that she has. I've never shouted her out before, so I probably should. Her Instagram is at M-E-L-O-A-I for anybody that's interested in breathwork. People do not realize the power of something that is so accessible, so free, and so easy to do, right? They want things to be more complicated, but it's not. And when I said the presence of oxygen is the absence of disease, it's absolutely true. Remember that every elevated emotional state that a human being can experience actually has in its molecular structure, oxygen is a component of that emotion. So if you look at the difference between passion, elation, joy, arousal, libido, and anger, for example, it's usually only one neurotransmitter and the presence of oxygen. The reason why no human being has ever woken up laughing is because you don't have the oxidative state to experience laughter right out of deep sleep. But can you wake up angry? Yes, because anger doesn't require oxygen. So every morning contact the surface of the earth and then spend eight minutes doing, I do a Wim Hof style of breath work. I give credit where credit's due. He's the father of breath work as far as I'm concerned. So I do three rounds of 30 deep breaths, like obnoxiously deep breaths. And I start by trying to take my belly button and pull my belly button out towards the wall imagine there's a string pulling your belly button towards the wall and then you you fill from the lobes of the lung to the apex of the lung and then you exhale and just relax God knows what they think we're doing out there. Right outside this podcast. They're like, a bunch of freaks. I knew it was a cult. I knew he was a cult leader. So you do three rounds of 30 breaths. On the 30th breath, you exhale and you hold. Allow the carbohydrate receptor to reset. When you don't feel you can hold anymore, you take a deep breath in. You hold again and you let it out slow. And you start again. I would suggest that you start with three rounds of five breaths. Then work to 10, 15, 20, 25, and 30. If you get lightheaded, this is a good sign that the oxygen tension is changing in your brain. If your fingers and toes get tingly, this is a good sign that the oxygen tension is changing in your brain if your fingers and toes get tingly this is a good sign that you're changing the oxygen tension if you feel some kind of heat temperature change in your neck these are all great signs you will get to the point where you can actually hold your breath for two or three minutes sometimes four minutes between rounds of breath work um and then the last thing is to expose yourself to natural sunlight. First thing in the morning, the first 45 minutes of the day, God gives us a very, very special type of light. It's called first light. There's no UVA. There's no UVB rays in this light. So that it's not the damaging rays from the sun. It still generates vitamin D3. It has a positive effect on cortisol, on vitamin D3. First light is the best way to reset your circadian rhythm. So by contacting the surface of the earth, doing breath work and getting first light, you can get to the same place that Dana White did with 150 grand in equipment. What about oxygen masks? Because I'll be honest, when I read about the Dana story, I went on Amazon soon after and I was like, I'm just going to buy an oxygen canister. Good idea, bad idea? So what you want to do is, you know, you get an oxygen concentrator, which takes 21% oxygen from, which is what the concentration at sea level, it turns it into 95% O2 and it fills this bag and it can refill this bag over and over and over again. Okay. I use one called the Hypermax. You can see it on my Instagram and you plug it in, you turn it on, it fills this bag. And then you go in, you put an oxygen mask on, and you exercise for 10 minutes, only 10 minutes. Cycle for three minutes, sprint for 30 seconds, cycle for three minutes, sprint for 30 seconds, cycle three minutes, sprint 30 seconds, and you're done. And what this does is it raises something called the partial pressure, the storage of oxygen in your blood. is it raises something called the partial pressure, the storage of oxygen in your blood. The only two-time Nobel laureate prize winner in medicine, Dr. Otto Warburg, won both of his Nobel prizes for his work in exercise with oxygen therapy. You want to be a superhuman, do mild exercise every day while breathing 95% O2. It's important that you're exercising. And then after that, you move into a red light therapy bed, photobiomodulation. So, you know, if you don't have access to a hypermax oxygen machine, just do the breath work. Get the breath in, you know, exchange the oxygen tension in the tissues and expose yourself to first light. What about cold water plunging? So I'm a huge fan of cold water plunging, but probably not for the reasons why you think. You know, I also sit on the board of the NFL Alumni Association Athletica as a health service director. There was a time when we used to think that putting athletes in cold water after exercise was good because of its anti-inflammatory effects. We know now that that's only about 15% of the benefit. The majority of the benefit comes from something called a cold shock protein. If you really want to be fascinated, Google cold shock proteins. These are reserve proteins that are reserve proteins that are in your liver they're dumped into the bloodstream in an effort to save your life when you put yourself in cold water they scour the body of free radical oxidation they increase the rate of protein synthesis muscle repair they are free you get them when you put yourself in cold water i don't know what the celsius conversion is but I use 50 degrees for three minutes minimum, six minutes maximum. Cold. Yes. It's actually not that cold. I mean, you know, I see people getting in 37, 38 degree water. There's no evidence that I've read that shows that colder is better. You get a peripheral vasoconstriction, so it forces all the oxygen into the core and up to the brain. And you get an activation of something called brown fat, right? Thermogenesis comes from brown fat. And for the women that are listening, for some reason, I seem to ensnare the women when I say this. Remember that the definition of a calorie is a measure of heat, right? I mean, the definition of a calorie is the amount of energy it takes to raise one cubic centimeter of water, one degree centigrade. So if a calorie is a measure of heat, then this means that when heat's leaving your body, calories are leaving your body. So if there is nothing, nothing, no amount of exercise hits cardio, no type of cardiovascular or weight training that comes anywhere close to immersing yourself in cold water in terms of what will strip fat off your body fast. If you want to strip fat off your body, get in cold water three to six minutes a day. That's fascinating. Because the oxygen rushes to my head, that's why it has a really profound impact on mood. That's why it has a very profound impact on mood. Because if you think about it, what's the reason why we need deep sleep? What happens in deep sleep that's so special? There's a secondary oxygen transfer. We transfer oxygen from the periphery, from the extremities to the brain. Remember, the brain's a non-metabolic organ so in other words it's unlike a muscle if i pick up a weight and start to work out my muscle my arm my body will send more blood more amino acids more oxygen to that muscle because it's working well if i'm sitting at my computer and i'm watching reruns of the simpsons or i'm sitting in my computer and i'm solving the most complex joint venture agreement, partnership agreement with all kinds of mathematical equations, my brain gets the same amount of nutrients, same amount of blood flow, same amount of oxygen. So it eats the same meal, whether or not it's in a dead sprint or whether or not it's just chilling on the couch, except in deep sleep and when you're in cold water because it's forcing the oxygen up to the brain. You said earlier about comfort. Yes. I was speaking to someone yesterday about this thing called, he referred to it as the comfort crisis and how as we've become more, I would say civilized, but I don't know if that's the right terminology, as we've become more advanced technologically as humans, we can make our lives increasingly more comfortable. Correct. Sounds like a good thing. Terrible. It accelerates aging in every form. I mean, aging is the aggressive pursuit of comfort. We have got to stop telling grandma not to go outside. It's too hot. Not to go outside. It's too cold. Just to lay down. Just to relax. To eat at the very first pang of hunger. This is collapsing all of our own natural defense mechanisms. You know, if we don't load our bones, they don't strengthen. If you don't tear a muscle, it doesn't grow. If you don't challenge the immune system, it weakens. And so stress is very often very good for the body. Thermal stress, weight-bearing exercise, breath work, these things put stressors into the body that are very, that have a very positive effect at strengthening you. We want to regulate everything now. We regulate our temperature. We go from a, you know, a temperature-controlled office to a temperature-controlled car to a temperature-controlled home. You know, we don't, we don't thermoregulate anymore. I mean, you know, usually when you, when I ask people to start taking cold showers, they take their first cold shower, they never do it again. Why? Because they don't want to be uncomfortable. And so when you learn to deal and become comfortable with being uncomfortable, this is like a metaphor for life. It's almost like yoga. If you've ever done really intense yoga and you're holding a yoga pose and you're trying to remain calm and focus on your breath while your body's in intense pain. Now you're not in any risk, but your ass feels like it's going to peel off your legs and your hamstrings are firing and you're sweating and you're shaking and you're doing this thing that's called the candlestick, but it's really painful. And if you can maintain calm and breathe through a situation like that, what happens four hours later when you get a nasty Instagram message? Nothing. It doesn't shift your mood. And if we don't learn to control our emotional state, we will never control our future. You know, MIT did an incredible clinical study that showed that the amygdala of the brain, which is where we experience emotion, is the sole gateway to an area of the brain called the hippocampus, which is where we experience emotion, is the sole gateway to an area of the brain called the hippocampus, which is where we hold our memories. So just imagine that the emotional center of the brain is the sole gateway to the memory of the brain. This is why if you've ever had an argument with your spouse, you can always recall with incredible accuracy every other time they've made you feel this way. You know, you did this on September 21st. You did this when we were on the boat with my boys. You did this at, you know, our Christmas holiday party four Christmases ago because that emotion is linked to that memory. So you can recall that memory very accurately. Well, our memory, our hippocampus is what projects into the prefrontal cortex. It determines our future. It's our conscience. So this means if emotion is the only gateway to memory and memory projects to our conscience, which is our future, this means that your current emotional state determines your future. That's a biophysiologic fact. So like, for example, if you had an argument with your spouse on the way to work and you get out of the car and you slam the door and you walk into the office, when you break the plane of the door of that office, the only memories you can recall about the office at that moment are negative. You're gonna walk through the door of the office, you can be like, they don't respect me around here. I'm gonna have a stern talk into management today. You know, my office better not, you know, nobody better be at my desk. And you know what, Mary better not run into me today because she doesn't respect me. You can just start going through all the negative things about the office. The office didn't do anything to you. How do I prevent that? You learn to control your emotion. How? Well, first you start by putting the right nutrients into the body that allows you to achieve elevated emotional states. And you learn to do things like when you feel like you are beginning to lose control of your emotional state, you actually break that cycle. I usually do it with breath work. And so, first it begins by having the right raw materials, but this is just taking you back to the cold punch. If you can start your day in an elevated emotional state, if anybody listening to this has ever really done a cold punch, tell me if you were ever in a bad mood getting out of a cold punch. Just try to be in a bad mood getting out of a cold punch. They say if you want to cure depression, push somebody in cold water. You know? And it's so true. You're in such an elevated emotional state. You're like, wow. Now you go cruising into the day and get a little negative, you know, Instagram message and your spouse calls you and tells you she forgot what you wanted to get at the grocery store and you get to work and you got a little problem at the office. These things roll off your back instead of shifting your state, which now shifts your memory, which now changes the trajectory of your prefrontal cortex, which affects your future. I do a lot of traveling, Gary. I travel all over the world all the time. Oh, it's one of my favorites. Yeah, you travel a lot? Tons. You've got a bit of a system for traveling because when I travel, I feel like shit. Oh, I feel amazing when I travel. And I post all about it on my Instagram. You know, all I do is teach on Instagram. But you know, I went Miami, Atlanta, New York, London, Stade, Switzerland, Dubai, Dubai, Miami, Miami, Vegas. I got up at four o'clock this morning to come here. I mean, I landed here at I think 7.15 this morning. And you're flying out of here. I'm flying out of here now. In five here. I'm flying out of here now. In five minutes. I'm flying out of here in five minutes and I'm going on a red eye back to Miami. And you feel good when you travel and I feel like shit. What do you think we're doing differently? If you were to guess. Okay, so there's three things that you can do when you travel. And first and foremost, and I don't know why anybody talks about this, is that everybody talks about waking with the sun, which I'm a big believer in, or forcing yourself to stay up to try to get onto a new time zone. When you change time zones, the single most important thing that you can do is preserve your sleeping window. Do not eat during your normal sleeping window. Let me tell you what I mean. Let's say that you're on the East Coast, you live in New York, and you go to bed at 10 and you get up at 6 a.m. Okay. You go to bed at 10 p.m., get up at 6 a.m. Most people go to bed at midnight, get up at 6 a.m. So let's say you go to bed at 10, you get up at 6 a.m. And now you go to London. Okay. So now London is, depending on the time of the year, six hours ahead. If you eat during 10 p.m. to 6 a.m. New York time, there is zero chance you will adjust to that time zone. We are more tied to our digestion in terms of our circadian rhythm than we are to the sleep-wake cycle of the sun. So in other words, if I fly to London and I start eating when it's 3.30 in the morning my time, my body body goes what the heck are you doing we're having steak and eggs and a champagne it's 3 30 in the morning right your circadian rhythm is screwed up so shift your sleeping window and preserve that sleeping window in your new time zone and do not eat during those times so in other words in london that would be um 6 a.m to noon so be 6 a.m. to noon. So between 6 a.m. and noon, I'm not going to eat. I'll have coffee or I'll have water or fluids, but I will not start to eat until noon. For how long do I preserve that window? Well, depending on how long you're going to be there, usually I'm only, if you're there a week or less, preserve your sleeping window the entire week that you're there. When you're there for more than 7 to 10 days, then you need to really adjust to that time zone. What happens physiologically if I don't? If you don't, you will irrevocably mess up your circadian rhythm. I mean, just imagine it. So let's say you lived in New York and you didn't travel. You go to bed at 10 and you wake up at 6 a.m. Try for three nights setting alarm for 3.30 in the morning, get up, need a big breakfast at 3.30 and try to go at 6 a.m. Try for three nights setting alarm for 3.30 in the morning, get up, eat a big breakfast at 3.30 and try to go back to bed. And watch what happens to your sleep cycle, right? You'll destroy it. So we don't, we preserve our sleeping window. The other thing is I fast on domestic flights. I don't care what time or where I'm going. If I fly anywhere in the continental United States, I fast on airplanes. So I allow myself, you know, water,. I hydrate and I have black coffee on international flights. I just came back from Dubai. It was 16 hours. I ate on that plane. And then for flights that are more than an hour, every hour on the hour, I get up out of my seat. I don't care how weird it looks. I go to the back of the plane. I do 25 air squats. I go either into the bathroom or in the back of the plane. I do 25 deep breaths. On a 10-hour flight, I'll do 250 deep breaths and 250 air squats over the course of a 10-hour flight. You feel amazing. And then I eat fats and proteins on flights. Carbohydrates at altitude are terrible for you, and it's usually where all the salt hides. Remember that there are essential fatty acids, meaning they're essential for life. There's two of them. If you don't get these fatty acids, you'll die. There's nine essential amino acids. They're proteins. They're essential for life. If you don't get these nine essential amino acids, you'll die. There is no such thing as an essential carbohydrate. Why are carbs so bad in the air? Carbs are bad in the air because as soon as you divert blood from your brain to digestion, let's not forget it's a 30-foot long tube. The higher the consumption of carbohydrate, the more blood floods to your gut. So you're going to feel like crying. So now you're seated and your gut is flooded with blood. It's all come from your brain. So now you're tired. You're not focused. You're exhausted. And you're seated. So this isn't a good place to be tired because you're not going to get good sleep. So I set a huge priority on energy. Energy is a huge priority to me. So when I look at food, I look at it two ways. It's going to serve me, it's going to steal from me. When I'm flying first class back to Miami tonight, it's a red eye. I'm going to actually use that time to sleep. I'm not going to waste it on eating because I know that, first of all, I'm past my feeding window now because I preserved my East Coast time feeding window. And this is what keeps the train running on time. What is energy? You said energy though, what is energy? Energy is oxygen in your blood. Everything that you perceive about energy is nothing more than oxygen in your blood. If you told me, Gary, I had a lot of energy today. Physiologically, what you're saying is, I had a lot of oxygen in my blood today. Oxygen equals energy. At the start of this conversation, I asked you a question. I said, why should people listen to your message? We're now at the end of the conversation. What have I missed that is pertinent, important to your message that we haven't discussed? You know, I feel like I could talk about this forever. Like to me, I feel like the podcast is just getting going. You know, I believe in human beings and the ability of the body to heal itself. I believe in the power of the mind and frequency and the power of this to heal this. I guess my message would be, and I don't think that you've missed anything. My message would be that optimal health is found in the basics, not in the complicated, fancy neurotropics or some rare root that's buried deep in the Amazon jungle. It's found in the basics. The further we get away from the basics, magnetism, oxygen, light, whole foods, the more unhealthy we become. And, you know, my message, what I try to teach on Instagram and what the message that I get to the I get to the world is not that you need a lot of fancy equipment. Sure, if you can afford it, you can have it, but if you get back to the basics with Mother Nature and back to the basics with our foods, a good rule of thumb is if your great-grandmother wouldn't recognize it, don't eat it. Then you'll find a state of optimal health that's beyond anything that you thought imaginable. There's a superhuman inside of everybody listening to this podcast. Gary, we have a closing tradition on this podcast where the last guest asks a question for the next guest, not knowing who they're asking it for. The question that's been left for you is, what is the unobvious thing that you struggle with? What is the unobvious thing that I struggle with? What is the unobvious thing that I struggle with? You know, I believe that I'm so committed to my craft and I'm so committed to being authentic that I struggle sometimes like everyone else when I have an extra burden of guilt whenever I want to just have any kind of marginal enjoyment, right? Like if I just want to have a cocktail or I want to eat some birthday cake, which I know is not going to hurt me and it's not going to throw me off, but I really struggle with that. I feel like I'm letting the whole world down when i do that um and i know that it's not and it's ridiculous and you shouldn't be that you know being that discipline actually is a you know it's not sustainable over over a long period of time um i think that like a lot of people listening to this podcast i listening to this podcast, I'm very hard on myself. I'm my own worst enemy sometimes. And I have an insatiable appetite to do what I'm doing. And I think that if I just wasn't so hard on myself, I probably would find it a lot easier. Thank you so much. Everybody listening to this should go check out 10x Health. Much of the information you've discussed here lives within that ecosystem and your Instagram and your website and those channels, which I've explored, you know, in depth are incredible resources to understand how to start your journey to living a more healthy life. And that's, that's everything that you espouse. You're incredible, Gary. Thank you so much. And you know, it's very, very rare that I find someone online. then dm them and I nagged you on dm for a couple of months to get you here but but I think everybody listening to this can understand why um you're doing incredible work incredibly important work an incredibly important time to shift a narrative and um as I said to you um I think before we start recording or in an interval I realized that this is literally just the start for you in the journey and the mission that you're on. That's so clear to me. Thank you. I feel the same way. I've now been a Huel drinker for about four years, roughly, so much so that I ended up investing in the company. And I play a role on the board of the company, but they also very kindly sponsor this podcast. And to be honest, I've never said this before, but Huel believed in this podcast before anybody else. The CEO, Julian, told me before we even launched the podcast how successful it would be and that Huel would back it. And I absolutely have a huge amount of gratitude for them for that support, but an even greater sense of gratitude for the fact that they've helped me stay nutritionally complete throughout the chaos and hecticness of my tremendously busy business schedule. So if you haven't tried out Huel, which I hope most of you have at least given it a go by now, try it out. It's an unbelievable way to try and stay nutritionally on course if you have a hectic, busy schedule. And let me know what you think. Send me a tweet and a DM, tag me. Let me know what you think. Thank you."
with open ('whisper_vL3_3.txt', 'w+') as f:
    f.write(whisper_vL3_3)